## Scripts for scrutinizing the datasets

In [ ]:
#! /usr/bin/env python3

import os
import shutil
import csv
from collections import namedtuple
from datetime import datetime
import urllib.request
import re

import openpyxl


In [ ]:
def list_all_sheets(workbook):
    # non-essential sanity check function
    sheets = [sheet for sheet in available_wb.get_sheet_names()]
    print(sheets)

In [ ]:
def show_all_info(urn):
    # non-essential script that prints all the info related to a urn.
    if urn in main_sheet:
        print("Main example:", main_sheet[urn], "\n")
    if urn in filenames_sheet:
        print("Filenames example:", filenames_sheet[urn], "\n")
    if urn in keywords_sheet:
        print("Keywords example:", keywords_sheet[urn], "\n")
    if urn in advisors_sheet:
        print("Advisors example:", advisors_sheet[urn], "\n")
    if urn in catalog_sheet:
        print("Catalog example:", catalog_sheet[urn], "\n")

In [ ]:
def show_combinations_of_advisors(advisors_sheet):
    urn_advisortitles = dict()
    for urn, advisors_nt_list in advisors_sheet.items():
        for item in advisors_nt_list:
            if item.urn in urn_advisortitles:
                urn_advisortitles[item.urn].append(item.advisor_title)
            else:
                urn_advisortitles[item.urn] = [item.advisor_title, ]

    a_set = set()
    for urn, titles in urn_advisortitles.items():
        for title in titles:
            a_set.add(title)
    print(a_set)

    advisors_permutations = set()

    for urn, titles in urn_advisortitles.items():
        this_permutation = (titles.count('Committee Chair'),
                            titles.count('Committee Co-Chair'),
                            titles.count('Committee Member'),
                            titles.count("Dean's Representative"),
                            )
        advisors_permutations.add(this_permutation)
    for i in advisors_permutations:
        print(i)
    return advisors_permutations

In [ ]:
def find_mismatching_files(filenames_sheet):
    sames = dict()
    for urn, filenames_namedtuple_list in filenames_sheet.items():
        for item in filenames_namedtuple_list:
            if item.urn in sames:
                if sames[item.urn] != item.availability:
                    print('there should be one {}'.format(item.urn))
            else:
                sames[item.urn] = item.availability
    return sames

In [ ]:
def find_misnamed_extensions(filenames_sheet):
    misnamed_urn_filename = []
    for urn, filenames_namedtuple_list in filenames_sheet.items():
        for item in filenames_namedtuple_list:
            if item.filename[-4] != "." and item.filename[-4:] not in ("docx", "r.gz"):
                misnamed_urn_filename.append((urn, item.filename))
                print(urn, item.filename)
    return misnamed_urn_filename

In [ ]:
def find_legacy_school_names(main_sheet):
    schools_etds = dict()
    for urn, itemnamedtuple in main_sheet.items():
        if itemnamedtuple.department in schools_etds:
            schools_etds[itemnamedtuple.department].append(urn)
        else:
            schools_etds[itemnamedtuple.department] = [urn, ]
    for school, urns in schools_etds.items():
        print(school)
    return schools_etds

In [ ]:
def find_page_by_page_pdfs(filenames_sheet):
    split_files = dict()
    for urn, filenames_namedtuples_list in filenames_sheet.items():
        for item in filenames_namedtuples_list:
            if item.urn not in split_files:
                split_files[item.urn] = [item.filename, ]
            else:
                split_files[item.urn].append(item.filename)
    page_by_page_pdfs = []
    for urn, filelist in split_files.items():
        split = False
        for i in filelist:
            if "chap" in i.lower():
                split = True
        if len(filelist) > 1 and split == True:
            print(urn, '\n', filelist, '\n')
            page_by_page_pdfs.append((urn, filelist))
    return page_by_page_pdfs

In [ ]:
def is_catalog_subset_of_database(catalog_sheet, main_sheet):
    outside_uris = []
    for uri in catalog_sheet:
        if uri not in main_sheet:
            # print(uri)
            outside_uris.append(uri)
    print(len(outside_uris))
    return outside_uris

In [ ]:
def are_all_urns_in_main_sheet(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    all_urns = make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet)
    main_urns = set(i for i in main_sheet)
    if (all_urns - main_urns) != set():
        print(all_urns - main_urns)
    else:
        print('all urns in mainsheet')

In [ ]:
# do all files have a pdf, (or no file at all).
def check_for_no_file_objects(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    no_files = []
    for urn in make_set_all_urns(main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
        pdf = False
        if urn not in filenames_sheet:
            no_files.append(urn)
            continue
        for nt in filenames_sheet[urn]:
            if 'pdf' in nt.filename.lower():
                pdf = True
        if pdf == False:
            print(urn)
    print('these have no uploaded files:', no_files)

## Reading & parsing source files

In [ ]:
test_items = ('etd-0807101-102716',)

In [ ]:
def parse_main_sheet(filepaths):
    csv_dict = dict()
    for filepath in filepaths:
        with open(filepath, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';', quotechar='"', escapechar='\\')
            for num, row in enumerate(spamreader):
                if num == 0:
                    headers = [*row]
                    CSVSheet = namedtuple('CSVSheet', headers)
                    continue
                item = CSVSheet(*row)
                if item.urn in csv_dict:
                    for k, v in item._asdict().items():
                        if v != item._asdict()[k]:
                            print('duplicate item')
                            print(item.urn)
                            break
                csv_dict[item.urn] = item
    return csv_dict

In [ ]:
def parse_filenames_sheet(filepaths):
    csv_dict = dict()
    for filepath in filepaths:
        with open(filepath, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';', quotechar='"', escapechar='\\')
            for num, row in enumerate(spamreader):
                if num == 0:
                    headers = [*row]
                    CSVSheet = namedtuple('CSVSheet', headers)
                    continue
                item = CSVSheet(*row)
                if item.urn not in test_items:
                    if not item.urn:
                        print("{} has no urn".format(item))
                    if item.urn not in csv_dict:
                        csv_dict[item.urn] = [item, ]
                    else:
                        if not item.timestamp or not item.filename:
                            continue
                        row_timestamp = datetime.strptime(item.timestamp, "%Y-%m-%d %H:%M:%S")
                        filenames_list = [i.filename for i in csv_dict[item.urn]]
                        if item.filename in filenames_list:
                            previous_filename_entry = [i for i in csv_dict[item.urn] if i.filename == item.filename]
                            previous_timestamp = datetime.strptime(previous_filename_entry[0].timestamp, "%Y-%m-%d %H:%M:%S")
                            if row_timestamp > previous_timestamp:
                                previous_filename_entry[0] = item
                        else:
                            csv_dict[item.urn].append(item)
    csv_dict = sort_descending_size(csv_dict)
    return csv_dict

def sort_descending_size(filenames_sheet):
    for urn, list_of_namedtuples in filenames_sheet.items():
        list_of_namedtuples = sorted(list_of_namedtuples, key=lambda x:int(x.size), reverse=True)
    return filenames_sheet

In [ ]:
def parse_keywords_sheet(filepaths):
    csv_dict = dict()
    for filepath in filepaths:
        with open(filepath, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';', quotechar='"', escapechar='\\')
            for num, row in enumerate(spamreader):
                if num == 0:
                    headers = [*row]
                    CSVSheet = namedtuple('CSVSheet', headers)
                    continue
                item = CSVSheet(*row)
                if item.urn not in test_items:
                    if item.urn not in csv_dict:
                        if not item.urn:
                            print("{} has no urn".format(item))
                        csv_dict[item.urn] = [item, ]
                    else:
                        csv_dict[item.urn].append(item)
    return csv_dict

In [ ]:
def parse_advisors_sheet(filepaths):
    csv_dict = dict()
    for filepath in filepaths:
        with open(filepath, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';', quotechar='"', escapechar='\\')
            for num, row in enumerate(spamreader):
                if num == 0:
                    headers = [*row]
                    CSVSheet = namedtuple('CSVSheet', headers)
                    continue
                item = CSVSheet(*row)
                if item.urn not in test_items:
                    if not item.urn:
                        print("{} has no urn".format(item))
                    if item.urn not in csv_dict:
                        csv_dict[item.urn] = [item, ]
                    else:
                        row_timestamp = datetime.strptime(item.timestamp, "%Y-%m-%d %H:%M:%S")
                        if item.advisor_name in csv_dict[item.urn]:
                            previous_advisor_entry = [i for i in csv_dict[item.urn] if i.advisor_name == item.advisor_name]
                            previous_timestamp = datetime.strptime(previous_advisor_entry[0].timestamp, "%Y-%m-%d %H:%M:%S")
                            if row_timestamp > previous_timestamp:
                                previous_advisor_entry[0] = item
                        else:
                            csv_dict[item.urn].append(item)
    return csv_dict


In [ ]:
def parse_catalog_sheet():
    """ returns a dictionary in form of:
        {urn: NamedTuple
         urn: NamedTuple}
    """
    catalog_sheet = dict()
    sourcepath = 'Aug2016/Catalogtables'
    sourcefile = 'ETDCatalogRecords20161108.csv'
    with open(os.path.join(sourcepath, sourcefile), encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='|')
        for num, row in enumerate(csvreader):
            if num == 0:
                headers = (i for i in row)
                Catalog = namedtuple('Catalog', headers)
                continue
            values = (i for i in row)
            item = Catalog(*values)
            urn = [i for i in os.path.split(item.URL) if 'etd-' in i]
            urn = os.path.split(urn[0])[1]
            if urn not in test_items:
                catalog_sheet[urn] = item
    return catalog_sheet

## Building the csv

In [ ]:
def csv_writer(data, path):
    with open(path, "w", newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for line in data:
            writer.writerow(line)

In [ ]:
def concatinate_keywords(keywords_sheet, urn):
    if urn in keywords_sheet:
        return ', '.join(str(nt.keyword).strip() for nt in keywords_sheet[urn] if nt.keyword)
    return ''

In [ ]:
def organize_advisors(advisors_sheet, urn):
    Advisor = namedtuple('Advisor', ('urn', 'advisor_name', 'advisor_title',
                                           'advisor_email', 'approval', 'timestamp'))
    blank_Advisor = Advisor('', '', '', '', '', '')
    Advisors_nt = advisors_sheet[urn]
    advisors_rank = {'Committee Chair': 1, 'Committee Co-Chair': 2, 'Committee Member': 3, "Dean's Representative": 4}
    alpha_Advisors = sorted(Advisors_nt, key=lambda x: x.advisor_name)
    sorted_advisors = sorted(alpha_Advisors, key=lambda x: advisors_rank[x.advisor_title])
    if len(sorted_advisors) > 7 and "Dean's Representative" in sorted_advisors[-1]:
        sorted_advisors = sorted_advisors[:6] + sorted_advisors[-1:]
    elif len(sorted_advisors) > 7:
        sorted_advisors = sorted_advisors[:7]
    elif len(sorted_advisors) < 7:
        missing = 7 - len(sorted_advisors)
        for i in range(missing):
            sorted_advisors.append(blank_Advisor)
    return sorted_advisors

In [ ]:
def strip_slash_and_padding(text):
    if not text:
        return
    text = str(text).strip()
    if text[-1] == '/':
        text = text[:-1]
    text = text.strip()
    return text

In [ ]:
def replace_null_with_nothing(text):
    if not text:
        return ''
    return text.replace('NULL', '')

In [ ]:
def combine_title(catalog_sheet, main_sheet, urn):
    if urn in catalog_sheet:
        title = catalog_sheet[urn].Title
        title = strip_slash_and_padding(title)
        subtitle = catalog_sheet[urn].Subtitle
        subtitle = strip_slash_and_padding(subtitle)
        if title[-1] == ':':
            title = title[:-1]
        if subtitle:
            csv_title = "{}:  {}".format(title,
                                         subtitle)
        else:
            csv_title = title
    else:
        title = main_sheet[urn].title
        title = strip_slash_and_padding(title)
        csv_title = title
    return csv_title

In [ ]:
def remove_non_nums(text):
    if not text:
        return ''
    only_nums = ''
    for i in text:
        if i.isnumeric():
            only_nums += i
    return only_nums

In [ ]:
def find_pub_date(urn, catalog_sheet, main_sheet):
    if urn in catalog_sheet:
        if catalog_sheet[urn].SeriesDate:
            return remove_non_nums(catalog_sheet[urn].SeriesDate)
        if catalog_sheet[urn].PubDate:
            return remove_non_nums(catalog_sheet[urn].PubDate)
    return remove_non_nums(main_sheet[urn].ddate)[:4]

In [ ]:
year_month_day = re.compile(r'^(\d{4})[/.-](\d{1,2})[/.-](\d{1,2})$')
year_only = re.compile(r'^(\d{4})$')

In [ ]:
def find_defense_date(main_sheet, urn):
    if main_sheet[urn].ddate:
        return main_sheet[urn].ddate
    return ''

def find_submission_date(main_sheet, urn):
    if main_sheet[urn].sdate:
        return main_sheet[urn].sdate
    return ''

In [ ]:
Filenames = namedtuple('Filenames', ['urn', 'filename', 'path', 'size', 'page_count', 'availability', 'description', 'timestamp'])
special_cases = {'etd-07062015-144158': Filenames(urn='etd-07062015-144158', filename='Moran_Thesis.pdf', path='/export/ETD-db/docs/withheld/etd-07062015-144158/withheld', size='1684241', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-07-07 17:15:34'),
                 'etd-07062015-144155': Filenames(urn='etd-07062015-144155', filename='Morris_Thesis.pdf', path='/export/ETD-db/docs/submitted/etd-07062015-144155/withheld', size='805581', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-07-08 08:59:42'), 
                 'etd-04032015-080642': Filenames(urn='etd-04032015-080642', filename='wang_diss.pdf', path='/export/ETD-db/docs/withheld/etd-04032015-080642/withheld', size='6969877', page_count='NULL', availability='withheld', description='NULL', timestamp='2015-04-13 14:39:16'),
                 'etd-08082016-164729': Filenames(urn='', filename='', path='', size='', page_count='', availability='', description='', timestamp=''),
                 'etd-06062010-192030': Filenames(urn='', filename='', path='', size='', page_count='', availability='', description='', timestamp=''),
                 }

def find_thesis(filenames_sheet, urn):
    if urn in special_cases:
        return special_cases[urn]
    largest_pdf_file = None
    for file in filenames_sheet[urn]:
        extension = file.filename.split('.')[-1].lower().strip()
        if extension != 'pdf':
            continue
        if not largest_pdf_file:
            largest_pdf_file = file
        elif int(file.size) > int(largest_pdf_file.size):
            largest_pdf_file = file
    if largest_pdf_file:
        return largest_pdf_file
    else:
        return Filenames(urn='', filename='', path='', size='', page_count='', availability='', description='', timestamp='')

In [ ]:
def parse_dropbox_url(urn, filenames_sheet):
    thesis_file = find_thesis(filenames_sheet, urn)
    if thesis_file:
        return 'https://examplewebsite.com/{}/{}'.format(urn, thesis_file.filename)
    else:
        return ''
    
def parse_gdrive_to_url():
    gdrive_dict = dict()
    for urn, FilenamesNT in special_cases.items():
        if FilenamesNT.filename == '':
            gdrive_dict[urn] = ''
    with open('data/gdrive_to_url.csv', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for urn, name, url in csvreader:
            if urn in special_cases:
                if name == special_cases[urn].filename:
                    gdrive_dict[urn] = url
                continue
            elif urn not in filenames_sheet:
                gdrive_dict[urn] = ''
                continue
            elif name == find_thesis(filenames_sheet, urn).filename:
                gdrive_dict[urn] = url
    return gdrive_dict


In [ ]:
def make_set_all_urns(sheets):
    all_urns = set()
    for sheet in sheets:
        for urn in sheet:
            all_urns.add(urn)
    all_urns = set(i for i in all_urns if i)
    return all_urns

In [ ]:
overlooked_degrees = set()

def read_legacy_dept_map():
    legacy_current = dict()
    sourcepath = 'Aug2016/LegacyNames.csv'
    with open(sourcepath, encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for num, row in enumerate(csvreader):
            if num == 0:
                headers = (i for i in row)
            current, legacy = row[0].strip(), row[1].strip()
            if current and current != 'Program no longer active':
                if legacy not in legacy_current:
                    if legacy not in ('New', ):
                        legacy_current[legacy] = current
                else:
                    print(legacy, 'has two mappings')
    return legacy_current

def lookup_current_dept(legacy_dept):
    legacy_current = read_legacy_dept_map()
    if legacy_dept in legacy_current:
        return legacy_current[legacy_dept]
    else:
        overlooked_degrees.add(legacy_dept)
        return legacy_dept

In [ ]:
non_matching_degree_abbrevs = dict()

abbr_degree = {"MPT": "Master of Physical Therapy (MPT)",
    "MUP": "Master of Urban Planning (MUP)",
    "DM": "Doctor of Music (DM)",
    "MTS": "Master of Theological Studies (MTS)",
    "AuD": "Doctor of Audiology (AuD)",
    "MSEE": "Master of Science in Electrical Engineering (MSEE)",
    "MSIB": "Master of Science in International Business (MSIB)",
    "MCSM": "Master of Construction Science and Management (MCSM)",
    "PsyD": "Doctor of Psychology (PsyD)",
    "MSEM": "Master of Science in Engineering Management (MSEM)",
    "MSMSE": "Master of Science in Materials Science and Engineering (MSMSE)",
    "RhD": "Doctor of Rehabilitation (RhD)",
    "MATE": "Master of Arts in the Teaching of English (MATE)",
    "DPT": "Doctor of Physical Therapy (DPT)",
    "MSAgE": "Master of Science in Agricultural Engineering (MSAgE)",
    "PhDOtol": "PhD Otolaryngology (PhDOtol)",
    "MSHRM": "Master of Science in Human Resources Management (MSHRM)",
    "MIM": "Master of International Management (MIM)",
    "DMin": "Doctor of Ministry (DMin)",
    "MSIE": "Master of Science in Industrial Engineering (MSIE)",
    "MSISE": "Master of Science in Infrastructure Systems Engineering (MSISE)",
    "DPA": "Doctor of Public Administration (DPA)",
    "HSOP": "Doctor of Philosophy in Health Services Research (HSOP)",
    "MMatSE": "Master of Materials Science and Engineering (MMatSE)",
    "MAeroE": "Master of Aeronautical Engineering (MAeroE)",
    "MMT": "Master in Management of Technology (MMT)",
    "MSJ": "Master of Science in Jurisprudence (MSJ)",
    "MHP": "Master of Historic Preservation (MHP)",
    "DEng": "Doctor of Engineering (DEng)",
    "MBA": "Master of Business Administration (MBA)",
    "MRED": "Master of Real Estate Development (MRED)",
    "MCTE": "Master of Career and Technology Education (MCTE)",
    "MSAeroE": "Master of Science in Aerospace Engineering (MSAeroE)",
    "MAR": "Master of Arts in Religion (MAR)",
    "MST": "Master's of Science in Teaching (MST)",
    "MJS": "Master of Judicial Studies (MJS)",
    "MALA": "Master of Arts in Liberal Arts (MALA)",
    "MSETM": "Master of Science in Environmental Technology Management (MSETM)",
    "MSHTM": "Master of Science in Hospitality and Tourism Management (MSHTM)",
    "Th.M": "Master of Theology (Th.M)",
    "MSM": "Master of Science in Management (MSM)",
    "MCRP": "Master of City and Regional Planning (MCRP)",
    "MBS": "Master of Building Science (MBS)",
    "MAIS": "Master of Arts in Interdisciplinary Studies (MAIS)",
    "DBA": "Doctor of Business Administration (DBA)",
    "MPH": "Master of Public Health (MPH)",
    "MIDS": "Master of Interdisciplinary Studies (MIDS)",
    "MPA/JD": "Master of Public Administration/Juris Doctorate (MPA/JD)",
    "PhD": "Doctor of Philosophy (PhD)",
    "DMgt": "Doctor of Management (DMgt)",
    "MCIS": "Master of Computer and Information Science (MCIS)",
    "MAE": "Master of Arts in Education (MAE)",
    "MHD": "Master of Human Development (MHD)",
    "MM": "Master of Music (MM)",
    "MGS": "Master of General Studies (MGS)",
    "MSN": "Master of Science in Nursing (MSN)",
    "M.Div": "Master of Divinity (M.Div)",
    "MAC": "Master of Arts in Counseling (MAC)",
    "MCJ": "Master of Criminal Justice (MCJ)",
    "MFR": "Master of Forest Resources (MFR)",
    "MSSS": "Master of Science in Computer Science (MSCS)",
    "MSA": "Master of Science in Administration (MSA)",
    "MURP": "Master of Urban and Regional Planning (MURP)",
    "MAS": "Master in Advanced Studies (MAS)",
    "ND": "Doctor of Nursing (ND)",
    "ME": "Master of Engineering (ME)",
    "MSCRP": "Master of Science in Community and Regional Planning (MSCRP)",
    "MArch": "Master of Architecture (MArch)",
    "MLIS": "Master of Library and Information Science (MLIS)",
    "MSOtol": "MS Otolaryngology (MSOtol)",
    "MLS": "Master of Library Science/Master of Life Sciences (MLS)",
    "MSMANFE": "Master of Science in Manufacturing Engineering (MSMANFE)",
    "MSSE": "Master of Science and Software Engineering (MSSE)",
    "MEngr": "Master of Engineering (MEngr)",
    "MSB": "Masters of Science in Bioscience (MSB)",
    "PED": "Doctor of Physical Education (PED)",
    "MFA": "Master of Fine Arts (MFA)",
    "MMC": "Master of Mass Communication (MMC)",
    "MSBAE": "Master of Science in Biological and Agricultural Engineering (MSBAE)",
    "MAgEd": "Master of Agricultural Education (MAgEd)",
    "MSECE": "Master of Science in Electrical and Computer Engineering (MSECE)",
    "DMD": "Doctor of Dental Medicine (DMD)",
    "MSMatSE": "Master of Science in Material Science Engineering (MSMatSE)",
    "MAPC": "Master of Arts in Pastoral Counseling (MAPC)",
    "MSEd": "Master of Science in Education (MSEd)",
    "DPDS": "Doctor of Planning and Development Studies (DPDS)",
    "MRP": "Master of Regional Planning (MRP)",
    "MNS": "Master of Natural Sciences (MNS)",
    "EdD": "Doctor of Education (EdD)",
    "DrPH": "Doctor of Public Health (DrPH)",
    "DNS": "Doctor of Nursing Science (DNS)",
    "MSIEOR": "Master of Science in Industrial Engineering and Operations Research (MSIEOR)",
    "MAT": "Master of Arts in Teaching (MAT)",
    "MEE": "Master of Electrical Engineering (MEE)",
    "MS": "Master of Science (MS)",
    "MSECO": "Master of Science in Economics (MSECO)",
    "MLA": "Master of Landscape Architecture (MLA)",
    "PhDSurg": "PhD Surgergy (PhDSurg)",
    "MSES": "Master of Science in Engineering Science (MSES)",
    "MHI": "Masters of Health Informatics (MHI)",
    "MSME": "Master of Science in Mechanical Engineering (MSME)",
    "MMUS": "Master of Music (MMUS)",
    "MSW": "Master of Social Work (MSW)",
    "MME": "Master of Music Education (MME)",
    "DMA": "Doctor of Musical Arts (DMA)",
    "MPA": "Master of Public Administration (MPA)",
    "DA": "Doctor of Arts (DA)",
    "MApStat": "Master of Applied Statistics (MApStat)",
    "MSP": "Master of Science in Planning (MSP)",
    "MPP": "Master of Public Policy (MPP)",
    "MSExpSurg": "Medical Surgeon in Experimental Surgery (MSExpSurg)",
    "EdS": "Education Specialist (EdS)",
    "MF": "Master of Forestry (MF)",
    "MPlan": "Master of Planning (MPlan)",
    "MBT": "Master of Business Taxation (MBT)",
    "HSD": "Doctor of Health and Safety (HSD)",
    "MHRD": "Master of Human Resource Development (MHRD)",
    "MSPH": "Master of Science in Public Health (MSPH)",
    "MChE": "Master of Chemical Engineering (MChE)",
    "MSPE": "Master of Science in Petroleum Engineering (MSPE)",
    "MCompE": "Master of Computer Engineering (MCompE)",
    "MT": "Master in Taxation (MT)",
    "MAcc": "Master of Accounting (MAcc)",
    "MPM": "Master of Public Management (MPM)",
    "MSE": "Master of Science in Engineering (MSE)",
    "DME": "Doctor of Music Education (DME)",
    "DSW": "Doctor of Social Work (DSW)",
    "MSCE": "Master of Science in Civil Engineering (MSCE)",
    "DVM": "Doctor of Veterinary Medicine (DVM)",
    "MCE": "Master of Civil Engineering (MCE)",
    "MES": "Master of Environmental Studies (MES)",
    "MECom": "Master of Electronic Commerce (MECom)",
    "MHA": "Master of Health Administration (MHA)",
    "PharmD": "Doctor of Pharmacy (PharmD)",
    "MA": "Master of Arts (MA)",
    "Ded": "Doctor of Education (Ded)",
    "MEnvE": "Master of Environmental Engineering (MEnvE)",
    "ReD": "Doctor of Recreation (ReD)",
    "JD": "Juris Doctorate (JD)",
    "MSBiosyAgE": "Master of Science in Biosystems and Agricultural Engineering (MSBiosyAgE)",
    "PMBA": "Professional Master of Business Administration (PMBA)",
    "MHAMS": "Master of Historical Administration and Museum Studies (MHAMS)",
    "MSIS": "Master of Science in Interdisciplinary Studies (MSIS)",
    "IMES": "International Master of Environmental Sciences (IMES)",
    "MSChE": "Master of Science in Chemical Engineering (MSChE)",
    "MPAcc": "Master of Professional Accounting (MPAcc)",
    "MGIS": "Master of Geographic Information Science (MGIS)",
    "MBioSci": "Master of Biological Science (MBioSci)",
    "MCM": "Master of Construction Management (MCM)",
    "MSMS": "Master of Science in Medical Sciences (MSMS)",
    "MD": "Medical Doctor (MD)",
    "Medical Science": "Doctor of Philosophy (Medical Science)",
    "MGeoE": "Master of Geomechanics Engineering (MGeoE)",
    "MEd": "Master of Education (MEd)",
    "MAM": "Master in Agricultural Management (MAM)",
    "MPRTM": "Master of Parks, Recreation and Tourism Management (MPRTM)",
    "MAgr": "Master of Agriculture (MAgr)",
    }

def expand_degree_type(degree_name):
    if degree_name in abbr_degree:
        return abbr_degree[degree_name]
    else:
        if degree_name not in non_matching_degree_abbrevs:
            non_matching_degree_abbrevs[degree_name] = []
        

In [ ]:
def split_email(text):
    if not text:
        return ''
    list_a = text.split(',')
    list_b = list()
    for i in list_a:
        split_i = i.split(';')
        for n in split_i:
            list_b.append(n.split(' '))
    emails = [item for sublist in list_b for item in sublist]
    
    for i in emails:
        if 'lsu.edu' not in i and len(i) and verify_email(i):
            return i.strip().replace('\\','').replace('/','')
    else:
        fallback_email = emails[0].strip().replace('\\','').replace('/','')
        if verify_email(fallback_email):
            return fallback_email
        else:
            return ''

def verify_email(text):
    if re.match(r"^[^@]+@[^@]+\.[^@]+$", text):
        return True
    return False
    
def lookup_email(urn, main_sheet):
    pack = main_sheet[urn]
    if not pack.author_email:
        return ''
    one_email = split_email(pack.author_email)
    if not one_email:
        return ''
    if verify_email(one_email):
        return one_email
    else:
        one_email = "{}.edu".format(one_email)
        if verify_email(one_email):
            return one_email
        else:
            print('{} email didnt work {}'.format(urn, one_email))
            return ''

In [ ]:
def lookup_hide_email(urn, main_sheet):
    pack = main_sheet[urn]
    translation = {'yes': 'false', 'no': 'true', 'on': 'true'}
    yes_or_no = pack.publish_email.lower()
    return translation[yes_or_no]

In [ ]:
# current_department & department mapped -- we need department & legacy department.

def build_csv(urns_to_do, main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet):
    for papertype in ('dissertation', 'thesis', 'major_paper'):
        csv_data = []
        csvfieldnames = ["title",
                         "fulltext_url",
                         'keywords',
                         'abstract',
                         "author1_fname",
                         'author1_mname',
                         'author1_lname',
                         'author1_suffix',
                         'author1_email',
                         'author1_institution',
                         'advisor1',
                         'advisor2',
                         'advisor3',
                         "advisor1_email",
                         'advisor1_title',
                         "advisor2_email",
                         'advisor2_title',
                         "advisor3_email",
                         'advisor3_title',
                         'advisor4',
                         "advisor4_email",
                         'advisor4_title',
                         'advisor5',
                         "advisor5_email",
                         'advisor5_title',
                         'advisor6',
                         "advisor6_email",
                         'advisor6_title',
                         'advisor7',
                         "advisor7_email",
                         'advisor7_title',
                         'availability',
                         'availability_description',
                         'disciplines',
                         'defense_date',
                         'degree_name',
                         'department',
                         "document_type",
                         'file_name',
                         'file_size',
                         'hide_author_email',
                         'legacy_department',
                         'publication_date',
                         'season',
                         'submission_date',
                         'urn',
                        ]
        csv_data.append(csvfieldnames)

        papertype_urns = [urn for urn in urns_to_do if urn not in test_items and main_sheet[urn].dtype.lower() == papertype]
        for urn in papertype_urns:
#             csv_title = combine_title(catalog_sheet, main_sheet, urn)
            csv_title = strip_slash_and_padding(main_sheet[urn].title)
            csv_urn = urn
            fulltext_url = parse_dropbox_url(urn, filenames_sheet)
#             fulltext_url = gdrive_dict[urn]
            csv_first_name = main_sheet[urn].first_name
            csv_middle_name = main_sheet[urn].middle_name
            csv_last_name = main_sheet[urn].last_name
            csv_suffix = main_sheet[urn].suffix
            csv_suffix = replace_null_with_nothing(csv_suffix)
            csv_author_email = lookup_email(urn, main_sheet)
            csv_hide_author_email = lookup_hide_email(urn, main_sheet)
            sorted_advisors = organize_advisors(advisors_sheet, urn)
            csv_document_type = main_sheet[urn].dtype.lower()
            csv_degree = expand_degree_type(main_sheet[urn].degree)
            csv_legacy_department = main_sheet[urn].department
            csv_department = lookup_current_dept(main_sheet[urn].department)
            csv_disciplines = ''
            csv_keywords = concatinate_keywords(keywords_sheet, urn)
            csv_abstract = main_sheet[urn].abstract
            csv_publication_date = find_pub_date(urn, catalog_sheet, main_sheet)
            csv_defense_date = find_defense_date(main_sheet, urn)
            csv_submission_date = find_submission_date(main_sheet, urn)
            csv_availability = main_sheet[urn].availability
            csv_availability_desc = main_sheet[urn].availability_description
            csv_filename = find_thesis(filenames_sheet, urn).filename
            csv_filesize = find_thesis(filenames_sheet, urn).size
            csv_season = ''

            if urn in filenames_sheet:
                filename = filenames_sheet[urn][0].filename
            else:
                filename = ''
            csv_data.append([csv_title,
                             fulltext_url,
                             csv_keywords,
                             csv_abstract,
                             csv_first_name,
                             csv_middle_name,
                             csv_last_name,
                             csv_suffix,
                             csv_author_email,
                             'Louisiana State University and Agricultural and Mechanical College',
                             sorted_advisors[0].advisor_name,
                             sorted_advisors[1].advisor_name,
                             sorted_advisors[2].advisor_name,
                             sorted_advisors[0].advisor_email,
                             sorted_advisors[0].advisor_title,
                             sorted_advisors[1].advisor_email,
                             sorted_advisors[1].advisor_title,
                             sorted_advisors[2].advisor_email,
                             sorted_advisors[2].advisor_title,
                             sorted_advisors[3].advisor_name,
                             sorted_advisors[3].advisor_email,
                             sorted_advisors[3].advisor_title,
                             sorted_advisors[4].advisor_name,
                             sorted_advisors[4].advisor_email,
                             sorted_advisors[4].advisor_title,
                             sorted_advisors[5].advisor_name,
                             sorted_advisors[5].advisor_email,
                             sorted_advisors[5].advisor_title,
                             sorted_advisors[6].advisor_name,
                             sorted_advisors[6].advisor_email,
                             sorted_advisors[6].advisor_title,
                             csv_availability,
                             csv_availability_desc,
                             csv_disciplines,
                             csv_defense_date,
                             csv_degree, 
                             csv_department,
                             csv_document_type,
                             csv_filename,
                             csv_filesize,
                             csv_hide_author_email,
                             csv_legacy_department,
                             csv_publication_date,
                             csv_season,
                             csv_submission_date,
                             urn,
                             ])
    #     print(csv_data)
        csv_writer(csv_data, '../../scrap{}.csv'.format(papertype))

# if __name__ == '__main__':

In [ ]:
uploaded_urns = ['etd-04152010-105757', 'etd-11042005-152503', 'etd-01142010-103035', 'etd-11152014-161720', 'etd-06012016-093023', 'etd-09082005-141156', 'etd-11152013-105431', 'etd-11182010-134832', 'etd-04122013-203310', 'etd-11112015-104433', 'etd-01122006-153035', 'etd-0409103-091155', 'etd-04142009-162904', 'etd-1028102-114128', 'etd-07022007-214620', 'etd-04282011-104110', 'etd-08262008-105107', 'etd-07022015-152643', 'etd-10142015-225126', 'etd-11082011-122841', 'etd-03292004-173247', 'etd-04132005-181229', 'etd-11152012-080937', 'etd-04132015-121922', 'etd-11072014-151629', 'etd-03162005-092843', 'etd-0602103-130516', 'etd-06092015-123043', 'etd-07102016-205506', 'etd-04082015-093405', 'etd-05312004-131216', 'etd-04042004-012104', 'etd-07072014-103038', 'etd-04082016-110841', 'etd-06142006-081756', 'etd-0712102-112020', 'etd-04262011-171828', 'etd-08252014-142821', 'etd-03312006-155114', 'etd-04082008-145552', 'etd-03132007-123855', 'etd-06122007-093459', 'etd-07082004-132027', 'etd-04042005-195555', 'etd-06012012-164555', 'etd-11132008-212901', 'etd-04162009-120741', 'etd-11162015-131202', 'etd-0820103-082715', 'etd-04102007-225229', 'etd-0903102-205825', 'etd-06032004-230342', 'etd-07052005-114902', 'etd-10232007-115145', 'etd-04092009-105427', 'etd-04162009-091629', 'etd-04042007-171659', 'etd-06072013-124152', 'etd-11162005-121544', 'etd-05262006-111821', 'etd-11172006-075503', 'etd-04092008-135815', 'etd-11092004-193914', 'etd-04112013-121601', 'etd-0606102-124240', 'etd-08132012-143116', 'etd-08222009-091648', 'etd-04022013-171109', 'etd-07092015-125342', 'etd-04052016-114922', 'etd-07142008-120541', 'etd-11122008-084318', 'etd-10252010-144437', 'etd-05312013-152247', 'etd-12232004-161321', 'etd-01272014-142921', 'etd-07012011-143856', 'etd-11152010-131758', 'etd-07112016-142817', 'etd-04162009-113110', 'etd-0331103-103046', 'etd-05302008-182755', 'etd-07122006-141237', 'etd-04032005-182442', 'etd-11162005-134520', 'etd-10192013-161108', 'etd-07132006-152107', 'etd-10222007-101208', 'etd-1107103-124027', 'etd-07122007-153118', 'etd-11162015-115028', 'etd-04282015-151803', 'etd-1112103-192504', 'etd-05252016-171313', 'etd-04082013-162925', 'etd-04072005-110000', 'etd-11142005-151539', 'etd-11022011-100856', 'etd-04152013-150934', 'etd-11132007-183718', 'etd-04252012-223815', 'etd-09272013-155509', 'etd-04302010-113152', 'etd-01242005-175732', 'etd-11162005-133233', 'etd-08292011-173732', 'etd-11112008-170918', 'etd-11102009-152614', 'etd-10072013-150336', 'etd-06022004-174954', 'etd-07012011-102908', 'etd-11092009-135327', 'etd-07052016-105439', 'etd-09022009-012419', 'etd-03122005-172245', 'etd-1111103-154448', 'etd-11032005-101407', 'etd-08252005-130402', 'etd-04012004-112624', 'etd-05182010-162907', 'etd-11112004-174523', 'etd-03292004-181506', 'etd-03222005-073650', 'etd-04012014-112757', 'etd-11152006-215539', 'etd-04142014-101509', 'etd-07062004-113139', 'etd-0708102-111113', 'etd-05302016-135627', 'etd-07092008-205524', 'etd-04122005-155027', 'etd-04102013-180926', 'etd-07112014-144145', 'etd-0418102-140236', 'etd-04062014-213507', 'etd-05262015-092444', 'etd-04122012-200549', 'etd-04052005-144033', 'etd-1111102-115404', 'etd-11032004-114106', 'etd-04082005-111109', 'etd-03312009-114117', 'etd-04072013-225126', 'etd-01172004-225708', 'etd-05022010-191925', 'etd-09082004-152254', 'etd-04192012-160232', 'etd-06022009-143756', 'etd-07112014-103220', 'etd-11092011-101114', 'etd-10232014-133051', 'etd-06032011-150547', 'etd-02252011-151717', 'etd-04272011-093156', 'etd-04262012-131111', 'etd-11102005-144012', 'etd-11092015-085848', 'etd-11122007-143256', 'etd-06132005-163156', 'etd-11102006-151711', 'etd-07152005-104328', 'etd-11102009-145556', 'etd-06282005-160617', 'etd-0211103-135707', 'etd-07122014-113947', 'etd-07092004-104620', 'etd-11082004-112330', 'etd-05202013-105033', 'etd-06092010-143509', 'etd-01142015-153345', 'etd-01272011-151106', 'etd-04122005-112516', 'etd-07102006-155549', 'etd-10112004-134529', 'etd-04052006-160318', 'etd-07012011-144141', 'etd-10222004-135928', 'etd-11192010-105306', 'etd-11112004-121808', 'etd-06022016-044816', 'etd-04062009-170250', 'etd-04132007-102644', 'etd-07132015-152120', 'etd-01222006-170722', 'etd-11122014-144949', 'etd-06122007-220134', 'etd-04162009-021139', 'etd-07082013-120110', 'etd-07072016-124553', 'etd-11012005-083859', 'etd-0707103-131556', 'etd-07052012-110306', 'etd-07132007-122349', 'etd-06072007-152128', 'etd-04142004-145321', 'etd-11162006-134428', 'etd-05312011-130705', 'etd-05252016-111428', 'etd-04102010-151013', 'etd-1113102-085005', 'etd-0708103-181831', 'etd-04092008-192154', 'etd-07072010-104947', 'etd-11092012-124855', 'etd-01252010-094716', 'etd-01172014-154034', 'etd-04122010-092452', 'etd-11112005-085846', 'etd-01242005-144012', 'etd-07082010-111610', 'etd-11152006-214523', 'etd-03162006-181757', 'etd-10292007-104319', 'etd-11142005-171744', 'etd-04112005-154524', 'etd-04112014-183528', 'etd-06052014-033113', 'etd-11082015-112501', 'etd-07122007-141201', 'etd-0711102-224806', 'etd-06292015-114703', 'etd-01252012-004511', 'etd-06262009-105556', 'etd-04152013-111909', 'etd-04152009-161334', 'etd-06272007-163108', 'etd-11102004-114145', 'etd-09022009-114608', 'etd-0403103-220359', 'etd-07062012-094027', 'etd-08312013-110314', 'etd-1107102-164200', 'etd-04102014-142610', 'etd-01232013-141158', 'etd-10192006-145427', 'etd-10162013-201151', 'etd-03162013-155624', 'etd-06232011-172653', 'etd-01162008-150910', 'etd-11162015-104143', 'etd-04102010-012947', 'etd-04032014-142925', 'etd-05042006-160735', 'etd-04082013-133504', 'etd-0701103-164735', 'etd-06262012-123158', 'etd-11142005-110327', 'etd-03252016-030936', 'etd-04172012-165935', 'etd-07062009-130610', 'etd-04152013-151015', 'etd-04102014-162423', 'etd-11162012-115927', 'etd-11102011-125418', 'etd-0410103-115226', 'etd-0709103-175151', 'etd-0409103-160833', 'etd-06092010-174311', 'etd-04082004-163913', 'etd-01172008-113818', 'etd-07092010-111611', 'etd-11102011-151907', 'etd-04122009-205155', 'etd-04132005-085105', 'etd-07292012-174652', 'etd-09022014-111540', 'etd-01242013-155946', 'etd-07132005-134343', 'etd-01152012-183124', 'etd-11142005-101616', 'etd-0826103-140253', 'etd-01232007-114859', 'etd-03232005-095121', 'etd-04102014-135536', 'etd-01042005-183805', 'etd-04092008-195240', 'etd-06292011-105102', 'etd-04012008-105506', 'etd-10032005-131910', 'etd-11192010-095633', 'etd-07092004-113059', 'etd-06032015-130516', 'etd-0605102-112627', 'etd-04052007-135919', 'etd-09292004-162358', 'etd-04102015-100705', 'etd-04102008-110918', 'etd-11162007-084951', 'etd-07142014-151231', 'etd-03252015-125953', 'etd-07142005-144545', 'etd-07042007-182939', 'etd-1106103-130505', 'etd-07112016-110318', 'etd-07122007-165444', 'etd-06092010-150854', 'etd-10262011-101820', 'etd-07052006-173930', 'etd-04282010-140659', 'etd-05272004-185314', 'etd-06032011-105726', 'etd-06152016-113323', 'etd-03032014-215033', 'etd-07092014-144617', 'etd-08312010-153445', 'etd-11122007-100403', 'etd-06132005-221041', 'etd-08112004-104811', 'etd-04062015-164343', 'etd-04152005-101555', 'etd-11012004-114252', 'etd-07062016-143033', 'etd-09142004-094741', 'etd-08222011-095144', 'etd-07062015-073300', 'etd-1112103-094818', 'etd-0806103-020544', 'etd-04022007-134213', 'etd-10202014-153559', 'etd-09032013-113122', 'etd-11152012-160114', 'etd-0403103-212507', 'etd-11112009-193648', 'etd-05202009-112018', 'etd-05242016-195107', 'etd-10022011-130950', 'etd-07082016-094845', 'etd-09062007-142802', 'etd-01232006-134857', 'etd-08222005-160304', 'etd-11082004-105730', 'etd-11172006-110852', 'etd-11162012-082742', 'etd-04012008-134143', 'etd-0405103-131627', 'etd-1118103-115616', 'etd-08162006-124346', 'etd-1108102-013819', 'etd-02242015-184605', 'etd-08292007-154008', 'etd-10112004-141426', 'etd-11112009-192849', 'etd-04132005-095244', 'etd-0711102-135846', 'etd-0531102-173508', 'etd-07102007-145805', 'etd-04142005-164100', 'etd-04052016-101908', 'etd-04032006-135008', 'etd-0605103-150602', 'etd-11202015-150636', 'etd-04092008-135243', 'etd-08192008-160519', 'etd-01172008-144900', 'etd-07092009-163446', 'etd-04152004-184118', 'etd-11172005-133131', 'etd-11182013-144154', 'etd-04292011-100032', 'etd-06132005-182824', 'etd-04022014-023542', 'etd-04082015-163006', 'etd-04082015-224023', 'etd-0604102-113054', 'etd-04152005-104826', 'etd-11102009-171413', 'etd-01282009-133835', 'etd-0610103-163750', 'etd-04272011-153235', 'etd-01222014-141216', 'etd-05142012-224050', 'etd-04042016-121816', 'etd-04102009-161635', 'etd-0519103-152012', 'etd-07102015-140026', 'etd-04162009-224518', 'etd-0530102-095332', 'etd-10242012-232950', 'etd-06042004-120413', 'etd-09172014-153804', 'etd-07022013-224213', 'etd-08302011-110325', 'etd-04092009-171942', 'etd-05282004-130909', 'etd-11092011-161836', 'etd-04122005-200025', 'etd-11072015-214444', 'etd-07102012-220432', 'etd-04242012-002135', 'etd-09032007-181409', 'etd-11112012-155620', 'etd-06272008-092536', 'etd-07102015-091513', 'etd-04072010-142121', 'etd-04292010-132837', 'etd-07102006-164430', 'etd-01192014-232734', 'etd-07082014-101740', 'etd-07022015-150859', 'etd-04112014-125430', 'etd-04112014-092539', 'etd-01242005-192139', 'etd-01212005-144136', 'etd-04062012-122029', 'etd-11182005-095815', 'etd-04092016-130125', 'etd-06302009-221451', 'etd-04122015-124431', 'etd-06092013-124254', 'etd-06012010-214651', 'etd-11212005-190249', 'etd-0408103-220732', 'etd-07072011-094810', 'etd-07112012-025642', 'etd-01252006-155538', 'etd-07122007-232839', 'etd-10092013-142149', 'etd-04122010-170915', 'etd-11142014-093450', 'etd-04182012-150609', 'etd-04072006-093608', 'etd-04272010-212919', 'etd-07152009-173404', 'etd-02292016-141246', 'etd-04152013-094149', 'etd-06072005-235542', 'etd-06012016-203252', 'etd-04282010-122823', 'etd-04142014-152332', 'etd-07132007-112833', 'etd-04132005-141508', 'etd-07132014-173120', 'etd-11112004-105232', 'etd-07062011-140306', 'etd-11122004-071142', 'etd-07072006-112445', 'etd-11172014-003502', 'etd-11152007-221205', 'etd-08312010-160906', 'etd-04142013-202723', 'etd-0626103-112039', 'etd-11142014-130946', 'etd-04062006-073354', 'etd-07122007-191812', 'etd-04142010-132933', 'etd-04092015-181523', 'etd-11152007-100213', 'etd-11152007-125223', 'etd-04192012-164729', 'etd-04082013-162448', 'etd-04262012-111647', 'etd-09242012-160207', 'etd-04112016-102109', 'etd-07062004-111717', 'etd-1113102-223702', 'etd-04142005-171033', 'etd-04112007-165514', 'etd-11052015-170948', 'etd-11012004-115333', 'etd-07142005-092131', 'etd-04062004-184200', 'etd-11082004-213649', 'etd-07092009-113052', 'etd-1113102-150012', 'etd-1113102-112657', 'etd-04122005-173708', 'etd-11112004-171053', 'etd-04112016-122956', 'etd-04082013-190245', 'etd-06132006-152536', 'etd-04092008-014603', 'etd-0708102-150553', 'etd-04112011-151905', 'etd-04082014-115441', 'etd-04032009-103923', 'etd-06082011-154750', 'etd-04272010-151828', 'etd-06142006-234710', 'etd-06062007-135845', 'etd-1112102-134210', 'etd-0710102-150954', 'etd-04112014-151802', 'etd-10302009-103052', 'etd-1112103-183958', 'etd-01262005-133356', 'etd-11072011-200036', 'etd-04112013-152716', 'etd-04042016-195144', 'etd-06012011-112253', 'etd-0609103-215846', 'etd-11182005-085849', 'etd-11122013-152628', 'etd-11082004-100236', 'etd-11152014-131055', 'etd-04112015-144217', 'etd-04022014-194729', 'etd-11152007-100302', 'etd-07102012-164634', 'etd-04302010-183731', 'etd-04112005-155320', 'etd-05042012-104133', 'etd-06132005-174405', 'etd-1111103-174600', 'etd-08302010-004553', 'etd-03312015-130646', 'etd-04272012-114012', 'etd-0418102-094729', 'etd-11172006-115544', 'etd-0611103-110623', 'etd-06132006-125156', 'etd-11102009-202427', 'etd-07042007-120232', 'etd-10312008-125107', 'etd-07052006-234928', 'etd-07092012-145208', 'etd-01142014-185208', 'etd-11182010-130411', 'etd-05302008-021723', 'etd-04112016-132801', 'etd-06092010-200522', 'etd-04112004-192013', 'etd-06062012-164102', 'etd-06082004-161239', 'etd-07022004-231204', 'etd-04282011-152449', 'etd-07062013-102527', 'etd-11032006-132926', 'etd-03082012-151418', 'etd-07062006-185252', 'etd-05312012-131733', 'etd-0704103-080530', 'etd-07082009-113642', 'etd-07062011-133536', 'etd-07112014-131822', 'etd-06022015-103220', 'etd-04082009-113415', 'etd-03122015-171432', 'etd-07132014-234534', 'etd-11152006-162056', 'etd-08142013-214423', 'etd-07072011-174745', 'etd-11112008-114140', 'etd-08032007-210655', 'etd-06302011-115240', 'etd-01262005-155314', 'etd-11162007-094553', 'etd-0412102-144208', 'etd-06182004-122626', 'etd-11032010-162509', 'etd-0611103-184324', 'etd-01122006-152604', 'etd-11112005-083838', 'etd-04152005-092319', 'etd-1114102-122427', 'etd-06162005-104702', 'etd-08132014-150432', 'etd-04122007-183353', 'etd-05262016-143850', 'etd-07082009-162948', 'etd-0404103-035324', 'etd-01122012-131516', 'etd-08192004-224424', 'etd-04072014-175943', 'etd-04062004-102152', 'etd-04112008-085237', 'etd-04202011-133123', 'etd-06172011-090919', 'etd-07082013-143322', 'etd-03072008-104040', 'etd-04102008-162119', 'etd-04152011-172158', 'etd-12152003-154103', 'etd-04062016-145807', 'etd-04142005-172229', 'etd-06102004-152737', 'etd-08042004-200816', 'etd-04132004-074926', 'etd-0902103-150730', 'etd-02082012-092827', 'etd-0320103-104205', 'etd-07032013-133932', 'etd-07062010-144215', 'etd-07102008-175259', 'etd-07082009-122037', 'etd-07112006-220909', 'etd-11142014-140005', 'etd-05092013-212420', 'etd-11112004-100046', 'etd-0709103-132124', 'etd-08252011-175101', 'etd-0915102-151452', 'etd-11132008-133324', 'etd-11102009-123504', 'etd-11172005-190237', 'etd-01172005-192715', 'etd-06292005-160447', 'etd-1110103-154436', 'etd-06072007-130053', 'etd-06152011-190824', 'etd-04252012-144940', 'etd-06102013-135003', 'etd-07082009-152432', 'etd-11092006-163100', 'etd-04212011-101531', 'etd-03102014-084353', 'etd-07032015-131422', 'etd-11082006-151644', 'etd-10312012-174712', 'etd-0701102-134213', 'etd-11022006-162629', 'etd-05032010-160931', 'etd-01102008-150648', 'etd-0411102-125147', 'etd-04052015-201834', 'etd-0528102-105721', 'etd-07102008-192659', 'etd-07012011-112712', 'etd-11052014-085359', 'etd-04142009-234500', 'etd-06302011-124114', 'etd-06112015-134148', 'etd-06022014-134538', 'etd-04052011-105808', 'etd-03282010-134524', 'etd-07102014-134202', 'etd-0418102-232204', 'etd-01182011-210957', 'etd-0418102-090832', 'etd-01222013-162013', 'etd-07302007-134204', 'etd-04122004-150752', 'etd-01132004-114420', 'etd-0128103-213513', 'etd-04182011-213419', 'etd-06102008-131317', 'etd-10302008-224954', 'etd-11142006-192128', 'etd-07012004-101913', 'etd-11042014-172653', 'etd-0410103-172007', 'etd-04032015-204943', 'etd-0330103-174711', 'etd-04272011-111335', 'etd-0410103-230516', 'etd-11172010-144103', 'etd-0708103-111405', 'etd-04132013-151026', 'etd-07082004-104919', 'etd-0323102-193001', 'etd-04122005-160920', 'etd-01272004-110513', 'etd-11162005-181900', 'etd-06142007-084318', 'etd-06212006-113212', 'etd-0612102-150125', 'etd-0128102-201502', 'etd-04252011-233326', 'etd-11112011-094940', 'etd-01102006-135121', 'etd-11102005-215035', 'etd-04292011-092621', 'etd-04262012-170458', 'etd-0709102-100417', 'etd-06252013-123055', 'etd-04272010-221042', 'etd-1109103-171900', 'etd-07112007-082054', 'etd-06292011-163239', 'etd-04262012-145658', 'etd-11082004-201124', 'etd-04122007-103232', 'etd-11132015-124001', 'etd-11132007-105835', 'etd-06262013-220837', 'etd-0401103-232720', 'etd-11082007-155709', 'etd-06142012-143141', 'etd-11112013-150354', 'etd-0419102-111029', 'etd-01232007-012258', 'etd-04152009-181041', 'etd-04142014-081214', 'etd-04162010-163830', 'etd-07072015-175341', 'etd-1111103-173913', 'etd-07072011-231500', 'etd-04122011-102759', 'etd-0708103-133728', 'etd-04102008-035013', 'etd-11152013-141532', 'etd-0707103-134718', 'etd-03302006-175827', 'etd-04102013-200554', 'etd-0415102-112656', 'etd-11112009-164423', 'etd-11142006-074613', 'etd-09062006-102551', 'etd-04252011-172916', 'etd-04192012-135431', 'etd-07102015-150858', 'etd-0711102-123012', 'etd-07132015-151731', 'etd-11092011-115925', 'etd-07082009-114631', 'etd-0709103-153221', 'etd-04112005-120455', 'etd-06082009-123220', 'etd-03232007-130307', 'etd-04032006-173356', 'etd-07102008-230306', 'etd-09142011-130734', 'etd-07072009-181647', 'etd-04122010-114839', 'etd-04092014-000411', 'etd-06082006-110005', 'etd-04112016-143126', 'etd-05212016-004731', 'etd-06072011-103035', 'etd-10272004-150409', 'etd-07072011-192636', 'etd-07062011-150311', 'etd-07112013-145111', 'etd-11072011-163609', 'etd-07132015-015315', 'etd-04162012-152226', 'etd-07062016-111309', 'etd-07132015-124330', 'etd-07082006-200008', 'etd-11052007-184025', 'etd-0411103-114830', 'etd-04242012-000041', 'etd-04142004-140419', 'etd-04142004-152417', 'etd-1114102-165241', 'etd-11082004-201210', 'etd-07062004-162728', 'etd-11032009-155745', 'etd-1025102-133538', 'etd-08152004-142832', 'etd-04132005-180725', 'etd-0225102-160905', 'etd-06102013-154144', 'etd-06112007-174348', 'etd-04202010-111813', 'etd-0411103-103209', 'etd-0130102-161843', 'etd-03292007-090639', 'etd-11152006-082336', 'etd-04082014-091530', 'etd-04042013-111405', 'etd-1115101-113559', 'etd-0526103-151738', 'etd-11152006-145418', 'etd-11182013-102603', 'etd-06062011-143639', 'etd-11142007-170943', 'etd-10182010-105045', 'etd-04202011-170529', 'etd-04302010-130425', 'etd-04092015-162336', 'etd-0711102-182913', 'etd-04062016-132725', 'etd-07102008-140625', 'etd-04072006-101332', 'etd-04082004-150047', 'etd-11182010-150610', 'etd-03272015-113238', 'etd-10232008-132829', 'etd-04102008-134208', 'etd-1113101-162847', 'etd-11152013-152635', 'etd-12162005-145644', 'etd-12222003-115435', 'etd-04092007-155537', 'etd-04062016-132540', 'etd-0530103-154951', 'etd-11192010-070016', 'etd-06102014-113211', 'etd-08142014-090444', 'etd-04242012-154908', 'etd-0417102-170544', 'etd-06092009-111918', 'etd-04102007-203356', 'etd-11162006-101838', 'etd-05262016-085417', 'etd-04282011-065004', 'etd-01052012-150637', 'etd-11112009-142047', 'etd-07132005-143634', 'etd-04112012-105233', 'etd-11162007-104936', 'etd-0610103-224404', 'etd-11082004-112134', 'etd-04062016-113804', 'etd-11112013-191819', 'etd-07032004-112854', 'etd-07132012-035018', 'etd-04132005-234758', 'etd-11102015-091749', 'etd-01242012-201855', 'etd-01052006-121418', 'etd-03252015-125650', 'etd-0404103-121428', 'etd-11082004-135255', 'etd-04202012-081513', 'etd-11162005-210229', 'etd-11162006-125102', 'etd-01252005-064635', 'etd-11152006-121554', 'etd-07142005-084056', 'etd-04152013-134306', 'etd-08312013-094009', 'etd-11162012-115417', 'etd-0603103-112538', 'etd-11112015-105214', 'etd-04022015-121419', 'etd-04082009-113609', 'etd-04272011-135545', 'etd-05232016-115807', 'etd-04062006-164221', 'etd-04102009-012625', 'etd-11202013-090513', 'etd-1109103-213211', 'etd-02272012-172501', 'etd-07112005-133748', 'etd-11112009-092220', 'etd-09222010-164535', 'etd-06212010-111919', 'etd-09042013-150559', 'etd-01172011-120802', 'etd-03022011-074148', 'etd-04052007-131133', 'etd-06062012-155305', 'etd-07012004-143102', 'etd-03282015-142253', 'etd-11152013-200438', 'etd-04112005-195230', 'etd-02252009-092709', 'etd-04142014-121409', 'etd-11152007-161413', 'etd-07072010-110740', 'etd-05062009-220100', 'etd-06302016-170437', 'etd-06052013-113942', 'etd-07022015-161605', 'etd-05152006-092746', 'etd-06052012-121236', 'etd-11092009-163619', 'etd-06302011-143941', 'etd-07112007-012708', 'etd-06092014-140552', 'etd-11112004-193110', 'etd-11052012-134539', 'etd-04022015-142902', 'etd-01262011-033902', 'etd-04022004-144548', 'etd-06042014-223737', 'etd-0709102-113122', 'etd-04102015-092031', 'etd-04012006-020209', 'etd-06262015-161528', 'etd-03152006-110106', 'etd-10202006-084858', 'etd-1128101-095316', 'etd-11042005-120721', 'etd-09152005-150030', 'etd-04042006-124302', 'etd-02182009-103434', 'etd-1114102-201259', 'etd-0418102-142132', 'etd-11112011-052936', 'etd-06032014-155229', 'etd-04062005-121828', 'etd-04252011-175539', 'etd-0502102-131203', 'etd-06102013-152446', 'etd-07132006-142755', 'etd-05252012-144155', 'etd-04062008-202353', 'etd-04042014-190104', 'etd-11092004-091831', 'etd-0320103-094017', 'etd-07132015-013229', 'etd-04032016-175740', 'etd-0409103-111757', 'etd-06262004-140955', 'etd-07152005-084150', 'etd-07082008-083329', 'etd-04042008-102844', 'etd-04042006-105557', 'etd-11172010-151513', 'etd-01202005-102411', 'etd-04012016-180941', 'etd-05222008-161656', 'etd-10292004-102526', 'etd-06102013-004429', 'etd-07082004-095554', 'etd-04042005-171447', 'etd-11132009-081957', 'etd-04012008-180807', 'etd-07062012-082644', 'etd-06052013-131126', 'etd-04112016-134534', 'etd-1112103-083723', 'etd-04112016-142715', 'etd-11172005-204730', 'etd-04152004-161117', 'etd-04112016-142758', 'etd-11012011-145108', 'etd-04112007-001927', 'etd-07092010-110611', 'etd-06282013-193921', 'etd-12012004-134154', 'etd-10092007-100620', 'etd-0328103-132815', 'etd-1112102-030154', 'etd-1110103-142617', 'etd-0410103-145753', 'etd-04152013-155544', 'etd-04102016-074116', 'etd-04112012-112650', 'etd-11132008-125305', 'etd-07022004-111634', 'etd-03232004-093027', 'etd-03302006-143124', 'etd-04142011-162805', 'etd-0707103-142120', 'etd-04042006-221129', 'etd-03292011-141434', 'etd-04032006-220749', 'etd-11132009-112020', 'etd-05242009-114857', 'etd-07012009-123131', 'etd-04152010-153003', 'etd-03272007-083755', 'etd-07092008-085901', 'etd-0416102-151044', 'etd-05242013-182424', 'etd-09052007-124840', 'etd-0410103-191114', 'etd-09092004-155353', 'etd-07112005-183504', 'etd-07092014-162328', 'etd-03112007-200919', 'etd-06032011-135301', 'etd-04242012-124150', 'etd-07132005-133937', 'etd-06282013-173518', 'etd-06262010-111917', 'etd-04282011-175003', 'etd-07072014-174349', 'etd-05162016-090623', 'etd-11132006-224239', 'etd-07112007-135924', 'etd-11082013-102007', 'etd-03202014-083129', 'etd-03252004-165820', 'etd-04112005-141253', 'etd-04262010-140636', 'etd-07052012-174103', 'etd-1112101-153735', 'etd-10272015-094143', 'etd-07062015-211932', 'etd-11112008-114857', 'etd-0404103-154914', 'etd-10052010-220858', 'etd-1109102-145122', 'etd-04282010-083206', 'etd-07022009-001507', 'etd-04152005-083208', 'etd-08272010-092510', 'etd-06152013-213900', 'etd-05162013-201035', 'etd-11102010-165935', 'etd-04072007-151257', 'etd-05312011-095723', 'etd-10212005-130147', 'etd-04132014-103437', 'etd-12192006-113604', 'etd-04292015-020803', 'etd-03292007-183532', 'etd-11102011-130607', 'etd-0709102-153856', 'etd-10252005-094017', 'etd-07062011-120910', 'etd-05232013-105119', 'etd-07142014-075200', 'etd-03282006-111609', 'etd-09112007-090523', 'etd-11092011-205155', 'etd-08192013-113334', 'etd-1009103-094222', 'etd-04152013-105345', 'etd-07112012-122340', 'etd-07082009-083522', 'etd-05232016-134706', 'etd-02262010-092056', 'etd-11132008-144951', 'etd-07132015-113753', 'etd-03042015-155716', 'etd-04052016-134143', 'etd-09022013-183036', 'etd-0710103-174408', 'etd-04302010-171225', 'etd-07012014-133507', 'etd-04162012-115502', 'etd-0330103-160922', 'etd-12092008-200526', 'etd-01242007-111109', 'etd-1108101-143901', 'etd-03242014-125147', 'etd-01082005-103926', 'etd-06272008-142334', 'etd-04142014-155834', 'etd-04082016-205631', 'etd-01222008-154806', 'etd-0419102-114734', 'etd-11102006-153930', 'etd-07082004-150137', 'etd-07062009-020543', 'etd-0417102-215042', 'etd-11162010-154549', 'etd-06062012-225125', 'etd-07092008-173122', 'etd-11112008-141527', 'etd-04242012-075315', 'etd-0830102-145811', 'etd-03022009-085729', 'etd-0411102-194709', 'etd-10242005-162307', 'etd-07032007-122846', 'etd-07072016-171533', 'etd-04112005-131603', 'etd-11142007-230454', 'etd-04162008-141353', 'etd-06072009-175525', 'etd-07052011-221923', 'etd-04102013-091629', 'etd-04262011-225831', 'etd-04142009-152404', 'etd-11122004-112840', 'etd-0516103-141609', 'etd-0402102-154912', 'etd-06262012-160409', 'etd-11132009-104544', 'etd-04032015-121942', 'etd-03142012-221535', 'etd-11172005-171213', 'etd-04152013-122143', 'etd-1114102-211348', 'etd-07052006-201255', 'etd-07082015-202701', 'etd-11162006-102101', 'etd-06062013-151334', 'etd-08282014-152138', 'etd-06052008-102543', 'etd-04102008-233609', 'etd-11162006-140457', 'etd-11072009-093300', 'etd-04112016-091604', 'etd-0711102-125805', 'etd-04092014-103933', 'etd-0407103-185546', 'etd-0607102-135829', 'etd-12232013-211220', 'etd-0603102-144115', 'etd-07082013-131811', 'etd-04112016-050653', 'etd-11172006-091519', 'etd-06292015-213159', 'etd-11112012-190037', 'etd-04092015-201659', 'etd-09252006-134408', 'etd-04272010-175047', 'etd-04112007-143937', 'etd-04132015-142139', 'etd-05272015-141735', 'etd-01262012-151136', 'etd-04062016-143658', 'etd-11162007-100232', 'etd-04272012-093750', 'etd-04122005-203314', 'etd-04102014-091806', 'etd-01202005-134859', 'etd-07082011-091900', 'etd-07142014-161650', 'etd-04052012-120711', 'etd-04152009-094001', 'etd-0410103-122444', 'etd-0608103-162721', 'etd-11122009-142514', 'etd-11162006-195517', 'etd-11172013-195817', 'etd-04212011-124638', 'etd-11172010-130107', 'etd-04132015-104207', 'etd-10222014-194148', 'etd-04082014-092710', 'etd-04052006-172605', 'etd-07062011-133533', 'etd-04112014-161230', 'etd-03312005-193516', 'etd-07062004-150234', 'etd-11132006-115451', 'etd-04112005-132451', 'etd-04112008-114436', 'etd-11122008-100456', 'etd-04282010-161838', 'etd-04132013-133658', 'etd-0411102-070847', 'etd-10032008-173527', 'etd-01132016-150749', 'etd-1113102-212109', 'etd-11052004-145157', 'etd-09022005-094806', 'etd-0131102-164314', 'etd-04202011-091522', 'etd-04112016-100249', 'etd-04112016-142944', 'etd-07032010-013600', 'etd-04122010-160834', 'etd-04042006-194233', 'etd-06212011-142520', 'etd-04282011-104818', 'etd-04012011-015743', 'etd-07032007-101237', 'etd-07082016-132720', 'etd-03312014-105753', 'etd-07012016-125344', 'etd-04072014-221010', 'etd-0416102-184739', 'etd-04152009-133710', 'etd-01122015-130331', 'etd-11092012-154619', 'etd-07132007-075601', 'etd-04072014-162250', 'etd-04092015-234706', 'etd-04072011-121327', 'etd-01292004-141532', 'etd-04142009-112949', 'etd-04112005-150711', 'etd-05192015-103813', 'etd-11162015-120200', 'etd-06062013-173536', 'etd-04282010-144026', 'etd-04102007-103358', 'etd-11082004-132339', 'etd-11092012-120635', 'etd-04262011-222019', 'etd-11082015-184238', 'etd-04082008-144807', 'etd-06252013-142858', 'etd-0410102-151815', 'etd-10272004-084348', 'etd-07082015-135301', 'etd-04282011-095509', 'etd-03162008-175714', 'etd-04262012-172938', 'etd-0927102-125908', 'etd-07072014-170726', 'etd-04112013-130104', 'etd-11132009-105045', 'etd-04142014-115743', 'etd-04072015-141919', 'etd-0410103-123323', 'etd-02082007-151626', 'etd-04082005-141028', 'etd-03252004-135756', 'etd-03262015-152830', 'etd-07072004-094725', 'etd-0418102-084638', 'etd-06052012-091104', 'etd-11162010-204916', 'etd-05202016-162645', 'etd-06052012-192854', 'etd-05122015-134133', 'etd-06232005-163326', 'etd-06272016-122942', 'etd-04192012-115227', 'etd-04162010-153355', 'etd-06022016-163746', 'etd-06122007-122820', 'etd-07092010-083717', 'etd-11072005-214521', 'etd-0411103-114729', 'etd-04142010-132618', 'etd-08262015-010240', 'etd-04272012-115256', 'etd-0331103-170040', 'etd-04132005-115008', 'etd-04072016-151644', 'etd-0707103-135356', 'etd-07112014-104704', 'etd-05192011-005159', 'etd-11112014-150356', 'etd-07062012-081210', 'etd-12192011-172227', 'etd-07022013-191335', 'etd-02242005-133322', 'etd-1114101-185702', 'etd-11062014-103552', 'etd-04112007-113225', 'etd-11162010-121817', 'etd-11122007-203748', 'etd-1106103-201904', 'etd-03282006-130742', 'etd-07022008-013657', 'etd-11052004-152838', 'etd-01182011-130229', 'etd-07032012-164131', 'etd-04072014-191558', 'etd-04112015-223252', 'etd-11092005-152334', 'etd-08222011-173310', 'etd-10262015-230057', 'etd-01242012-154223', 'etd-04192011-084243', 'etd-04142005-161537', 'etd-1102103-205315', 'etd-01122005-201431', 'etd-04102007-131052', 'etd-0411102-162551', 'etd-0709102-115258', 'etd-01172014-124019', 'etd-07022009-143617', 'etd-10052009-215336', 'etd-07082014-185654', 'etd-10102010-164535', 'etd-04112007-135355', 'etd-04052012-232438', 'etd-0515103-202624', 'etd-0415102-075822', 'etd-10232015-164138', 'etd-07072010-144721', 'etd-04142010-204912', 'etd-12192006-121617', 'etd-04062006-110959', 'etd-04292010-141019', 'etd-0326103-192448', 'etd-01262015-154917', 'etd-11122007-143350', 'etd-06132013-151305', 'etd-04072006-091143', 'etd-08082014-140842', 'etd-04112013-164551', 'etd-01102008-165432', 'etd-0417102-144343', 'etd-0611103-112314', 'etd-03092006-205849', 'etd-04132012-174640', 'etd-04292011-082207', 'etd-07142006-112623', 'etd-03192014-163532', 'etd-11132006-211541', 'etd-04142004-115227', 'etd-04022013-152408', 'etd-03292012-193752', 'etd-07142005-232038', 'etd-11112011-100404', 'etd-11182010-234139', 'etd-0530103-085516', 'etd-11062011-155437', 'etd-08052013-164517', 'etd-07022007-113018', 'etd-10172008-144608', 'etd-04182012-121521', 'etd-0414102-135429', 'etd-04072015-231330', 'etd-04072009-134011', 'etd-0403103-153026', 'etd-0605102-124737', 'etd-11112008-121037', 'etd-11122008-194055', 'etd-0416102-150028', 'etd-08132013-163410', 'etd-11072012-164012', 'etd-1101102-142703', 'etd-04022014-090754', 'etd-04122007-002428', 'etd-1113103-122552', 'etd-07082014-113259', 'etd-1115102-082947', 'etd-03262013-121703', 'etd-04222010-141902', 'etd-10222010-103809', 'etd-11022004-225207', 'etd-04152009-175333', 'etd-07142014-143834', 'etd-05272016-180310', 'etd-10022015-104853', 'etd-04082015-153419', 'etd-09012009-084129', 'etd-07112005-184347', 'etd-01242008-125555', 'etd-07242010-112837', 'etd-0210103-083019', 'etd-07122007-103329', 'etd-07082010-121436', 'etd-11142014-164020', 'etd-01212007-175048', 'etd-11222005-065828', 'etd-04022015-095919', 'etd-04152013-153132', 'etd-11142012-084124', 'etd-04062004-154900', 'etd-0419102-093318', 'etd-04102007-183440', 'etd-04112015-082807', 'etd-05092008-093532', 'etd-07082008-173817', 'etd-1104103-113636', 'etd-05162009-143252', 'etd-03032015-140517', 'etd-11102011-090846', 'etd-10262007-082622', 'etd-0404103-002340', 'etd-06272016-140723', 'etd-06152012-101240', 'etd-04262011-193227', 'etd-02202013-132920', 'etd-11182005-092209', 'etd-04202011-224416', 'etd-04282010-151225', 'etd-0401103-143658', 'etd-06032004-155123', 'etd-11102004-134343', 'etd-04052011-102551', 'etd-04022007-175836', 'etd-04282011-093442', 'etd-04022008-124945', 'etd-01232006-184352', 'etd-04022007-211811', 'etd-08302004-153703', 'etd-07072011-183310', 'etd-01162014-211815', 'etd-07082009-143307', 'etd-06092004-130835', 'etd-06052008-135410', 'etd-04252012-163423', 'etd-0608103-130811', 'etd-04152004-201532', 'etd-09302014-132817', 'etd-04052005-143257', 'etd-11032009-154443', 'etd-07092007-150143', 'etd-06082010-124222', 'etd-04072008-134508', 'etd-03232005-161456', 'etd-11112004-093751', 'etd-07082005-104637', 'etd-06092009-102311', 'etd-11152012-113952', 'etd-04072014-094226', 'etd-11052006-193341', 'etd-04152004-131444', 'etd-07062015-125210', 'etd-11072014-141014', 'etd-0609103-131101', 'etd-07092004-113612', 'etd-04022015-144915', 'etd-04112005-224801', 'etd-04152010-090038', 'etd-0709102-002036', 'etd-11172005-151724', 'etd-08302010-155649', 'etd-0415102-183759', 'etd-12212008-211507', 'etd-07132007-112813', 'etd-04102013-144846', 'etd-07082010-183754', 'etd-11162006-182434', 'etd-01252005-131546', 'etd-04102014-022944', 'etd-06142005-164259', 'etd-11022011-171409', 'etd-03312006-150401', 'etd-07082013-005157', 'etd-04272011-152744', 'etd-05262016-100421', 'etd-04092008-123135', 'etd-0625103-110131', 'etd-06132007-122837', 'etd-11122012-154709', 'etd-05012010-002203', 'etd-11122008-122047', 'etd-07082013-090933', 'etd-11052009-132028', 'etd-11132006-084443', 'etd-11182013-144858', 'etd-04042008-085846', 'etd-11132014-235426', 'etd-0515103-095458', 'etd-07132006-150511', 'etd-11182013-111230', 'etd-06302015-124635', 'etd-04142005-174335', 'etd-04242012-142932', 'etd-04152013-081754', 'etd-04092007-144131', 'etd-05232012-125133', 'etd-11112009-174803', 'etd-02232009-190627', 'etd-1103103-112433', 'etd-05242004-165024', 'etd-04132005-140539', 'etd-0411102-220101', 'etd-03282016-194404', 'etd-0522103-120700', 'etd-05242013-154823', 'etd-04072016-125914', 'etd-07082010-021415', 'etd-06302016-144023', 'etd-11162006-154048', 'etd-07112014-170420', 'etd-07132012-102910', 'etd-03042005-150137', 'etd-05292015-111723', 'etd-04152013-122905', 'etd-11042008-143741', 'etd-11022009-105619', 'etd-03042013-113651', 'etd-07062011-030839', 'etd-03272013-104835', 'etd-04142005-181015', 'etd-11132007-185208', 'etd-01222006-155406', 'etd-04292011-090634', 'etd-11092004-170632', 'etd-04142010-181234', 'etd-0412102-082645', 'etd-05312010-130652', 'etd-07132015-141118', 'etd-03272012-190237', 'etd-0731102-220002', 'etd-07122005-142543', 'etd-12192003-173942', 'etd-0407103-123955', 'etd-11152005-135439', 'etd-04132014-133958', 'etd-11082011-094452', 'etd-08262004-203334', 'etd-04142014-104311', 'etd-04252011-191232', 'etd-04282011-135154', 'etd-11092005-125857', 'etd-04232012-140513', 'etd-08292011-141729', 'etd-11182010-131545', 'etd-10082011-215759', 'etd-08272014-192439', 'etd-05302006-160505', 'etd-05262016-135134', 'etd-1111103-152109', 'etd-04232012-151737', 'etd-0408103-131723', 'etd-0411103-101951', 'etd-0709103-135015', 'etd-11032008-193053', 'etd-11082012-185447', 'etd-04112005-142536', 'etd-09062006-125211', 'etd-04282010-100158', 'etd-1111103-101742', 'etd-11102015-114715', 'etd-03252005-165703', 'etd-11102011-105511', 'etd-0405102-234921', 'etd-0409103-130641', 'etd-04142010-170734', 'etd-06192009-011654', 'etd-03242005-144257', 'etd-07062010-132958', 'etd-03172013-162846', 'etd-11092011-183046', 'etd-02182014-203428', 'etd-10282013-110602', 'etd-07092008-224607', 'etd-04292011-090330', 'etd-08042011-180939', 'etd-01262010-053937', 'etd-04092007-130602', 'etd-08292011-223056', 'etd-04282010-154301', 'etd-0916103-154052', 'etd-06102010-141524', 'etd-07142005-145922', 'etd-05162016-114951', 'etd-04112007-230312', 'etd-07032013-084444', 'etd-07102006-130303', 'etd-06222012-175202', 'etd-0710102-172554', 'etd-11162015-122951', 'etd-11112009-113914', 'etd-04142014-155552', 'etd-04092007-181309', 'etd-1111103-115709', 'etd-07072011-104906', 'etd-0927102-135929', 'etd-06032013-142800', 'etd-01132010-125038', 'etd-0521103-163102', 'etd-0611102-141208', 'etd-07072016-015631', 'etd-04252011-180855', 'etd-06082004-131956', 'etd-11052009-081133', 'etd-0611103-101847', 'etd-04122007-112125', 'etd-0823102-142941', 'etd-04122007-151353', 'etd-12192014-144758', 'etd-04062016-181603', 'etd-03242006-013142', 'etd-04242012-162525', 'etd-04022015-161420', 'etd-04032006-155756', 'etd-06232011-211905', 'etd-04072006-114156', 'etd-06122014-193901', 'etd-07272012-151912', 'etd-04272011-155810', 'etd-07052006-181044', 'etd-04092007-142845', 'etd-0417102-115228', 'etd-05252010-141807', 'etd-06092004-144122', 'etd-03182013-190045', 'etd-1112102-210446', 'etd-07072009-140443', 'etd-07112012-113316', 'etd-06052015-172230', 'etd-04132011-195257', 'etd-0903102-153114', 'etd-01262004-143729', 'etd-01252012-103054', 'etd-04122005-125155', 'etd-07112007-105418', 'etd-04142014-120632', 'etd-05242011-185851', 'etd-04092014-183836', 'etd-06032011-162726', 'etd-09092008-153306', 'etd-01182006-102127', 'etd-06142007-145340', 'etd-04072009-130600', 'etd-07072011-145443', 'etd-04112016-132222', 'etd-11162005-195606', 'etd-10202009-094300', 'etd-04292010-204404', 'etd-07112016-153156', 'etd-11072011-222002', 'etd-03072010-213744', 'etd-04112005-134544', 'etd-07072011-111637', 'etd-03302004-105649', 'etd-06292011-105339', 'etd-0416102-171159', 'etd-03302006-160650', 'etd-10132004-095000', 'etd-0402103-151744', 'etd-07092008-211059', 'etd-04082005-125138', 'etd-04262012-143532', 'etd-10202014-161318', 'etd-04132005-144440', 'etd-06172009-115912', 'etd-0629103-134945', 'etd-07012013-223832', 'etd-04142009-092649', 'etd-07142014-144131', 'etd-11182013-090326', 'etd-04262012-094911', 'etd-02172006-154429', 'etd-06092010-183224', 'etd-11152007-012126', 'etd-11182014-131633', 'etd-05082006-144610', 'etd-04022015-162440', 'etd-0827103-121346', 'etd-11172010-123224', 'etd-07022013-185713', 'etd-09102008-134713', 'etd-04052011-195831', 'etd-06192008-092427', 'etd-07052016-091603', 'etd-0707103-110638', 'etd-08282004-132636', 'etd-11052007-094556', 'etd-11102008-120315', 'etd-02252010-102733', 'etd-10282013-134806', 'etd-08252010-001820', 'etd-04162009-155621', 'etd-04072004-180148', 'etd-0130102-144242', 'etd-04142011-162307', 'etd-02202009-182950', 'etd-04142014-140718', 'etd-07092009-120927', 'etd-08192005-145124', 'etd-07092015-092024', 'etd-11072014-162838', 'etd-04052016-145019', 'etd-0411103-120728', 'etd-04072007-160714', 'etd-06132005-084216', 'etd-06302004-160846', 'etd-0407103-164922', 'etd-04142014-092310', 'etd-07102015-201159', 'etd-0612102-103031', 'etd-07112005-143912', 'etd-02252007-121902', 'etd-07122014-094906', 'etd-03242008-172028', 'etd-11132012-154312', 'etd-03252010-192503', 'etd-07292011-093219', 'etd-11142005-063352', 'etd-11172005-222404', 'etd-0326103-164954', 'etd-11152007-184628', 'etd-0130103-090238', 'etd-07122007-160505', 'etd-07112012-180904', 'etd-04112007-094810', 'etd-0409103-090119', 'etd-07122012-111434', 'etd-09222015-180348', 'etd-04142005-104733', 'etd-1113102-202122', 'etd-04272012-093340', 'etd-07082014-153821', 'etd-11052009-232342', 'etd-0708102-185912', 'etd-04182005-093111', 'etd-04202011-220632', 'etd-07102009-115333', 'etd-05142012-124555', 'etd-11092015-094530', 'etd-06112014-160035', 'etd-04142005-102300', 'etd-04142005-101456', 'etd-10312008-085847', 'etd-07082010-104609', 'etd-08292011-225033', 'etd-03182014-133244', 'etd-1112102-125556', 'etd-0410103-135838', 'etd-08122013-104039', 'etd-08282006-112443', 'etd-07052011-210443', 'etd-07082013-103340', 'etd-09012006-091730', 'etd-06222016-193523', 'etd-08192013-075536', 'etd-04262012-223814', 'etd-06032014-180631', 'etd-01262010-114216', 'etd-06092008-100037', 'etd-07282005-120913', 'etd-03172010-103211', 'etd-05272005-104315', 'etd-04292010-142845', 'etd-04272010-165945', 'etd-05032008-005508', 'etd-01092013-150852', 'etd-04092007-223932', 'etd-07062015-170213', 'etd-0607102-152622', 'etd-07142014-103436', 'etd-04112015-194249', 'etd-04062015-140508', 'etd-04092008-102009', 'etd-11092011-230808', 'etd-04122007-170946', 'etd-04082005-135049', 'etd-06052007-115906', 'etd-06072012-122548', 'etd-06222004-145908', 'etd-04092014-203931', 'etd-0709103-205226', 'etd-04142014-121723', 'etd-04112013-174836', 'etd-11162015-154417', 'etd-07082016-214001', 'etd-04082013-152300', 'etd-11102009-062412', 'etd-04122007-121356', 'etd-05292008-103454', 'etd-04092015-171013', 'etd-04122005-151355', 'etd-06022011-105614', 'etd-11172005-135854', 'etd-06042013-091759', 'etd-11102009-150148', 'etd-11152012-103115', 'etd-0604103-151157', 'etd-07072011-152233', 'etd-07122006-125829', 'etd-04042016-152133', 'etd-09042012-034903', 'etd-07082004-142531', 'etd-04102014-174102', 'etd-04062006-162429', 'etd-11122007-135043', 'etd-07112016-150302', 'etd-04072006-101336', 'etd-04032006-115724', 'etd-07032012-125234', 'etd-04292010-162453', 'etd-04062015-171317', 'etd-04152004-190749', 'etd-1114102-154731', 'etd-07102016-182449', 'etd-04152013-091643', 'etd-06012010-150523', 'etd-1214101-190232', 'etd-08252014-122358', 'etd-09062006-083805', 'etd-04152013-163018', 'etd-04102007-121129', 'etd-1107102-185505', 'etd-04072005-132133', 'etd-12142005-154559', 'etd-11102011-122352', 'etd-10032013-113623', 'etd-07052011-085925', 'etd-1114101-111710', 'etd-09022005-215146', 'etd-11132009-010511', 'etd-04032006-211914', 'etd-01192009-140513', 'etd-04072014-182816', 'etd-04142005-162001', 'etd-11162010-004924', 'etd-11092006-153743', 'etd-05302009-115017', 'etd-04222012-072633', 'etd-03272013-165129', 'etd-02172014-125611', 'etd-04082007-181001', 'etd-06032012-222744', 'etd-0411102-170458', 'etd-04112016-022030', 'etd-11182010-014225', 'etd-01252005-122056', 'etd-04302010-144410', 'etd-02272009-141533', 'etd-04042007-090939', 'etd-06182004-151123', 'etd-1113103-180528', 'etd-06232011-114118', 'etd-04072004-185223', 'etd-11102013-230409', 'etd-0417102-153001', 'etd-07132005-125715', 'etd-07112014-214309', 'etd-07062015-221739', 'etd-04092016-004710', 'etd-06132005-151105', 'etd-06052011-165433', 'etd-06012012-125300', 'etd-07092010-034117', 'etd-07112006-183314', 'etd-05292013-162532', 'etd-07132006-154808', 'etd-11122013-153210', 'etd-07142006-093118', 'etd-07122007-191306', 'etd-05272004-164455', 'etd-11122007-091256', 'etd-06092004-124443', 'etd-0705102-152746', 'etd-04042007-131147', 'etd-04122014-121134', 'etd-04082014-155436', 'etd-04062016-135928', 'etd-01252012-235407', 'etd-07052015-150818', 'etd-06142007-114724', 'etd-06152005-110953', 'etd-01152013-123524', 'etd-04132004-185813', 'etd-11092011-124608', 'etd-04152004-112044', 'etd-0418102-120103', 'etd-01182014-171830', 'etd-1031102-055140', 'etd-0711102-135713', 'etd-0710102-200906', 'etd-04112016-110433', 'etd-06282012-100026', 'etd-11072014-163851', 'etd-04092014-173505', 'etd-04292004-103556', 'etd-06052013-211617', 'etd-0626103-161512', 'etd-04092015-112546', 'etd-0612103-153504', 'etd-01242007-200500', 'etd-04092014-125758', 'etd-0903103-143446', 'etd-09182012-122435', 'etd-0128103-183352', 'etd-07082013-094604', 'etd-12142005-110718', 'etd-06122005-095822', 'etd-04142005-190013', 'etd-04162012-143240', 'etd-07142005-130117', 'etd-08272009-003127', 'etd-07142014-131317', 'etd-11192010-111421', 'etd-07082013-160959', 'etd-0416102-155145', 'etd-1114101-172927', 'etd-11122009-172512', 'etd-0417102-191255', 'etd-0709103-110308', 'etd-0418102-153143', 'etd-0318102-121917', 'etd-03192009-154239', 'etd-07112005-221802', 'etd-03162007-170047', 'etd-07062015-155406', 'etd-08192006-134649', 'etd-04112011-114831', 'etd-0903103-142422', 'etd-09172006-133216', 'etd-05272009-161827', 'etd-11132008-153934', 'etd-01232005-164221', 'etd-04142014-154844', 'etd-08122010-132114', 'etd-1111103-094734', 'etd-07082004-091436', 'etd-0712102-125121', 'etd-04122013-142337', 'etd-09222006-101639', 'etd-1112102-201553', 'etd-11012006-192748', 'etd-07162007-092825', 'etd-0405103-193813', 'etd-11052010-141445', 'etd-03212006-071703', 'etd-11162006-151406', 'etd-04042005-172526', 'etd-07292004-154529', 'etd-07112016-103820', 'etd-07062010-073923', 'etd-12082015-175150', 'etd-01072004-183436', 'etd-01182006-091746', 'etd-03262015-130404', 'etd-07092014-080739', 'etd-04022015-121416', 'etd-06062008-093207', 'etd-04142005-184453', 'etd-1113103-124913', 'etd-11112011-103621', 'etd-07072004-170332', 'etd-04072005-130656', 'etd-04052015-110001', 'etd-04062015-133013', 'etd-06062016-160049', 'etd-11052014-174731', 'etd-06112014-143801', 'etd-04132011-183705', 'etd-04122005-142648', 'etd-11122013-174659', 'etd-07112008-110053', 'etd-09252014-133624', 'etd-04272012-011005', 'etd-11112004-121033', 'etd-06082015-171518', 'etd-11102004-115241', 'etd-01122012-174758', 'etd-01262012-095741', 'etd-0407103-225827', 'etd-07082011-113804', 'etd-1113103-163532', 'etd-04082016-011901', 'etd-0114102-153450', 'etd-06152010-201208', 'etd-04012008-101702', 'etd-06292016-132414', 'etd-08242010-164758', 'etd-07132015-125201', 'etd-06222007-110425', 'etd-06242010-111608', 'etd-01242005-113710', 'etd-11122007-184535', 'etd-01232012-154927', 'etd-05132009-120913', 'etd-11162010-175412', 'etd-07062005-132141', 'etd-06092011-101057', 'etd-04072009-011724', 'etd-11112010-224811', 'etd-11132006-112930', 'etd-04152013-155839', 'etd-11132008-111446', 'etd-0904102-160204', 'etd-04142009-160434', 'etd-07062005-095648', 'etd-06272008-114733', 'etd-02192014-202235', 'etd-0904103-135750', 'etd-01232006-212520', 'etd-04062006-155730', 'etd-04252011-070227', 'etd-06272014-134004', 'etd-04122013-180236', 'etd-07062007-194455', 'etd-07092008-143317', 'etd-1112102-100748', 'etd-07062010-155144', 'etd-11112015-143631', 'etd-03232006-072655', 'etd-04162009-203608', 'etd-0407102-002158', 'etd-08302010-175207', 'etd-02092016-224832', 'etd-01232014-124134', 'etd-03222015-113413', 'etd-06092008-090752', 'etd-07122015-180034', 'etd-0124102-111609', 'etd-11092008-134145', 'etd-11102008-093334', 'etd-04052016-172626', 'etd-1105102-150556', 'etd-07082016-095655', 'etd-04102014-203801', 'etd-07032012-113855', 'etd-11102010-205858', 'etd-03062013-185135', 'etd-07132006-123523', 'etd-0129103-004815', 'etd-04042013-113120', 'etd-1119102-163547', 'etd-04232012-113036', 'etd-03272006-193913', 'etd-04152013-155609', 'etd-04072004-104249', 'etd-05202004-143653', 'etd-10122004-135222', 'etd-04082009-100641', 'etd-06132005-171707', 'etd-04192013-131237', 'etd-04282010-164251', 'etd-04042008-093335', 'etd-11122015-210639', 'etd-01222013-111807', 'etd-0701102-195347', 'etd-0407103-102749', 'etd-05262016-162346', 'etd-04142005-160118', 'etd-0703103-143335', 'etd-07072008-133941', 'etd-04132005-161415', 'etd-08172009-160422', 'etd-07032010-194437', 'etd-12092011-143546', 'etd-12192011-114553', 'etd-10292004-110654', 'etd-0703103-120331', 'etd-06072004-141016', 'etd-03222005-094129', 'etd-04022007-223148', 'etd-04202011-174839', 'etd-04062015-115239', 'etd-11122015-225412', 'etd-04072011-150934', 'etd-12092004-230815', 'etd-04112007-162105', 'etd-11152012-163816', 'etd-04072005-115036', 'etd-11072011-084435', 'etd-04142011-220952', 'etd-06272013-101731', 'etd-04152013-114747', 'etd-07102009-101519', 'etd-04292010-122824', 'etd-04142013-222653', 'etd-05272015-221203', 'etd-07082004-113315', 'etd-10012008-094630', 'etd-0321102-113852', 'etd-11102004-152234', 'etd-12032004-070104', 'etd-08292007-110755', 'etd-0225103-174014', 'etd-11022012-085228', 'etd-11162015-103146', 'etd-10102011-105127', 'etd-04142005-032052', 'etd-07062007-112342', 'etd-11042004-215509', 'etd-04192006-102931', 'etd-04122005-105836', 'etd-07112007-113005', 'etd-11192010-103220', 'etd-04242012-124136', 'etd-04092013-200636', 'etd-0626103-163535', 'etd-11092004-102719', 'etd-11182010-204302', 'etd-0707103-141310', 'etd-07062005-165558', 'etd-03292013-112915', 'etd-04072016-074848', 'etd-04132007-091145', 'etd-04112007-133416', 'etd-01192004-014024', 'etd-04202011-155934', 'etd-06102009-081716', 'etd-07032012-150016', 'etd-04082016-102524', 'etd-11112009-230423', 'etd-11102004-043712', 'etd-08092007-160251', 'etd-06092005-110839', 'etd-12122003-141334', 'etd-11162006-121407', 'etd-03042010-081654', 'etd-11182013-141849', 'etd-0118102-150932', 'etd-04082014-144051', 'etd-04142014-133302', 'etd-06192009-113231', 'etd-04082016-190401', 'etd-10222015-220527', 'etd-04012014-164441', 'etd-11172013-133147', 'etd-0708103-124302', 'etd-08242015-141926', 'etd-01242005-220608', 'etd-10312007-094056', 'etd-07092009-133247', 'etd-05242009-173947', 'etd-11162012-104109', 'etd-11142007-174431', 'etd-1111102-124457', 'etd-06082009-115927', 'etd-04102008-152426', 'etd-06302011-133644', 'etd-11122007-220957', 'etd-07092008-100712', 'etd-03092012-103220', 'etd-07112005-180647', 'etd-01272010-180434', 'etd-07082009-221337', 'etd-09162011-113342', 'etd-04112005-201256', 'etd-07022009-174343', 'etd-04122007-135119', 'etd-11172014-144400', 'etd-06252014-171237', 'etd-04292011-114651', 'etd-12112012-171800', 'etd-11112011-103225', 'etd-05252012-133308', 'etd-07042011-003953', 'etd-04162012-104207', 'etd-07272012-180852', 'etd-11052008-203747', 'etd-06112007-150715', 'etd-07062004-082131', 'etd-04132005-181109', 'etd-05312007-143223', 'etd-10302013-142750', 'etd-1110103-200134', 'etd-12152004-111012', 'etd-11142007-110724', 'etd-04012013-135410', 'etd-0707103-145026', 'etd-11162005-155731', 'etd-11102011-231423', 'etd-11172005-123005', 'etd-04302010-121841', 'etd-1115101-152410', 'etd-11122012-092220', 'etd-06102010-152234', 'etd-11152014-111103', 'etd-0409103-110550', 'etd-07152005-093546', 'etd-11142005-172101', 'etd-11102009-114651', 'etd-07012013-113825', 'etd-1109102-124123', 'etd-04262012-163640', 'etd-01262012-143445', 'etd-04252011-152953', 'etd-04062010-221559', 'etd-01182008-133454', 'etd-0608102-023345', 'etd-04112008-085356', 'etd-1112103-092155', 'etd-07072011-103258', 'etd-07142005-134611', 'etd-04172012-123015', 'etd-1114103-001446', 'etd-04132005-163217', 'etd-04142005-120018', 'etd-05112007-132523', 'etd-04282011-124147', 'etd-06142005-152111', 'etd-06192009-011453', 'etd-04062014-153057', 'etd-11152012-160909', 'etd-04152013-113601', 'etd-07012004-091858', 'etd-01242007-233119', 'etd-07102009-095343', 'etd-07052015-043332', 'etd-06072013-182710', 'etd-06102004-124306', 'etd-01222009-121258', 'etd-11132014-111510', 'etd-04272011-163508', 'etd-06052012-142835', 'etd-11162005-173601', 'etd-08072014-172950', 'etd-07072004-124535', 'etd-07122012-232344', 'etd-0331103-130706', 'etd-04052008-195424', 'etd-05302013-141118', 'etd-07062013-132327', 'etd-07082004-102126', 'etd-04162010-152611', 'etd-04122007-092927', 'etd-10012012-125615', 'etd-04082016-140957', 'etd-07082011-100324', 'etd-06232006-082316', 'etd-04122007-215338', 'etd-04052006-145017', 'etd-11092008-142649', 'etd-11112004-154152', 'etd-09012007-145510', 'etd-11062014-163059', 'etd-09242007-122040', 'etd-07032013-144441', 'etd-04052016-010115', 'etd-05132010-220628', 'etd-04182012-091617', 'etd-11172014-141748', 'etd-07072005-095958', 'etd-06042015-133806', 'etd-04112008-080231', 'etd-04262012-142938', 'etd-06022006-161456', 'etd-0411102-144434', 'etd-04112013-142903', 'etd-1116101-094520', 'etd-05132013-142333', 'etd-09102006-232313', 'etd-06302016-230620', 'etd-04102008-155457', 'etd-04262012-114034', 'etd-0522103-113110', 'etd-11152012-174353', 'etd-04122005-200327', 'etd-11072008-213756', 'etd-04092007-155436', 'etd-1017103-140518', 'etd-04142014-115000', 'etd-04132013-121418', 'etd-11102006-113703', 'etd-04252012-201208', 'etd-04022015-131512', 'etd-07082014-121704', 'etd-04062006-090126', 'etd-04152010-162352', 'etd-04162009-175537', 'etd-07102015-222844', 'etd-0929103-165159', 'etd-07102007-155715', 'etd-04162012-140729', 'etd-0425102-225712', 'etd-11142015-230745', 'etd-07122006-152037', 'etd-11142006-165021', 'etd-03272009-142614', 'etd-12182006-123257', 'etd-08172011-133813', 'etd-11142012-040550', 'etd-11042015-124404', 'etd-06212006-133037', 'etd-04272011-140713', 'etd-04142011-132212', 'etd-0114102-162311', 'etd-11092004-091237', 'etd-0410103-231917', 'etd-06092011-103357', 'etd-04052006-162459', 'etd-07252012-152241', 'etd-03172005-000448', 'etd-1010102-080917', 'etd-04112016-152320', 'etd-1026102-231639', 'etd-06012006-154808', 'etd-1220102-000202', 'etd-01022016-173452', 'etd-04092007-222043', 'etd-04152013-113343', 'etd-04262010-165549', 'etd-01122006-131338', 'etd-04082016-141919', 'etd-03232013-134110', 'etd-07062012-103459', 'etd-02172011-125639', 'etd-01222008-120610', 'etd-04152004-134854', 'etd-08192005-161333', 'etd-11072014-140615', 'etd-0110102-085711', 'etd-04082008-173825', 'etd-06082016-215945', 'etd-11122009-184612', 'etd-11022015-092653', 'etd-04132005-144806', 'etd-05282008-235527', 'etd-05282010-091957', 'etd-11122014-144352', 'etd-10302004-221748', 'etd-1112103-160619', 'etd-05222010-151216', 'etd-04162009-084910', 'etd-04142005-075005', 'etd-04102008-122811', 'etd-11152005-124938', 'etd-04092009-111949', 'etd-04142009-171239', 'etd-07052012-101705', 'etd-07082015-221240', 'etd-04032006-222421', 'etd-04132005-093708', 'etd-0710103-102634', 'etd-03292011-190359', 'etd-04122009-173747', 'etd-08052010-163557', 'etd-11102012-142437', 'etd-08222007-015144', 'etd-11172006-064635', 'etd-0326102-172237', 'etd-03282005-165158', 'etd-05272016-162513', 'etd-11102011-115906', 'etd-04062016-203518', 'etd-09242014-172917', 'etd-0411102-135053', 'etd-03242016-114229', 'etd-04142014-083941', 'etd-11162006-111100', 'etd-07102006-190506', 'etd-09132008-223612', 'etd-0417102-143155', 'etd-06112007-162947', 'etd-11162015-143750', 'etd-04102014-114029', 'etd-04052016-185104', 'etd-0711103-093202', 'etd-11152006-102208', 'etd-08312005-130609', 'etd-11092012-100002', 'etd-01312008-125913', 'etd-04112014-185904', 'etd-07032007-152113', 'etd-07032007-112751', 'etd-04272010-231859', 'etd-04102013-094841', 'etd-08312010-162305', 'etd-07032012-110918', 'etd-01152007-182354', 'etd-04262012-143455', 'etd-07072004-114606', 'etd-01112013-164000', 'etd-06292008-173809', 'etd-05202008-170015', 'etd-02022010-202558', 'etd-04192012-180014', 'etd-07142006-114036', 'etd-04292010-154023', 'etd-04072006-082646', 'etd-04142005-184449', 'etd-07142014-124616', 'etd-1110103-105638', 'etd-0416102-140916', 'etd-07062015-093505', 'etd-04072009-095448', 'etd-08252015-170029', 'etd-04152011-004352', 'etd-04132009-122420', 'etd-1110103-153129', 'etd-03092012-140419', 'etd-11152005-124419', 'etd-04132012-155252', 'etd-04102012-094620', 'etd-04042012-082959', 'etd-10232013-225748', 'etd-01272010-145525', 'etd-05312012-200839', 'etd-0701102-111853', 'etd-06032015-141412', 'etd-04042007-121040', 'etd-10192013-150711', 'etd-11162005-175017', 'etd-06032004-112424', 'etd-11042011-095705', 'etd-05312012-212545', 'etd-04152005-010441', 'etd-02202015-161715', 'etd-0305103-143757', 'etd-11152007-171346', 'etd-07052011-143318', 'etd-04152009-140513', 'etd-11192013-155408', 'etd-11102011-004836', 'etd-08272009-165154', 'etd-04142014-145546', 'etd-0811103-164117', 'etd-07102012-202739', 'etd-05092013-141605', 'etd-05042006-150256', 'etd-04152013-123002', 'etd-11102008-134955', 'etd-05242004-104611', 'etd-0407103-105949', 'etd-08062010-100101', 'etd-04292011-112248', 'etd-06272008-152925', 'etd-04142005-184207', 'etd-03312006-200152', 'etd-11152005-135305', 'etd-04122007-084759', 'etd-04122007-121853', 'etd-11102014-202909', 'etd-07092009-093842', 'etd-0903102-104034', 'etd-04052007-175636', 'etd-04212011-132444', 'etd-11082006-161338', 'etd-07072010-171808', 'etd-09042014-133832', 'etd-04262012-221515', 'etd-12012004-132618', 'etd-08152005-112116', 'etd-04122013-101453', 'etd-08272012-155249', 'etd-06112013-170137', 'etd-0402103-115549', 'etd-0610102-120359', 'etd-04102008-122558', 'etd-07122006-141211', 'etd-0122102-171421', 'etd-08262004-121210', 'etd-04262011-131720', 'etd-04152013-111500', 'etd-01222013-155000', 'etd-11072007-114346', 'etd-0610103-223721', 'etd-07142014-131712', 'etd-04112014-154157', 'etd-04102014-114955', 'etd-1109101-104807', 'etd-04072016-155622', 'etd-07122005-101758', 'etd-04252011-120026', 'etd-04092008-120514', 'etd-07142005-114202', 'etd-07072011-124809', 'etd-1113101-171156', 'etd-04102015-115030', 'etd-04122013-211410', 'etd-06282006-132639', 'etd-0409103-015539', 'etd-04272011-123059', 'etd-03282014-100612', 'etd-01212009-142138', 'etd-07102012-123331', 'etd-05202011-154316', 'etd-03182013-140832', 'etd-04302010-120028', 'etd-11132007-105030', 'etd-07102008-112235', 'etd-07072004-130134', 'etd-04292010-213554', 'etd-06072010-130848', 'etd-0710102-132751', 'etd-04072006-112642', 'etd-05292015-181515', 'etd-04042007-213813', 'etd-04152009-124237', 'etd-11092006-170821', 'etd-04062006-102829', 'etd-05202011-160100', 'etd-01192016-151705', 'etd-10212015-201521', 'etd-03262015-084047', 'etd-11152005-113447', 'etd-0410103-193544', 'etd-07092012-114044', 'etd-0605103-194013', 'etd-08242015-224539', 'etd-05272004-162258', 'etd-07042007-150741', 'etd-07122005-163632', 'etd-11112008-182237', 'etd-05162016-095915', 'etd-01142008-171414', 'etd-0306102-132545', 'etd-1113103-164538', 'etd-03282006-122132', 'etd-04082016-094654', 'etd-11182010-154014', 'etd-06132013-160753', 'etd-06292012-084125', 'etd-05022010-091452', 'etd-04092009-090436', 'etd-07032013-013835', 'etd-0809102-163100', 'etd-04022008-104941', 'etd-04272010-160358', 'etd-06012007-095017', 'etd-04072015-170427', 'etd-07052011-154022', 'etd-06022007-114446', 'etd-04132010-215312', 'etd-0630103-102630', 'etd-04102016-231746', 'etd-06102006-003016', 'etd-06112014-180005', 'etd-04072016-130238', 'etd-10222004-092627', 'etd-11172014-125059', 'etd-04082014-224245', 'etd-1019101-205937', 'etd-0212103-124037', 'etd-07082010-104436', 'etd-05302007-112522', 'etd-07092007-182525', 'etd-1101102-224710', 'etd-04192011-231558', 'etd-11092005-155756', 'etd-04142005-115636', 'etd-03212013-203144', 'etd-0711103-101559', 'etd-04152009-165600', 'etd-0116103-074250', 'etd-11102011-101240', 'etd-04272011-102751', 'etd-0708103-114809', 'etd-07122005-135934', 'etd-05312005-190711', 'etd-06142006-134336', 'etd-01042012-150704', 'etd-05312016-023959', 'etd-04022004-161418', 'etd-06042015-131341', 'etd-01082007-175058', 'etd-06302014-134907', 'etd-11122009-153318', 'etd-10302008-040624', 'etd-04082014-115552', 'etd-11182010-193155', 'etd-04112016-132757', 'etd-04022013-160638', 'etd-03122009-120024', 'etd-01242005-145140', 'etd-04122004-144735', 'etd-07092009-164501', 'etd-0415102-134111', 'etd-06272015-170924', 'etd-04172009-120423', 'etd-07072010-161657', 'etd-0526103-140935', 'etd-05292009-165311', 'etd-04072015-091931', 'etd-06122006-125429', 'etd-11072014-130735', 'etd-06272015-024831', 'etd-11092004-111800', 'etd-04142014-105341', 'etd-04122005-171051', 'etd-01172005-051703', 'etd-01222013-095832', 'etd-04092009-091121', 'etd-04142005-115901', 'etd-1114102-133418', 'etd-0710102-212205', 'etd-07112016-075801', 'etd-04032006-143345', 'etd-03242011-193729', 'etd-03312014-143820', 'etd-04292010-145244', 'etd-11032010-184010', 'etd-04072016-131616', 'etd-04042013-184850', 'etd-04142009-144643', 'etd-0822103-121632', 'etd-03142010-223313', 'etd-1112103-152602', 'etd-06082004-232601', 'etd-07102009-110514', 'etd-06242015-142701', 'etd-07082013-124416', 'etd-04292010-204438', 'etd-05312013-113630', 'etd-04012013-231840', 'etd-06022014-114214', 'etd-07072016-114345', 'etd-01092011-141358', 'etd-12112014-094116', 'etd-04122011-182849', 'etd-11142007-183128', 'etd-0409103-200342', 'etd-03212014-100226', 'etd-0418102-220607', 'etd-07052012-080553', 'etd-11092011-162245', 'etd-08172004-130755', 'etd-0902102-221252', 'etd-1115102-090126', 'etd-04152013-113649', 'etd-1113102-222139', 'etd-11162005-182307', 'etd-04122007-135529', 'etd-07092007-154123', 'etd-03172015-121942', 'etd-04262012-152948', 'etd-04102014-144149', 'etd-11052008-123052', 'etd-0707103-201715', 'etd-12122011-105427', 'etd-04112013-111718', 'etd-04112009-113634', 'etd-0731102-120443', 'etd-06142006-164510', 'etd-1113102-140006', 'etd-03242004-175322', 'etd-05252015-224839', 'etd-04162009-153820', 'etd-06302004-134707', 'etd-11092012-135102', 'etd-07102015-164800', 'etd-11102009-173141', 'etd-06152006-154657', 'etd-07052011-133220', 'etd-04092007-114034', 'etd-0829103-144532', 'etd-08212015-220714', 'etd-11162014-191107', 'etd-04292010-111443', 'etd-12022004-010717', 'etd-04212005-095210', 'etd-12192014-113424', 'etd-06092010-150557', 'etd-11182013-151831', 'etd-04132012-173812', 'etd-0409103-121920', 'etd-08212007-161342', 'etd-07012013-160705', 'etd-11142014-150115', 'etd-04162012-080300', 'etd-07122013-114602', 'etd-0712102-092659', 'etd-11092009-102004', 'etd-11052004-113146', 'etd-11182010-161021', 'etd-11122007-154057', 'etd-04062015-175312', 'etd-04132005-173111', 'etd-11152013-134645', 'etd-01092008-082825', 'etd-04132014-174432', 'etd-08252011-120930', 'etd-06132007-155336', 'etd-0903103-182255', 'etd-04122007-142419', 'etd-04112005-125206', 'etd-06052013-151242', 'etd-06102013-153909', 'etd-07022013-145716', 'etd-04082015-205232', 'etd-11092010-161740', 'etd-11172010-000558', 'etd-08202004-143655', 'etd-04092015-122012', 'etd-06032015-144508', 'etd-0417102-132600', 'etd-06122008-100133', 'etd-08312007-102354', 'etd-07092012-114911', 'etd-04142014-105844', 'etd-04112013-114815', 'etd-11142012-132731', 'etd-08312004-140811', 'etd-06252013-145027', 'etd-07122007-142022', 'etd-05252012-140911', 'etd-04112011-082450', 'etd-06142005-153505', 'etd-04122010-185023', 'etd-10232011-234737', 'etd-01272004-203526', 'etd-06122008-092433', 'etd-04122007-003429', 'etd-06092006-175221', 'etd-04272012-094822', 'etd-03172014-105946', 'etd-11112013-231733', 'etd-06102013-180819', 'etd-04142014-090546', 'etd-04062008-163931', 'etd-09072005-135429', 'etd-0130102-072730', 'etd-11092009-232207', 'etd-04212011-091441', 'etd-04132005-091652', 'etd-07082013-121705', 'etd-1104103-132505', 'etd-06252013-192115', 'etd-04042014-164139', 'etd-04092014-174606', 'etd-07182005-072533', 'etd-01142008-150207', 'etd-07092007-134803', 'etd-11102009-160031', 'etd-06132007-105238', 'etd-0603103-102219', 'etd-11162006-154246', 'etd-07032012-164215', 'etd-04152009-124621', 'etd-03292006-161251', 'etd-08212015-174445', 'etd-07082015-102028', 'etd-11172005-160955', 'etd-06182015-155915', 'etd-06302013-163935', 'etd-04112005-153743', 'etd-0530102-220448', 'etd-04072006-110457', 'etd-04262011-151723', 'etd-11182013-120951', 'etd-07022004-151651', 'etd-0416102-175706', 'etd-04112007-113057', 'etd-06102013-125557', 'etd-04072008-112214', 'etd-0827102-124236', 'etd-04072016-133334', 'etd-05262014-145753', 'etd-10282010-173615', 'etd-1108102-152630', 'etd-11152007-173114', 'etd-11092011-092137', 'etd-03132015-113819', 'etd-07112008-082618', 'etd-06132007-053922', 'etd-04112014-113732', 'etd-06132006-072250', 'etd-11102015-095831', 'etd-04112007-180913', 'etd-11132015-120151', 'etd-08312005-104009', 'etd-06072010-161925', 'etd-04112007-190908', 'etd-04272012-092853', 'etd-04012008-230410', 'etd-0131102-125405', 'etd-11182005-094704', 'etd-07082013-165746', 'etd-04292010-141731', 'etd-05202010-073251', 'etd-03282008-140240', 'etd-01102014-112535', 'etd-04042013-104148', 'etd-07082007-102140', 'etd-04132005-122603', 'etd-04032007-191751', 'etd-04142005-161100', 'etd-0418102-084838', 'etd-04302010-141955', 'etd-05192016-193017', 'etd-04262012-130414', 'etd-07082004-235053', 'etd-04162009-183500', 'etd-07302010-174023', 'etd-03232005-144548', 'etd-04152005-085655', 'etd-0415102-092019', 'etd-04102008-101701', 'etd-11082011-131922', 'etd-07032007-100117', 'etd-04152004-125755', 'etd-11142013-104507', 'etd-04072010-101356', 'etd-04272011-105721', 'etd-11212015-001003', 'etd-0125102-125349', 'etd-0702102-182619', 'etd-06282013-234535', 'etd-06062012-161518', 'etd-11042015-125939', 'etd-04022014-111815', 'etd-07092008-223941', 'etd-04282010-111748', 'etd-04142004-154547', 'etd-0419102-103048', 'etd-10312005-150203', 'etd-07112016-130357', 'etd-03172016-121747', 'etd-10242011-153604', 'etd-04292010-145246', 'etd-07082007-222626', 'etd-11152005-085556', 'etd-06092004-150805', 'etd-05302007-183453', 'etd-11122007-141814', 'etd-04072015-181424', 'etd-04052016-113640', 'etd-10282008-094243', 'etd-0416102-153709', 'etd-07102015-113746', 'etd-04152005-110009', 'etd-0402103-125735', 'etd-07072004-211420', 'etd-0129103-132127', 'etd-11162005-152703', 'etd-11142005-124931', 'etd-07032012-131830', 'etd-04152004-125305', 'etd-07082004-152330', 'etd-12192013-153010', 'etd-09282005-014119', 'etd-01042007-143202', 'etd-01142016-154448', 'etd-07012009-173743', 'etd-0327103-145843', 'etd-04142010-124602', 'etd-05152007-163915', 'etd-1113102-193925', 'etd-07142005-082657', 'etd-06042014-143007', 'etd-11092004-132617', 'etd-11022005-130425', 'etd-1114102-110301', 'etd-01192007-162240', 'etd-07062015-144158', 'etd-06292011-152332', 'etd-07102007-164658', 'etd-04152004-135621', 'etd-0703102-120155', 'etd-11092012-190748', 'etd-04192010-200102', 'etd-04032012-143710', 'etd-11182010-115435', 'etd-04202012-150457', 'etd-04242012-084700', 'etd-11132008-163044', 'etd-0711102-192732', 'etd-11092009-162025', 'etd-03122013-195835', 'etd-04142009-221615', 'etd-07032009-075202', 'etd-05302014-145309', 'etd-06062007-234426', 'etd-04012014-115814', 'etd-02152006-093546', 'etd-04082016-141120', 'etd-04252012-112642', 'etd-04122007-125608', 'etd-1111101-154752', 'etd-04042014-142133', 'etd-0905102-101559', 'etd-10212009-091310', 'etd-11142007-113721', 'etd-04262012-111929', 'etd-07082009-112606', 'etd-04092015-161851', 'etd-04142005-131028', 'etd-07082010-112039', 'etd-04262011-131328', 'etd-04132004-154747', 'etd-11102012-131425', 'etd-06292006-011812', 'etd-11152006-090410', 'etd-05282016-164345', 'etd-11162005-092852', 'etd-07122015-144221', 'etd-11162015-010522', 'etd-04152013-121105', 'etd-12292003-151736', 'etd-07072006-124838', 'etd-11132007-120220', 'etd-09182013-230111', 'etd-04152005-104631', 'etd-04022013-122329', 'etd-04142005-104058', 'etd-07022010-174317', 'etd-09292006-142420', 'etd-0821102-133045', 'etd-04142009-220821', 'etd-1112103-211729', 'etd-07022013-075843', 'etd-07122005-104553', 'etd-0711102-130955', 'etd-06062012-124944', 'etd-0405102-094607', 'etd-01242006-165635', 'etd-04082014-084012', 'etd-01242005-151053', 'etd-03162016-204808', 'etd-11112010-150548', 'etd-04132015-155341', 'etd-06302015-151438', 'etd-11162006-133628', 'etd-04092008-154108', 'etd-04252011-214015', 'etd-0710102-172407', 'etd-07022015-144703', 'etd-07152005-083738', 'etd-06112014-160931', 'etd-11112011-102243', 'etd-04262011-140320', 'etd-04102008-202907', 'etd-08032011-165733', 'etd-06082004-211423', 'etd-06072005-105003', 'etd-0407103-163959', 'etd-08022005-201105', 'etd-04262011-083851', 'etd-04142004-073338', 'etd-0218103-102118', 'etd-11182013-154902', 'etd-06062011-083753', 'etd-04152013-104450', 'etd-0707103-111615', 'etd-06272016-163928', 'etd-07102006-102745', 'etd-04112008-111948', 'etd-05312013-190705', 'etd-1114102-125456', 'etd-04102014-163828', 'etd-06062008-162427', 'etd-04172013-161513', 'etd-04142004-151850', 'etd-11102004-121616', 'etd-11162007-105056', 'etd-11072011-233133', 'etd-04082016-153742', 'etd-11142012-120234', 'etd-11182010-093506', 'etd-04232012-160428', 'etd-0407103-142502', 'etd-06122006-192002', 'etd-11042009-152651', 'etd-04012013-185947', 'etd-04142013-141514', 'etd-01282004-111538', 'etd-0613102-131926', 'etd-03282015-124357', 'etd-11182010-093717', 'etd-06042015-071316', 'etd-04142004-140341', 'etd-04102008-120826', 'etd-0604102-203723', 'etd-1011103-211310', 'etd-04242012-152544', 'etd-07142005-163536', 'etd-04292010-164207', 'etd-11182013-161937', 'etd-0820102-120526', 'etd-07102008-112549', 'etd-12202003-145735', 'etd-11132007-195952', 'etd-06282011-144209', 'etd-01152010-143232', 'etd-08162012-192406', 'etd-11092004-194025', 'etd-01122015-110335', 'etd-04032006-151427', 'etd-1030103-122315', 'etd-07102016-072850', 'etd-02152005-112641', 'etd-07112006-192620', 'etd-06102009-142109', 'etd-07032007-152559', 'etd-12172013-184644', 'etd-04082014-155955', 'etd-07082009-090811', 'etd-04082016-152528', 'etd-1112103-155132', 'etd-0410103-101627', 'etd-04132004-193119', 'etd-06102005-104340', 'etd-07062007-202840', 'etd-06092012-125419', 'etd-0408103-100711', 'etd-04062005-112716', 'etd-04122011-150559', 'etd-07262012-164128', 'etd-07062016-152319', 'etd-03282006-172832', 'etd-06102009-141512', 'etd-04142005-174821', 'etd-07132006-094127', 'etd-11132007-111128', 'etd-07062015-144155', 'etd-06282008-141328', 'etd-07132005-131432', 'etd-04022008-150434', 'etd-11042008-200745', 'etd-08262011-154518', 'etd-04172012-230225', 'etd-07142005-104310', 'etd-11082005-201642', 'etd-0707103-121017', 'etd-07142005-121243', 'etd-06142005-122736', 'etd-08112010-163505', 'etd-0409103-143112', 'etd-07052010-214944', 'etd-11162015-003712', 'etd-07122006-213208', 'etd-03182012-012256', 'etd-0710102-160806', 'etd-1102103-142417', 'etd-04142005-182108', 'etd-04282010-214847', 'etd-08012013-105649', 'etd-10282006-083841', 'etd-03302011-144750', 'etd-07012010-102256', 'etd-11142006-155008', 'etd-11092009-124748', 'etd-11182005-112454', 'etd-06292015-115429', 'etd-04062015-200926', 'etd-0612102-192601', 'etd-11172006-091911', 'etd-11132007-142447', 'etd-0701103-133352', 'etd-04202012-194149', 'etd-0416102-130906', 'etd-04092013-213418', 'etd-10162012-191021', 'etd-05282014-194756', 'etd-1109103-211240', 'etd-07092010-090934', 'etd-01262004-172257', 'etd-04152004-154733', 'etd-05262015-071931', 'etd-04152013-080048', 'etd-09302004-204524', 'etd-04102007-155324', 'etd-01222013-120106', 'etd-11162013-165344', 'etd-10052007-134825', 'etd-0322102-120443', 'etd-07092007-173548', 'etd-1110103-093730', 'etd-04122010-111056', 'etd-06032011-113926', 'etd-07082004-231554', 'etd-05282004-132928', 'etd-06282011-231408', 'etd-0404103-084700', 'etd-04062015-202226', 'etd-06102014-160351', 'etd-11032009-155858', 'etd-04062009-231810', 'etd-04072016-082328', 'etd-07012016-120651', 'etd-07042012-073819', 'etd-04032015-064229', 'etd-07032012-115431', 'etd-04292010-183739', 'etd-07142014-114956', 'etd-04142009-110232', 'etd-07132010-155540', 'etd-04172012-223524', 'etd-11042005-205028', 'etd-11162014-140834', 'etd-05252016-125133', 'etd-11142006-145204', 'etd-04042016-153937', 'etd-08232011-162938', 'etd-04122010-104444', 'etd-04032007-134615', 'etd-02082016-140401', 'etd-04152005-100640', 'etd-04102013-085656', 'etd-11132012-154618', 'etd-11112004-190648', 'etd-1218101-192458', 'etd-06102014-192930', 'etd-04072016-125417', 'etd-11172006-085843', 'etd-04132004-152641', 'etd-05252016-161222', 'etd-1108102-173656', 'etd-04212011-160350', 'etd-11162006-161946', 'etd-07062011-175440', 'etd-06082004-120408', 'etd-11132013-185808', 'etd-07082004-131755', 'etd-07082005-102907', 'etd-07132007-115140', 'etd-11162006-141744', 'etd-07092010-100458', 'etd-05022011-134403', 'etd-04152004-113304', 'etd-08282008-172107', 'etd-0612102-154406', 'etd-04152013-000651', 'etd-11152010-155245', 'etd-04072004-165638', 'etd-04272012-115354', 'etd-11182013-091403', 'etd-07112014-161623', 'etd-11122012-221157', 'etd-11182010-094457', 'etd-07072010-110814', 'etd-04082008-154511', 'etd-07112007-150118', 'etd-06192012-193933', 'etd-04122007-203010', 'etd-04012008-094410', 'etd-0408103-120613', 'etd-07052012-115636', 'etd-04262012-102943', 'etd-04252011-170557', 'etd-04262011-114601', 'etd-07042011-145547', 'etd-07082010-190449', 'etd-11142008-093201', 'etd-07102008-154311', 'etd-11152005-222951', 'etd-04062016-205400', 'etd-12252015-111106', 'etd-04142010-140721', 'etd-03262014-130020', 'etd-11022004-201139', 'etd-11192015-130223', 'etd-01172016-085106', 'etd-05142014-151024', 'etd-03312016-152816', 'etd-11162005-090455', 'etd-10072013-131225', 'etd-0710102-143553', 'etd-08132014-112428', 'etd-0409103-132537', 'etd-07022015-153504', 'etd-04102007-145400', 'etd-06232010-092447', 'etd-07092008-163739', 'etd-04052006-154112', 'etd-04022015-113042', 'etd-11162007-070659', 'etd-01212009-113752', 'etd-07112016-131032', 'etd-04272012-103715', 'etd-04112013-093644', 'etd-04092013-114158', 'etd-11022008-215511', 'etd-07032012-001837', 'etd-10052006-103803', 'etd-07142014-111405', 'etd-04112005-155454', 'etd-1106103-120852', 'etd-03282007-205843', 'etd-06072006-154916', 'etd-04142005-224929', 'etd-05162004-005042', 'etd-07012012-231402', 'etd-11112010-214621', 'etd-03192013-151957', 'etd-04072016-222101', 'etd-06242011-024451', 'etd-04132009-134102', 'etd-01232014-210446', 'etd-0402103-124721', 'etd-04192006-211101', 'etd-07062011-111805', 'etd-04162013-113306', 'etd-11172005-132928', 'etd-07032013-222250', 'etd-11132008-124813', 'etd-04142005-222948', 'etd-06052013-113438', 'etd-04102008-141642', 'etd-09112009-113732', 'etd-04102016-172724', 'etd-11122014-140041', 'etd-04132005-153906', 'etd-04142009-172437', 'etd-04152004-123822', 'etd-1114101-125240', 'etd-04052005-164748', 'etd-07132014-152427', 'etd-04042016-165521', 'etd-04092013-202552', 'etd-11072010-113943', 'etd-11052015-091409', 'etd-04052016-134740', 'etd-04152004-184542', 'etd-04142014-150231', 'etd-04142014-144736', 'etd-07052006-105643', 'etd-04142014-141223', 'etd-04272012-094024', 'etd-04162009-175920', 'etd-06102004-105322', 'etd-09142004-151633', 'etd-09132004-185202', 'etd-04112013-143408', 'etd-11022010-202656', 'etd-04132007-094858', 'etd-11022014-153444', 'etd-12182012-143602', 'etd-01192014-020829', 'etd-07132005-160615', 'etd-04272010-161031', 'etd-11122009-161943', 'etd-0707103-170819', 'etd-04282011-115609', 'etd-04072004-192953', 'etd-11132012-170856', 'etd-10082012-132131', 'etd-07042008-000142', 'etd-05302007-170756', 'etd-01192016-141405', 'etd-07092008-134855', 'etd-06192012-115845', 'etd-04032008-133617', 'etd-12302003-121228', 'etd-07062006-154231', 'etd-06092008-192351', 'etd-08272004-134745', 'etd-04142014-022121', 'etd-04022007-171744', 'etd-1114101-191429', 'etd-0408103-170754', 'etd-04242012-211906', 'etd-06072004-202100', 'etd-11142012-113110', 'etd-07062012-122105', 'etd-10272005-123634', 'etd-09012005-145941', 'etd-04162010-160714', 'etd-11132008-182023', 'etd-07122005-165749', 'etd-04142014-144240', 'etd-04232012-164221', 'etd-11112008-175227', 'etd-04102014-103551', 'etd-11092005-092833', 'etd-07102008-153109', 'etd-05232016-172358', 'etd-03152010-193125', 'etd-04162010-123309', 'etd-07082009-114923', 'etd-07102006-185322', 'etd-04062016-160949', 'etd-07082004-110035', 'etd-0707102-155816', 'etd-09012006-163824', 'etd-06202013-173741', 'etd-11072008-013716', 'etd-06092009-215555', 'etd-11102004-143547', 'etd-0403103-162930', 'etd-01202009-143411', 'etd-11122009-160534', 'etd-06102009-100440', 'etd-07112007-000114', 'etd-05312007-143648', 'etd-07052013-120311', 'etd-11102014-214623', 'etd-01282004-104948', 'etd-07072014-110647', 'etd-11122009-141820', 'etd-07122015-150842', 'etd-11042011-080759', 'etd-04272012-103738', 'etd-0417102-170705', 'etd-11102011-172016', 'etd-04122007-105140', 'etd-04012006-152300', 'etd-06252012-100224', 'etd-07122006-160111', 'etd-07082013-144346', 'etd-06192014-080930', 'etd-04252011-174119', 'etd-06102009-153456', 'etd-01212015-093510', 'etd-05242011-081918', 'etd-04072016-215358', 'etd-04102007-015922', 'etd-11172010-225345', 'etd-07072007-125734', 'etd-0401103-155852', 'etd-05102012-161822', 'etd-04112014-071640', 'etd-11152005-152738', 'etd-04162009-202352', 'etd-04032006-161914', 'etd-06142005-181128', 'etd-07072009-204750', 'etd-0712102-093702', 'etd-1110103-225402', 'etd-11072014-122259', 'etd-04072004-180134', 'etd-07072004-220634', 'etd-06082011-131020', 'etd-04052007-121331', 'etd-07022008-174027', 'etd-06112008-122621', 'etd-04262012-234223', 'etd-11182013-152753', 'etd-11062009-155508', 'etd-06092010-221901', 'etd-04082015-163502', 'etd-04162009-161149', 'etd-11122004-110638', 'etd-0410103-065154', 'etd-11142006-001747', 'etd-01072010-234822', 'etd-05202014-114042', 'etd-04052006-161157', 'etd-05072009-110114', 'etd-06072011-163746', 'etd-07022015-145444', 'etd-07092014-162114', 'etd-06102004-083658', 'etd-04102008-134537', 'etd-07132015-093738', 'etd-07122014-185212', 'etd-07112008-090700', 'etd-11112011-095906', 'etd-04062009-193611', 'etd-0413102-132317', 'etd-09032007-131732', 'etd-07022013-073722', 'etd-11162015-005043', 'etd-06282006-102815', 'etd-07032012-105955', 'etd-11072014-160426', 'etd-11072014-184156', 'etd-01122015-013948', 'etd-11102009-113627', 'etd-05312012-145237', 'etd-11152013-112056', 'etd-01162014-150947', 'etd-08272004-102750', 'etd-06292008-165012', 'etd-08182005-213847', 'etd-11092015-123040', 'etd-04152005-080526', 'etd-06082008-222244', 'etd-0127103-211300', 'etd-12212012-105358', 'etd-11072014-120427', 'etd-11122008-165855', 'etd-06132006-102125', 'etd-0415102-205738', 'etd-04152009-122340', 'etd-11112009-125538', 'etd-04152013-153824', 'etd-04062009-164922', 'etd-0416102-085340', 'etd-01272010-150437', 'etd-11162005-093854', 'etd-11152005-125056', 'etd-1111102-112243', 'etd-07132015-115310', 'etd-04142009-103050', 'etd-04262011-204528', 'etd-07032012-124945', 'etd-07062007-144234', 'etd-0813103-143524', 'etd-11182010-035738', 'etd-07122006-143331', 'etd-11172014-094019', 'etd-06052007-112453', 'etd-04092007-163417', 'etd-1031103-133101', 'etd-04112005-150526', 'etd-0710102-160310', 'etd-11162013-163007', 'etd-01062012-130016', 'etd-04252012-155457', 'etd-09122013-070549', 'etd-07142014-134628', 'etd-09032006-150231', 'etd-11182013-115645', 'etd-06192007-181425', 'etd-04062016-193223', 'etd-06102009-030258', 'etd-08082014-191107', 'etd-07132014-023345', 'etd-04062016-121004', 'etd-04102012-103841', 'etd-07072011-163921', 'etd-07032012-160303', 'etd-0611102-124326', 'etd-07102009-115117', 'etd-04242012-111048', 'etd-06082010-004937', 'etd-08232012-123106', 'etd-01172014-202130', 'etd-04042006-150248', 'etd-10152013-123414', 'etd-04302010-144347', 'etd-04012016-184124', 'etd-04152005-101825', 'etd-07092007-172712', 'etd-06092004-172848', 'etd-0127103-164555', 'etd-02262013-111738', 'etd-12222014-121513', 'etd-11172014-111704', 'etd-04092012-194710', 'etd-11172005-131031', 'etd-04132005-143242', 'etd-01102014-174829', 'etd-0710103-223610', 'etd-07082014-212417', 'etd-06232005-124454', 'etd-11112011-101742', 'etd-11102004-095808', 'etd-04282010-153305', 'etd-07052013-151800', 'etd-0710102-054039', 'etd-11162007-064655', 'etd-07092008-054914', 'etd-0924102-205423', 'etd-0401103-121512', 'etd-07122006-101110', 'etd-05312011-135439', 'etd-04112014-091556', 'etd-11032015-141209', 'etd-04122005-110354', 'etd-07122005-003916', 'etd-07092010-102353', 'etd-01212005-114146', 'etd-04142004-111444', 'etd-06152005-140544', 'etd-04152013-104028', 'etd-06122006-223212', 'etd-04012015-230000', 'etd-11142005-130338', 'etd-0711102-124445', 'etd-07112007-080524', 'etd-11052007-215229', 'etd-04052007-100551', 'etd-06092014-145316', 'etd-01252006-141752', 'etd-07132005-181504', 'etd-02232005-181427', 'etd-05192004-205606', 'etd-08112014-133148', 'etd-05132016-141042', 'etd-06092011-111457', 'etd-07122006-124938', 'etd-0326102-145701', 'etd-06082008-153915', 'etd-06032007-204040', 'etd-0515103-111714', 'etd-07062005-070755', 'etd-01272004-163120', 'etd-05312005-182116', 'etd-04122007-153806', 'etd-07092015-112925', 'etd-0818103-153127', 'etd-01212009-142106', 'etd-0712102-112337', 'etd-03222010-003817', 'etd-07072008-202058', 'etd-04142005-103625', 'etd-11122009-130445', 'etd-0606102-115820', 'etd-04112007-144822', 'etd-05312011-162739', 'etd-07142006-104355', 'etd-11072013-134342', 'etd-04182012-164219', 'etd-0711102-133745', 'etd-0604103-113808', 'etd-04102008-153039', 'etd-11162012-115803', 'etd-07102012-125006', 'etd-11132007-102209', 'etd-11052013-181632', 'etd-0513102-144354', 'etd-0610103-135503', 'etd-11132008-152312', 'etd-01172013-215017', 'etd-04012016-092003', 'etd-05022010-103435', 'etd-04102016-232717', 'etd-05312006-163518', 'etd-11072014-162204', 'etd-11062009-124817', 'etd-04232012-151422', 'etd-06072004-125021', 'etd-08242015-155429', 'etd-10282005-154037', 'etd-12182012-114405', 'etd-06022011-181641', 'etd-11152010-092521', 'etd-0605103-160256', 'etd-11102011-154243', 'etd-06132006-210624', 'etd-10132004-211301', 'etd-04152013-094329', 'etd-07132007-102318', 'etd-05022010-214403', 'etd-04142009-103539', 'etd-0708102-145931', 'etd-0417102-143711', 'etd-06202012-102753', 'etd-04112008-083508', 'etd-04092014-140936', 'etd-04172009-093009', 'etd-0612103-131621', 'etd-05302012-154547', 'etd-04112016-113128', 'etd-04192012-102434', 'etd-10152015-154629', 'etd-04082005-105715', 'etd-07092009-124015', 'etd-04182014-234551', 'etd-05172006-102745', 'etd-11142012-150127', 'etd-05182016-164247', 'etd-09092008-122747', 'etd-1121103-010631', 'etd-04082013-105503', 'etd-06142006-153308', 'etd-04182012-154159', 'etd-1114102-104706', 'etd-04152009-092929', 'etd-01162009-170305', 'etd-06092004-091046', 'etd-11052006-094055', 'etd-01252006-141357', 'etd-07122007-214128', 'etd-04082015-095530', 'etd-07062016-154527', 'etd-06062014-104953', 'etd-04012004-144102', 'etd-1108102-101121', 'etd-1115102-131146', 'etd-05092007-150605', 'etd-04082014-142044', 'etd-11072011-122138', 'etd-11132013-184130', 'etd-0708103-232016', 'etd-06282012-165206', 'etd-01192014-205944', 'etd-04092008-143047', 'etd-04152011-100130', 'etd-08272013-162913', 'etd-07052012-171558', 'etd-11172010-095959', 'etd-11032005-144604', 'etd-06032014-162153', 'etd-08162012-135354', 'etd-03252010-132523', 'etd-11162005-171011', 'etd-07082010-093213', 'etd-0630103-113609', 'etd-07142006-102431', 'etd-07032007-125943', 'etd-0626103-132651', 'etd-04092008-225506', 'etd-11162005-100643', 'etd-06092010-230857', 'etd-0108103-140103', 'etd-11152010-144051', 'etd-04082008-123312', 'etd-11112008-201927', 'etd-0905102-143754', 'etd-06282013-225852', 'etd-0423102-144103', 'etd-03312014-165620', 'etd-03282015-143929', 'etd-04112016-160002', 'etd-04152013-102438', 'etd-09222008-164905', 'etd-04112016-142710', 'etd-1114102-141915', 'etd-03112007-160900', 'etd-04072009-105447', 'etd-0710103-103435', 'etd-07092008-111442', 'etd-04122005-123751', 'etd-01252006-153425', 'etd-05232014-211726', 'etd-0417102-233950', 'etd-11132013-134216', 'etd-0407103-131120', 'etd-04092008-131720', 'etd-03232011-210718', 'etd-05312013-000147', 'etd-1109101-100345', 'etd-12182014-155530', 'etd-08272014-215649', 'etd-07052012-203541', 'etd-06102014-232243', 'etd-06232004-162858', 'etd-0413102-142601', 'etd-07142009-134947', 'etd-04052006-222512', 'etd-11082015-212415', 'etd-04082014-222817', 'etd-0418102-141608', 'etd-0131102-132358', 'etd-0405103-143959', 'etd-04132009-141607', 'etd-11192013-162819', 'etd-04262011-150353', 'etd-11142008-112505', 'etd-01262005-135906', 'etd-04142004-104615', 'etd-11102009-190927', 'etd-07082006-180148', 'etd-04172011-223913', 'etd-11132009-120424', 'etd-11072007-085256', 'etd-05012006-125319', 'etd-1106103-132359', 'etd-04192011-123723', 'etd-07072010-000611', 'etd-04252011-075708', 'etd-04112013-143638', 'etd-04142013-143134', 'etd-11112009-204537', 'etd-11052004-114545', 'etd-03312006-170606', 'etd-1012101-102114', 'etd-11152007-221012', 'etd-04062015-175341', 'etd-07092014-094325', 'etd-04132013-181557', 'etd-07102012-113238', 'etd-06232015-115007', 'etd-0702102-164050', 'etd-05252005-170310', 'etd-08242010-140559', 'etd-07082010-190228', 'etd-04112016-115851', 'etd-0127103-175535', 'etd-11152010-094121', 'etd-05262014-121554', 'etd-08142014-114904', 'etd-02192015-182651', 'etd-10232009-125139', 'etd-11132008-130753', 'etd-04162012-095021', 'etd-0903103-141321', 'etd-11112004-113024', 'etd-04142010-162912', 'etd-05032010-110259', 'etd-10312011-102824', 'etd-06132005-160237', 'etd-01262010-103858', 'etd-07092012-122240', 'etd-04272011-121244', 'etd-12162009-112641', 'etd-11142007-094408', 'etd-04092014-093842', 'etd-08282012-231239', 'etd-03282005-124326', 'etd-1107101-134928', 'etd-03282006-101408', 'etd-07012007-180241', 'etd-04072005-204752', 'etd-0418102-094358', 'etd-11182010-221606', 'etd-11072014-142009', 'etd-04112005-150748', 'etd-05292014-121633', 'etd-06052008-182416', 'etd-03312008-103036', 'etd-11162015-120813', 'etd-0612102-151846', 'etd-0702103-151700', 'etd-11112007-214948', 'etd-0327103-141013', 'etd-07092009-232534', 'etd-03142016-193545', 'etd-0708103-120755', 'etd-04122013-132014', 'etd-04282011-133304', 'etd-0610102-122818', 'etd-07012004-115042', 'etd-04222012-175048', 'etd-11162006-104012', 'etd-04122015-125210', 'etd-1113102-143537', 'etd-04272010-171250', 'etd-04282010-101710', 'etd-02042010-154352', 'etd-05222008-155157', 'etd-04172009-101049', 'etd-04122013-123414', 'etd-04152013-112448', 'etd-11182013-160957', 'etd-08262004-170344', 'etd-0601103-234303', 'etd-04152013-153146', 'etd-04162010-123115', 'etd-07072009-150815', 'etd-11152005-135514', 'etd-04032009-094316', 'etd-06162016-195108', 'etd-0526103-104932', 'etd-02252015-141240', 'etd-08212009-113914', 'etd-04082008-204445', 'etd-0902103-171251', 'etd-07032015-163206', 'etd-11032010-141154', 'etd-04062007-030010', 'etd-07112006-070802', 'etd-06022008-112535', 'etd-04012004-224204', 'etd-11032004-104040', 'etd-01082006-225902', 'etd-1107103-183524', 'etd-04232012-121947', 'etd-05192010-064633', 'etd-04022011-221554', 'etd-07082014-114853', 'etd-11072013-150451', 'etd-01212010-122722', 'etd-11122007-223504', 'etd-04162012-152016', 'etd-08282014-122133', 'etd-03152012-162627', 'etd-06052010-111924', 'etd-06242015-013156', 'etd-04152013-151436', 'etd-06082005-150533', 'etd-0708102-090233', 'etd-08222012-131201', 'etd-06262013-154541', 'etd-06072004-150022', 'etd-01232014-104043', 'etd-1029103-181924', 'etd-07132005-180936', 'etd-07082011-111617', 'etd-04152005-095943', 'etd-0411103-105200', 'etd-07012009-181133', 'etd-0411103-101753', 'etd-11092004-140538', 'etd-01242006-113255', 'etd-04152005-092745', 'etd-04142005-174336', 'etd-04072014-095955', 'etd-07062004-145926', 'etd-08072012-172506', 'etd-07012010-094214', 'etd-04262012-080430', 'etd-08202007-121603', 'etd-01242006-191616', 'etd-0326103-111821', 'etd-08272014-184335', 'etd-04192011-104718', 'etd-0603103-132428', 'etd-04102008-185157', 'etd-04052004-140700', 'etd-03262013-095613', 'etd-07062010-105602', 'etd-04062009-195507', 'etd-06032008-131749', 'etd-06292016-092055', 'etd-04042006-114658', 'etd-08112015-143235', 'etd-04152004-114700', 'etd-06012012-115855', 'etd-1111102-205855', 'etd-11072011-175708', 'etd-04112012-121135', 'etd-04032006-161002', 'etd-08192005-171247', 'etd-02032015-104358', 'etd-11142005-140210', 'etd-01122015-083045', 'etd-04152004-145043', 'etd-09202006-223014', 'etd-08302013-095329', 'etd-11132008-115852', 'etd-04272010-135131', 'etd-04132004-154236', 'etd-06062011-130134', 'etd-06202008-131226', 'etd-03172015-084434', 'etd-11122007-003804', 'etd-04032006-163030', 'etd-11052004-143204', 'etd-04072006-115441', 'etd-01232014-130926', 'etd-04082008-125746', 'etd-01222013-133321', 'etd-07292008-134312', 'etd-07052011-104813', 'etd-06132012-224706', 'etd-09082005-165133', 'etd-1111103-152345', 'etd-06052007-182855', 'etd-04102007-120515', 'etd-0411102-155836', 'etd-04022007-235744', 'etd-07072009-135413', 'etd-11092005-083248', 'etd-06122014-102709', 'etd-04262012-095714', 'etd-03172006-200922', 'etd-04142009-173520', 'etd-04152013-095442', 'etd-12212011-104540', 'etd-11112015-101344', 'etd-07102014-205910', 'etd-07142005-093123', 'etd-04072010-124421', 'etd-04072013-151645', 'etd-04272011-164628', 'etd-11182013-120505', 'etd-0609103-222622', 'etd-06152012-162448', 'etd-04042016-234206', 'etd-10192004-154516', 'etd-04192010-000122', 'etd-07072009-212904', 'etd-01192004-053159', 'etd-04142014-151305', 'etd-0709102-132242', 'etd-11132007-174026', 'etd-04142004-150747', 'etd-03282006-100706', 'etd-07122015-112034', 'etd-07132005-220653', 'etd-07022013-163055', 'etd-01242005-143159', 'etd-06282005-103739', 'etd-05302013-132131', 'etd-04132012-200451', 'etd-04162009-115921', 'etd-07022013-072304', 'etd-08262005-133712', 'etd-03092015-105146', 'etd-06162005-142747', 'etd-06252013-120151', 'etd-03312006-120306', 'etd-10042015-152131', 'etd-03022007-141419', 'etd-11162005-003854', 'etd-1219102-152426', 'etd-04042008-101908', 'etd-04252012-175722', 'etd-04082004-142650', 'etd-04052011-235207', 'etd-11262012-110801', 'etd-1115102-003746', 'etd-11132013-110048', 'etd-04122015-225258', 'etd-07152005-092326', 'etd-07132015-135047', 'etd-04132015-145548', 'etd-0410103-164030', 'etd-0402102-135533', 'etd-09072006-162924', 'etd-01252012-095733', 'etd-04122005-195932', 'etd-04082004-102706', 'etd-03112015-225124', 'etd-09052006-093802', 'etd-04102015-122035', 'etd-09152009-013617', 'etd-10192004-085142', 'etd-07072009-170832', 'etd-0207103-072636', 'etd-11072014-104549', 'etd-04052006-124009', 'etd-06302010-092323', 'etd-0709103-132756', 'etd-04052015-140327', 'etd-04102014-231811', 'etd-0730103-162224', 'etd-06092008-093632', 'etd-0411102-131419', 'etd-04192011-095645', 'etd-11132008-162559', 'etd-11102015-220515', 'etd-11022011-153905', 'etd-05302008-103815', 'etd-11102004-220727', 'etd-01232008-225658', 'etd-1115102-091515', 'etd-04132004-115513', 'etd-11152006-231527', 'etd-07092006-125400', 'etd-05212014-204423', 'etd-07092010-103454', 'etd-09262008-150825', 'etd-04212011-135340', 'etd-04152013-133156', 'etd-04022015-142237', 'etd-0417102-211325', 'etd-06242010-091330', 'etd-08312011-112450', 'etd-04132015-140713', 'etd-01252005-144734', 'etd-07052013-150000', 'etd-04032015-195731', 'etd-02052010-141640', 'etd-04102008-111620', 'etd-04012015-090711', 'etd-04032006-004254', 'etd-05312005-145010', 'etd-11182010-125539', 'etd-04142011-131411', 'etd-05022006-110316', 'etd-03312006-174712', 'etd-08262014-214928', 'etd-10262010-153502', 'etd-06122014-143919', 'etd-03242009-155921', 'etd-01242006-180955', 'etd-07072006-164320', 'etd-05152009-144335', 'etd-04092014-140326', 'etd-11142014-084827', 'etd-04062015-113329', 'etd-11122004-110029', 'etd-0408103-104408', 'etd-05312008-103652', 'etd-07092009-100847', 'etd-11112005-123648', 'etd-11192010-122754', 'etd-0416102-115928', 'etd-06102005-122235', 'etd-04082008-152801', 'etd-03132006-171644', 'etd-11182013-142237', 'etd-0828103-110339', 'etd-04112014-144547', 'etd-04052005-105607', 'etd-06042014-135957', 'etd-05312006-205307', 'etd-04072008-153622', 'etd-0409103-160949', 'etd-01192007-164521', 'etd-07102008-233340', 'etd-07022014-151226', 'etd-04152013-093857', 'etd-04122007-133815', 'etd-04192013-141011', 'etd-04142004-150323', 'etd-0325103-115548', 'etd-07022012-221205', 'etd-01182007-162211', 'etd-11132008-102953', 'etd-04222012-193250', 'etd-04092015-175351', 'etd-0102103-235307', 'etd-04032008-214416', 'etd-11122012-150141', 'etd-11102010-141206', 'etd-04112016-153536', 'etd-10252004-205544', 'etd-06012012-164646', 'etd-04082013-145504', 'etd-04012015-142115', 'etd-04152013-152031', 'etd-01212004-125820', 'etd-07062004-141750', 'etd-01082015-095853', 'etd-08222006-201052', 'etd-08252011-120545', 'etd-04012004-193144', 'etd-07082004-120520', 'etd-02142013-151412', 'etd-07072004-112736', 'etd-0710103-142959', 'etd-07142005-221136', 'etd-03022004-132725', 'etd-01272011-160024', 'etd-04252011-083956', 'etd-11142007-202651', 'etd-04142004-163818', 'etd-11072012-170328', 'etd-06272011-224217', 'etd-0619102-182614', 'etd-01232006-133456', 'etd-1112103-210714', 'etd-08222009-204359', 'etd-11112004-161757', 'etd-0711102-080847', 'etd-11182013-115504', 'etd-04202011-155813', 'etd-01132004-235239', 'etd-11152006-174359', 'etd-04082013-213532', 'etd-04152013-121702', 'etd-11132007-134449', 'etd-06182012-152543', 'etd-06032006-113117', 'etd-06022012-103658', 'etd-05012006-115234', 'etd-10292010-144052', 'etd-06162005-144248', 'etd-07012013-191036', 'etd-04262012-120907', 'etd-0710103-051947', 'etd-11132006-165441', 'etd-04262012-101634', 'etd-10122004-191951', 'etd-03282006-160416', 'etd-11192008-161805', 'etd-04132005-192407', 'etd-04082009-173718', 'etd-04142004-115456', 'etd-11082007-103811', 'etd-07122012-100858', 'etd-11142012-145713', 'etd-11172010-142309', 'etd-01212009-023639', 'etd-05282015-094657', 'etd-08132012-134537', 'etd-0905102-111038', 'etd-11072009-170319', 'etd-04112016-162313', 'etd-01232006-230233', 'etd-03182016-150501', 'etd-11042004-125810', 'etd-04022015-101529', 'etd-04292010-161009', 'etd-11132015-105344', 'etd-06032014-165859', 'etd-06112009-083231', 'etd-07082014-154610', 'etd-01202011-204235', 'etd-04082008-104150', 'etd-04042014-085932', 'etd-11052012-144412', 'etd-07062012-063441', 'etd-04232012-211924', 'etd-04142009-201914', 'etd-09042008-163420', 'etd-0326103-135506', 'etd-04052010-130458', 'etd-04252011-224457', 'etd-1112103-135018', 'etd-04242012-092033', 'etd-04032008-144658', 'etd-0830102-090058', 'etd-11072007-114349', 'etd-12082010-161626', 'etd-06042012-104004', 'etd-04252012-191716', 'etd-04142004-121130', 'etd-07062010-153822', 'etd-0417102-151129', 'etd-04152013-124346', 'etd-06042008-133647', 'etd-10312008-143141', 'etd-04092014-112705', 'etd-04112007-224534', 'etd-0709103-163751', 'etd-03272008-085840', 'etd-01222009-123844', 'etd-07082010-224847', 'etd-09062006-164036', 'etd-11162012-110819', 'etd-11142014-161351', 'etd-04152005-104143', 'etd-0609103-112838', 'etd-11102004-180012', 'etd-04072015-124358', 'etd-07082009-111852', 'etd-0408102-211030', 'etd-1003101-172351', 'etd-04022004-141738', 'etd-08312010-115028', 'etd-04132004-154150', 'etd-01162007-115703', 'etd-01072009-112827', 'etd-11092007-093218', 'etd-03312010-120227', 'etd-04292010-183907', 'etd-07162012-103728', 'etd-07062012-180210', 'etd-10192009-110325', 'etd-06152006-141418', 'etd-04112013-110621', 'etd-07052005-163922', 'etd-11082004-231115', 'etd-1204102-123159', 'etd-11032006-081627', 'etd-1112103-135205', 'etd-11132007-104055', 'etd-11182010-143648', 'etd-04052006-111228', 'etd-04142014-114058', 'etd-06022004-222715', 'etd-11072013-100517', 'etd-06162014-153414', 'etd-04102008-125136', 'etd-05182016-124518', 'etd-0417102-170452', 'etd-11192010-114102', 'etd-10182007-182636', 'etd-0419102-100900', 'etd-11122015-154737', 'etd-06182012-232710', 'etd-04292011-095502', 'etd-04072009-114852', 'etd-10242012-145420', 'etd-11112005-193802', 'etd-04042006-134727', 'etd-0905102-114913', 'etd-0408102-132617', 'etd-07082013-162923', 'etd-08162009-211103', 'etd-05212016-164108', 'etd-04262012-094534', 'etd-07032015-145141', 'etd-04072004-132752', 'etd-07052012-181939', 'etd-11082014-014326', 'etd-04032006-214922', 'etd-04072005-210612', 'etd-1107102-140122', 'etd-1207101-182907', 'etd-04102007-212509', 'etd-07022007-165132', 'etd-04262011-095652', 'etd-07112007-175513', 'etd-1023102-142927', 'etd-05112016-123444', 'etd-12152004-124203', 'etd-06302011-080255', 'etd-1110103-174217', 'etd-11172014-142533', 'etd-04192012-100213', 'etd-04072005-170522', 'etd-10112011-102624', 'etd-06182013-192815', 'etd-11162015-121120', 'etd-04142011-122038', 'etd-03032011-163036', 'etd-04142014-124445', 'etd-10262011-213536', 'etd-07092010-114420', 'etd-0821101-100809', 'etd-06022006-161425', 'etd-07072004-142401', 'etd-04142005-103606', 'etd-12212015-102447', 'etd-1112103-104824', 'etd-07092015-145255', 'etd-11102004-161129', 'etd-04242012-152129', 'etd-07132005-133401', 'etd-04252011-122547', 'etd-08162012-080805', 'etd-05162015-201418', 'etd-04032008-161153', 'etd-04112016-143012', 'etd-04062006-134604', 'etd-11042005-161219', 'etd-04092015-154746', 'etd-04132015-124237', 'etd-07012016-112242', 'etd-11122007-025400', 'etd-04052016-180241', 'etd-07142005-124550', 'etd-0419102-091551', 'etd-04072005-145922', 'etd-11042009-151525', 'etd-07072010-140653', 'etd-11042009-012159', 'etd-07122007-142343', 'etd-11092006-115814', 'etd-09292009-201004', 'etd-03302007-173916', 'etd-07022012-202919', 'etd-04122007-113729', 'etd-06212006-215531', 'etd-04152009-092827', 'etd-11182010-135949', 'etd-11112014-141234', 'etd-04192010-152017', 'etd-0419102-102014', 'etd-06112009-101628', 'etd-0317103-102113', 'etd-1103103-095701', 'etd-0129103-155734', 'etd-11112014-183906', 'etd-07072007-222408', 'etd-1031101-111553', 'etd-0609103-125359', 'etd-11072006-150203', 'etd-07162004-141309', 'etd-04262012-175018', 'etd-06142006-074506', 'etd-04152012-152439', 'etd-07022013-161458', 'etd-07082015-223841', 'etd-11082004-213020', 'etd-04282011-103834', 'etd-06012005-220446', 'etd-07082009-144715', 'etd-11172006-113518', 'etd-04292010-235934', 'etd-08082007-164218', 'etd-11012007-193518', 'etd-11162006-160527', 'etd-1113102-145806', 'etd-10012007-152915', 'etd-06082006-151124', 'etd-0402103-135539', 'etd-07102015-163427', 'etd-04292010-222634', 'etd-07122012-151431', 'etd-10202015-221459', 'etd-04152011-142614', 'etd-10312011-162534', 'etd-0410102-113624', 'etd-06072007-110638', 'etd-11102010-200649', 'etd-04202011-160003', 'etd-07032013-141006', 'etd-0407103-125118', 'etd-11132007-155015', 'etd-12312011-230123', 'etd-07062007-103649', 'etd-04062015-222201', 'etd-06282012-152606', 'etd-07072006-165000', 'etd-11152005-092832', 'etd-05232015-003120', 'etd-04152013-083440', 'etd-04112005-120103', 'etd-03202012-125334', 'etd-07062015-125314', 'etd-09072005-093949', 'etd-06052015-100427', 'etd-11192010-115249', 'etd-08302006-123936', 'etd-07122006-225442', 'etd-07072011-164926', 'etd-04142009-113817', 'etd-0401103-143500', 'etd-04042014-134757', 'etd-03292005-150730', 'etd-0320102-151248', 'etd-0611102-102309', 'etd-04262011-084817', 'etd-11152014-115047', 'etd-11162007-100525', 'etd-07082015-090121', 'etd-04152004-114153', 'etd-11122008-203810', 'etd-07112011-132809', 'etd-11132012-174922', 'etd-07112008-025037', 'etd-04132005-163806', 'etd-02092015-131221', 'etd-05312005-113524', 'etd-07042016-220053', 'etd-0610103-102401', 'etd-01162016-125159', 'etd-07132005-210635', 'etd-07072010-193605', 'etd-04032006-174214', 'etd-02242008-205541', 'etd-04262012-110937', 'etd-06062012-150935', 'etd-07042013-184335', 'etd-1113102-220210', 'etd-02212010-133607', 'etd-11172005-185717', 'etd-08302007-225637', 'etd-04102008-105208', 'etd-04142004-120201', 'etd-0709103-111512', 'etd-0710102-120938', 'etd-07022013-114311', 'etd-08202011-150424', 'etd-06082011-094408', 'etd-04122013-170418', 'etd-04022013-171358', 'etd-0703103-124620', 'etd-04222011-144421', 'etd-11042015-170123', 'etd-11132012-132152', 'etd-04262011-211319', 'etd-0417102-151016', 'etd-0612102-105228', 'etd-1029102-213903', 'etd-10282011-105603', 'etd-04082015-104914', 'etd-07072005-113808', 'etd-12092004-152022', 'etd-11112015-110605', 'etd-07012013-224034', 'etd-1112102-105425', 'etd-11122009-105909', 'etd-04152004-142117', 'etd-0326102-171815', 'etd-07112006-125540', 'etd-04122010-081149', 'etd-11052009-182655', 'etd-11162014-164225', 'etd-04142013-233215', 'etd-04292010-185535', 'etd-01182008-140636', 'etd-06302004-153939', 'etd-04192011-120345', 'etd-07022012-112838', 'etd-07092010-102940', 'etd-07102014-114015', 'etd-07082010-153124', 'etd-04032006-165227', 'etd-10112006-113524', 'etd-04142009-142923', 'etd-04082008-154729', 'etd-04072016-123535', 'etd-05302012-150041', 'etd-04132015-123846', 'etd-11112014-192757', 'etd-11092012-160754', 'etd-04152010-163157', 'etd-04222012-100629', 'etd-01202015-115708', 'etd-04062006-125655', 'etd-04142004-104506', 'etd-06112011-104922', 'etd-04052014-154919', 'etd-07022004-145101', 'etd-07022013-090004', 'etd-07102008-202016', 'etd-06102008-194156', 'etd-11102004-154214', 'etd-07102012-133351', 'etd-07062010-081041', 'etd-06152006-135542', 'etd-07122007-103137', 'etd-06152012-170311', 'etd-08192010-132021', 'etd-04112008-095202', 'etd-01242013-131911', 'etd-11122010-115305', 'etd-06292010-133339', 'etd-04302010-062044', 'etd-11082011-121323', 'etd-07062016-103539', 'etd-07072016-131553', 'etd-07062016-121952', 'etd-07082014-194007', 'etd-07112014-124343', 'etd-04142014-150125', 'etd-08292011-093058', 'etd-01142010-133415', 'etd-05302008-194733', 'etd-07092008-163356', 'etd-07072004-124015', ]
uploaded_urns.extend(['etd-0611102-115017', 'etd-0521103-095812', 'etd-10312005-235025', 'etd-10072007-130430', 'etd-1111103-195959', 'etd-09032007-204612', 'etd-06052006-083140', 'etd-04222010-105224', 'etd-0903103-205546', 'etd-06142005-160600', 'etd-07122006-102849', 'etd-04072005-111439', 'etd-06302004-182749', 'etd-06132005-121320', 'etd-0307101-134037', 'etd-11052011-161145', 'etd-09012004-114224', 'etd-0331103-072550', 'etd-10182007-110446', 'etd-11072007-130454', 'etd-08102006-081647', 'etd-0407103-164824', 'etd-04042006-170733', 'etd-11132009-083328', 'etd-10292007-124859', 'etd-0415102-005608', 'etd-04092016-072843', 'etd-01262004-193945', 'etd-06032004-133529', 'etd-04142005-103135', 'etd-11132012-162426', 'etd-06032010-071804', 'etd-0705102-113942', ])
uploaded_urns.extend(['etd-04102016-232827', 'etd-04262012-104000', 'etd-06052009-145905', 'etd-11052014-153520', 'etd-01172014-150850', 'etd-07062012-142142', 'etd-01132005-210955', 'etd-07112016-143714', 'etd-04092009-110952', 'etd-03142012-121519', 'etd-04112013-131010', 'etd-04102013-120216', 'etd-06062006-142409', 'etd-10302008-133452', 'etd-04142013-173949', 'etd-06032013-140227', 'etd-0124102-121448', 'etd-07072013-140202', 'etd-03032016-181315', 'etd-06222006-133421', 'etd-04092007-152238', 'etd-08042005-115838', 'etd-04032006-162838', 'etd-03272011-202942', 'etd-04172009-083023', 'etd-03312005-151954', 'etd-0125102-123818', 'etd-1104102-000906', 'etd-04142009-225600', 'etd-06232007-110351', 'etd-07122005-200150', 'etd-05272014-141325', 'etd-04212010-083030', 'etd-04212010-111944', 'etd-07032007-104652', 'etd-04112016-160152', 'etd-07092009-141241', 'etd-10282013-191905', 'etd-06172013-171344', 'etd-04062016-133732', 'etd-01262010-105903', 'etd-06282006-183323', 'etd-07012009-153533', 'etd-02242013-113411', 'etd-07012013-143919', 'etd-04152013-092646', 'etd-07072013-150752', 'etd-10252011-182837', 'etd-07082010-215028', 'etd-07082004-085745', 'etd-06302004-201016', 'etd-07072015-004918', 'etd-07112012-041823', 'etd-0107102-181103', 'etd-11092009-142356', 'etd-07102008-105943', 'etd-06202011-122520', 'etd-03252013-115725', 'etd-0612103-170849', 'etd-1113103-113408', 'etd-0710102-121458', 'etd-07152005-101224', 'etd-03162015-155318', 'etd-10022013-163614', 'etd-04082008-193344', 'etd-04262012-152207', 'etd-02022010-154429', 'etd-1113103-100724', 'etd-11022008-184757', 'etd-04012006-150124', 'etd-07042015-235931', 'etd-01222008-070558', 'etd-11152013-110148', 'etd-11102005-152407', 'etd-07102006-131728', 'etd-0701103-135033', 'etd-0130102-103857', 'etd-11082011-182511', 'etd-01272014-151353', 'etd-04142014-104624', 'etd-03262013-180714', 'etd-04072008-180332', 'etd-07112005-104419', 'etd-11092009-120419', 'etd-04132010-154654', 'etd-06252013-140304', 'etd-07072008-232214', 'etd-11092011-154402', 'etd-04152004-155704', 'etd-03072008-165726', 'etd-11152007-154114', 'etd-11092010-100056', 'etd-06012016-151746', 'etd-04072008-184811', 'etd-07042014-133808', 'etd-08062015-112957', 'etd-04142014-132956', 'etd-03082006-153046', 'etd-11112005-113750', 'etd-07102014-150352', 'etd-07112005-185010', 'etd-04182011-113049', 'etd-12212015-151455', 'etd-06292016-021029', 'etd-0903102-190634', 'etd-07062009-160543', 'etd-09182005-231121', 'etd-11172010-091050', 'etd-11132012-174953', 'etd-1107103-231802', 'etd-04152013-095629', 'etd-0418102-024341', 'etd-04042008-095620', 'etd-04122007-003255', 'etd-04082004-150238', 'etd-04122007-145924', 'etd-03262006-135648', 'etd-04032009-124752', 'etd-0417102-130949', 'etd-1110103-134954', 'etd-07112005-193458', 'etd-07112014-160259', 'etd-04092008-223420', 'etd-03042005-123430', 'etd-10272005-155654', 'etd-11142008-024331', 'etd-0319103-144641', 'etd-04152004-165814', 'etd-08032005-093651', 'etd-04072004-214149', 'etd-1107102-170208', 'etd-07092016-230129', 'etd-04022015-155412', 'etd-11122009-154604', 'etd-1112102-151209', 'etd-07072010-131821', 'etd-10222013-092249', 'etd-0707103-152144', 'etd-06142007-121955', 'etd-0406102-033213', 'etd-11062004-213102', 'etd-04042014-164813', 'etd-05312012-142854', 'etd-06162016-103256', 'etd-04072015-155115', 'etd-0410102-233318', 'etd-04142013-195038', 'etd-03202007-163153', 'etd-08192005-152601', 'etd-01202011-180121', 'etd-06302005-121732', 'etd-01192016-052319', 'etd-04142009-133745', 'etd-03282013-144017', 'etd-1109101-145509', 'etd-06152004-123008', 'etd-11012011-101720', 'etd-04102015-141538', 'etd-07062009-143626', 'etd-04122013-112146', 'etd-07112014-125706', 'etd-11172010-022830', 'etd-08022006-140010', 'etd-06292012-003727', 'etd-07052016-230958', 'etd-06232015-171843', 'etd-04042015-011112', 'etd-03312016-142746', 'etd-06102016-194109', 'etd-04122013-094826', 'etd-06232016-145848', 'etd-04022015-164836', 'etd-04132005-074319', 'etd-06192012-153816', 'etd-04122016-134804', 'etd-1110103-161818', 'etd-04142011-124037', 'etd-11082007-194406', 'etd-11112009-121622', 'etd-04262012-152205', 'etd-03242016-081146', 'etd-04012016-144952', 'etd-05252016-161418', 'etd-10162007-221252', 'etd-11012009-193730', 'etd-11152015-224741', 'etd-11122013-135958', 'etd-0903103-161902', 'etd-11062015-102222', 'etd-0706102-115040', 'etd-11052004-163310', 'etd-02222011-154621', 'etd-04072008-132025', 'etd-06092015-150519', 'etd-04202010-205653', 'etd-05272004-135621', 'etd-03052008-212159', 'etd-02202015-122943', 'etd-06062011-031330', 'etd-03072005-071555', 'etd-04172009-001138', 'etd-04062016-071840', 'etd-04092011-214746', 'etd-04052015-012706', 'etd-07052016-180347', 'etd-0415102-111547', 'etd-03292004-124246', 'etd-0705103-164712', 'etd-08252015-190022', 'etd-11112013-181807', 'etd-01072016-092541', 'etd-11092011-134928', 'etd-0626103-234527', 'etd-10232008-135021', 'etd-06022005-163534', 'etd-07012011-162203', 'etd-12282011-102156', 'etd-06082005-101010', 'etd-05192011-103451', 'etd-08212006-152120', 'etd-04122009-161120', 'etd-1019101-092354', 'etd-05102006-104506', 'etd-04142009-174010', 'etd-03242011-154544', 'etd-11112008-091659', 'etd-06272012-173654', 'etd-06172013-105703', 'etd-06012016-135948', 'etd-07132006-105503', 'etd-05092007-143026', 'etd-1106103-171111', 'etd-11102011-133823', 'etd-12162005-192904', 'etd-09022009-112005', 'etd-09152008-143521', 'etd-12152003-111636', 'etd-0407102-133614', 'etd-0829102-060923', 'etd-01062016-204726', 'etd-04262012-195539', 'etd-0409103-153131', 'etd-11092006-164234', 'etd-03312006-100431', 'etd-04032008-211137', 'etd-05042004-063609', 'etd-08202010-153424', 'etd-11142014-140557', 'etd-07192013-102207', 'etd-01102006-112619', 'etd-11072014-135555', 'etd-04152013-143750', 'etd-03092007-161802', 'etd-08222014-144719', 'etd-04172009-075916', 'etd-04212011-091948', 'etd-0708103-125358', 'etd-06222015-181044', 'etd-06012010-231720', 'etd-11122009-113851', 'etd-0609103-135757', 'etd-04112007-182226', 'etd-04102007-181504', 'etd-12092013-134948', 'etd-07052011-161631', 'etd-0404103-140717', 'etd-06212016-124300', 'etd-11112015-132336', 'etd-04062012-214523', 'etd-0415102-173827', 'etd-08212005-203641', 'etd-05202011-182552', 'etd-11152006-140553', 'etd-05232014-193901', 'etd-06172013-105108', 'etd-11132007-142342', 'etd-07062009-132639', 'etd-07102015-165945', 'etd-0214103-162012', 'etd-07112016-144554', 'etd-10162013-234347', 'etd-01212014-103346', 'etd-03142014-103046', 'etd-07132005-101937', 'etd-04142014-001215', 'etd-01082010-143837', 'etd-07102014-174446', 'etd-06272012-080110', 'etd-07022013-182906', 'etd-07022014-161114', 'etd-06112014-202230', 'etd-04152005-113904', 'etd-02282008-183035', 'etd-07062005-094112', 'etd-04062015-112725', 'etd-01282004-142920', 'etd-11112015-104559', 'etd-01122016-222953', 'etd-06282011-174613', 'etd-05132015-181512', 'etd-02202014-135120', 'etd-06022014-155539', 'etd-11132008-084612', 'etd-04082004-164705', 'etd-08142008-141720', 'etd-08102015-194446', 'etd-0902103-122344', 'etd-06012006-105940', 'etd-08312005-151603', 'etd-0403102-215300', 'etd-11162015-154823', 'etd-03252014-170304', 'etd-04022013-204739', 'etd-04112016-113911', 'etd-06032014-123906', 'etd-0602103-123059', 'etd-07022010-112516', 'etd-07092004-090922', 'etd-1028102-142230', 'etd-03252016-224603', 'etd-0905102-122350', 'etd-03232009-110239', 'etd-09202012-133728', 'etd-1103103-111054', 'etd-01242011-165701', 'etd-11072014-191602', 'etd-04112014-022415', 'etd-0409103-165104', 'etd-07012009-150925', 'etd-04152005-114301', 'etd-11172006-095153', 'etd-01222009-042457', 'etd-11072010-204354', 'etd-06242015-134353', 'etd-11032009-121739', 'etd-1107103-144724', 'etd-04112005-175540', 'etd-11082011-115900', 'etd-07022013-143720', 'etd-04262012-000345', 'etd-06052007-093854', 'etd-05182005-120737', 'etd-0402103-154736', 'etd-08222007-143725', 'etd-05182016-083215', 'etd-1105103-100340', 'etd-11182013-135014', 'etd-06302006-101434', 'etd-04042005-171306', 'etd-07142005-113026', 'etd-08162010-102550', 'etd-07022015-173357', 'etd-0708103-211606', 'etd-06222010-145323', 'etd-11102015-134533', 'etd-0404103-141239', 'etd-06192013-135121', 'etd-07112016-113124', 'etd-06302005-104805', 'etd-03162014-191410', 'etd-07082004-150311', 'etd-11152013-121357', 'etd-10162007-112305', 'etd-02072007-101535', 'etd-11082006-195059', 'etd-0612102-234815', 'etd-11142011-104614', 'etd-10152010-153405', 'etd-11142012-085642', 'etd-08132015-102510', 'etd-01232014-104155', 'etd-11132007-094713', 'etd-11162006-152427', 'etd-11202008-073220', 'etd-09012009-103329', 'etd-04112007-082344', 'etd-11062011-214738', 'etd-11092007-071610', 'etd-0903102-111626', 'etd-0403103-111832', 'etd-09182010-105119', 'etd-10242013-162311', 'etd-07012008-172605', 'etd-06162016-083309', 'etd-11072008-120338', 'etd-04152010-102429', 'etd-03252009-163342', 'etd-07022013-154323', 'etd-11042011-135735', 'etd-11162005-180657', 'etd-08212012-150412', 'etd-05252015-171624', 'etd-04152011-150931', 'etd-07082014-233129', 'etd-07122006-072639', 'etd-07102015-120911', 'etd-11082014-001347', 'etd-07102015-133319', 'etd-11062009-225138', 'etd-11142013-190618', 'etd-10282014-133803', 'etd-03292007-102554', 'etd-11052012-101847', 'etd-04112007-172506', 'etd-11052010-165512', 'etd-07082005-154655', 'etd-04142014-152127', 'etd-08222005-145449', 'etd-09062005-104654', 'etd-05232006-153150', 'etd-0412102-153450', 'etd-06012004-153913', 'etd-01182007-121019', 'etd-10202009-120627', 'etd-04132011-205535', 'etd-04262012-221519', 'etd-0710103-133035', 'etd-11122004-111847', 'etd-04072014-153552', 'etd-01082007-125902', 'etd-04052013-150210', 'etd-11072014-080817', 'etd-11142015-213354', 'etd-06062008-144403', 'etd-04162009-133304', 'etd-07132007-093939', 'etd-07092009-213127', 'etd-08072013-163649', 'etd-07072011-103933', 'etd-10092009-162534', 'etd-06262015-145429', 'etd-06292016-183423', 'etd-1110103-141920', 'etd-04072014-152859', 'etd-04132012-164527', 'etd-11122012-111035', 'etd-01252005-122322', 'etd-0217102-190537', 'etd-04172010-195700', 'etd-01032015-001851', 'etd-1111103-171256', 'etd-0611102-001208', 'etd-04062016-145438', 'etd-04192011-184316', 'etd-11132008-035422', 'etd-02202010-133406', 'etd-10272015-211853', 'etd-03212006-224737', 'etd-11072014-223156', 'etd-02102004-212454', 'etd-07012016-114839', 'etd-07252012-201449', 'etd-04212011-134645', 'etd-12202013-220625', 'etd-04012007-220927', 'etd-04032006-182820', 'etd-08042016-163916', 'etd-11072012-183049', 'etd-03072013-141830', 'etd-11112011-115315', 'etd-07072011-104742', 'etd-07072004-105247', 'etd-0417102-153157', 'etd-07032007-101841', 'etd-11082012-061946', 'etd-10142004-144705', 'etd-06062012-111138', 'etd-04112007-144751', 'etd-06212013-090226', 'etd-11162006-175114', 'etd-04112007-230414', 'etd-04102013-170059', 'etd-11062008-121913', 'etd-0405102-133349', 'etd-07032012-132140', 'etd-11122008-110301', 'etd-04102007-153340', 'etd-04052013-173453', 'etd-08302007-084124', 'etd-06022004-190806', 'etd-06272013-224014', 'etd-11092009-124212', 'etd-11072014-115410', 'etd-08072006-095041', 'etd-11032004-034832', 'etd-06072012-165838', 'etd-04142014-134154', 'etd-07052006-091527', 'etd-04152011-174740', 'etd-06262008-142201', 'etd-11142013-095056', 'etd-07052016-085127', 'etd-12132007-002831', 'etd-0612102-171120', 'etd-11132014-162508', 'etd-07032013-182611', 'etd-05182016-121901', 'etd-11092009-131616', 'etd-04032014-142458', 'etd-10312007-124354', 'etd-11072013-102139', 'etd-02222016-121647', 'etd-05272015-181458', 'etd-04152004-102159', 'etd-04102013-120012', 'etd-05132015-150923', 'etd-11032010-150610', 'etd-04112013-114232', 'etd-04072009-163106', 'etd-06132013-161429', 'etd-07032012-151159', 'etd-06072004-142209', 'etd-07292015-103419', 'etd-02022016-172217', 'etd-0712102-103747', 'etd-0627103-093457', 'etd-11082011-220644', 'etd-03122015-152520', 'etd-04092014-114053', 'etd-04132004-144402', 'etd-11152015-211541', 'etd-08272014-210813', 'etd-11082004-155505', 'etd-07162007-132432', 'etd-09102008-150741', 'etd-07082013-115915', 'etd-11032006-163022', 'etd-11192010-085050', 'etd-11022011-225010', 'etd-11062004-130541', 'etd-06302012-152750', 'etd-0405102-074904', 'etd-0407103-205042', 'etd-11082010-131647', 'etd-05252010-101928', 'etd-06082016-124639', 'etd-04142005-160028', 'etd-04202010-185750', 'etd-0709102-120453', 'etd-06102009-205400', 'etd-06282016-221816', 'etd-06012011-203430', 'etd-03052008-101313', 'etd-10062006-153636', 'etd-01242010-185628', 'etd-07032012-123551', 'etd-11162006-135027', 'etd-09202012-150334', 'etd-11022011-165822', 'etd-04182012-151443', 'etd-05252012-130050', 'etd-07022009-084939', 'etd-06262015-155245', 'etd-06272012-204609', 'etd-06202015-073503', 'etd-04072014-155944', 'etd-07012011-141828', 'etd-03272006-224513', 'etd-08192015-162751', 'etd-11042005-122348', 'etd-07062009-114849', 'etd-09222015-142043', 'etd-0827103-103409', 'etd-07062007-155333', 'etd-07092012-113356', 'etd-01262004-150127', 'etd-01282014-101732', 'etd-11102004-212928', 'etd-07092014-134014', 'etd-03092014-111930', 'etd-08272008-122113', 'etd-06232016-154210', 'etd-01252014-182538', 'etd-11072005-163200', 'etd-06252016-195957', 'etd-06012004-163350', 'etd-04052011-202414', 'etd-11082006-162523', 'etd-11092012-203649', 'etd-07132012-041706', 'etd-10312012-131445', 'etd-06282013-160141', 'etd-0602103-145436', 'etd-11112008-103627', 'etd-03262007-173132', 'etd-1112103-133259', 'etd-11142005-160319', 'etd-09282005-091721', 'etd-11022005-162404', 'etd-11022015-132932', 'etd-06302005-112618', 'etd-04072008-184524', 'etd-11082004-163843', 'etd-06212004-201435', 'etd-05192014-121401', 'etd-07112016-112004', 'etd-07112014-134803', 'etd-09252014-153646', 'etd-0813102-224530', 'etd-07052014-141237', 'etd-11042009-165355', 'etd-10032008-105249', 'etd-11142007-202701', 'etd-06022006-161901', 'etd-06062014-115148', 'etd-06302009-221900', 'etd-11152012-164957', 'etd-11112008-131942', 'etd-03282014-140002', 'etd-07082008-230409', 'etd-0703103-151402', 'etd-06132013-020939', 'etd-03142016-141053', 'etd-11042011-125339', 'etd-0819103-110929', 'etd-07072016-122243', 'etd-06262013-104843', 'etd-11132013-233423', 'etd-05202016-122544', 'etd-07132015-092244', 'etd-04262012-210042', 'etd-04152004-115516', 'etd-0325103-165637', 'etd-0418102-195129', 'etd-08112010-140057', 'etd-01112016-174339', 'etd-04202012-131127', 'etd-01242006-181753', 'etd-06212006-122847', 'etd-07062012-131952', 'etd-06102004-101211', 'etd-11092007-204825', 'etd-11102009-114408', 'etd-09182012-150731', 'etd-11172014-130516', 'etd-03242015-211546', 'etd-04102013-190507', 'etd-07072008-101023', 'etd-10222013-114013', 'etd-08102013-082306', 'etd-01202004-174310', 'etd-07112016-131235', 'etd-06182013-091219', 'etd-07052012-234402', 'etd-11162005-131958', 'etd-0418102-000316', 'etd-03212004-143251', 'etd-01172007-021340', 'etd-04042015-010146', 'etd-04022015-180739', 'etd-0419102-000538', 'etd-11162005-142155', 'etd-08102012-174538', 'etd-08022007-122825', 'etd-06062016-224015', 'etd-11172010-085322', 'etd-04142014-143035', 'etd-04252011-144734', 'etd-03242016-181529', 'etd-06012006-103009', 'etd-04072007-160534', 'etd-0901103-194446', 'etd-05152014-135123', 'etd-07092016-220939', 'etd-08132014-112841', 'etd-03082007-000723', 'etd-07022016-000813', 'etd-04022009-222146', 'etd-07092009-144958', 'etd-03232016-135113', 'etd-06082013-205000', 'etd-04082004-124842', 'etd-11142014-160628', 'etd-07092009-100152', 'etd-02152007-190320', 'etd-07112006-085946', 'etd-07082013-145752', 'etd-11102010-160752', 'etd-0610102-223350', 'etd-04082009-200008', 'etd-11092004-134056', 'etd-11072007-212654', 'etd-04262012-104146', 'etd-04012015-160918', 'etd-06282014-195818', 'etd-07132007-114639', 'etd-03252009-172523', 'etd-04152009-172347', 'etd-10032013-112549', 'etd-11142014-161205', 'etd-01222013-135505', 'etd-11022004-064101', 'etd-10272011-153327', 'etd-03262015-093452', 'etd-08272014-150308', 'etd-0403103-161754', 'etd-09142005-155619', 'etd-01252010-135613', 'etd-07112007-120318', 'etd-04152005-002604', 'etd-08302011-155548', 'etd-02212007-092521', 'etd-07132006-094645', 'etd-01252006-184722', 'etd-06082016-105005', 'etd-01042007-194007', 'etd-07072005-121012', 'etd-06022004-160351', 'etd-03312004-102301', 'etd-01042012-135713', 'etd-01142015-151533', 'etd-01172013-144551', 'etd-10312012-190339', 'etd-07072009-225836', 'etd-0711102-063741', 'etd-07012014-111645', 'etd-11052014-135432', 'etd-11052011-182642', 'etd-11082014-050759', 'etd-07052011-104558', 'etd-11132008-002553', 'etd-06112014-090853', 'etd-07012015-115658', 'etd-06272011-205937', 'etd-11132014-112746', 'etd-04072016-174516', 'etd-01312012-141528', 'etd-04152013-125045', 'etd-11112009-112958', 'etd-12192003-015815', 'etd-06132013-152740', 'etd-07072008-204632', 'etd-02262008-155553', 'etd-07062006-174751', 'etd-11052004-153514', 'etd-10282011-150114', 'etd-01202009-162111', 'etd-11182013-081034', 'etd-06222016-064950', 'etd-11132006-214810', 'etd-06052007-105431', 'etd-01112015-152501', 'etd-06252013-110432', 'etd-01232013-183839', 'etd-0624102-210642', 'etd-07072015-194931', 'etd-07072011-094115', 'etd-03272015-164340', 'etd-0418102-094919', 'etd-11182013-155744', 'etd-0415102-203956', 'etd-06142012-165933', 'etd-06232016-190021', 'etd-0417102-144640', 'etd-0708102-134248', 'etd-04052016-155859', 'etd-07112005-140541', 'etd-11052012-163633', 'etd-07132005-180922', 'etd-0320103-135616', 'etd-09222010-123945', 'etd-04262012-162749', 'etd-11092008-225824', 'etd-06082010-101827', 'etd-01032008-192224', 'etd-11022011-130255', 'etd-04042006-162059', 'etd-05192016-165254', 'etd-12102014-130109', 'etd-04022013-101656', 'etd-0414102-234031', 'etd-07112015-185116', 'etd-05132009-142650', 'etd-08142014-175532', 'etd-0710103-021510', 'etd-07052007-000931', 'etd-04112016-104522', 'etd-01242013-080254', 'etd-07102016-231339', 'etd-01262010-125830', 'etd-05262016-103215', 'etd-06122013-083208', 'etd-08182014-125357', 'etd-11112009-190811', 'etd-07062015-190634', 'etd-05262016-115631', 'etd-02192013-091628', 'etd-04062016-020401', 'etd-03152012-105649', 'etd-11122012-211006', 'etd-11172010-172521', 'etd-11092015-145821', 'etd-1110103-114629', 'etd-04262011-191045', 'etd-11112008-163919', 'etd-06202013-154722', 'etd-01232012-152906', 'etd-01112005-150557', 'etd-03102007-072045', 'etd-01112012-130920', 'etd-04062016-083339', 'etd-04262012-113223', 'etd-11102005-151604', 'etd-01312013-125810', 'etd-08302006-100849', 'etd-10262005-194944', 'etd-07132015-132247', 'etd-11022006-210954', 'etd-07062011-140052', 'etd-0408103-113707', 'etd-04202010-092827', 'etd-07112016-163408', 'etd-11152007-195658', 'etd-07072010-232250', 'etd-11162015-132812', 'etd-07142014-135910', 'etd-04142010-184221', 'etd-09112014-145620', 'etd-04062014-182723', 'etd-11052009-182059', 'etd-09252006-145513', 'etd-08042007-123938', 'etd-08252010-132719', 'etd-11192010-103613', 'etd-11182013-135820', 'etd-06032010-140059', 'etd-08262009-133235', 'etd-05052008-102729', 'etd-05152013-185130', 'etd-1030102-095941', 'etd-06112009-075955', 'etd-06142005-094933', 'etd-11022009-154616', 'etd-11152010-085801', 'etd-0123102-090803', 'etd-05012016-201927', 'etd-01272014-110452', 'etd-03122015-185529', 'etd-10272014-111941', 'etd-11042010-083857', 'etd-04272012-032346', 'etd-04072016-085543', 'etd-07272013-132302', 'etd-07072014-161918', 'etd-05302013-200417', 'etd-04192011-205428', 'etd-06222015-133054', 'etd-04022009-113505', 'etd-06062010-192030', 'etd-04062005-213344', 'etd-09072004-144045', 'etd-07092009-200839', 'etd-03262010-101156', 'etd-04042016-183902', 'etd-08092012-125715', 'etd-10212006-172902', 'etd-01222009-134158', 'etd-06232006-075937', 'etd-05162016-133118', 'etd-04222010-022409', 'etd-0903103-104340', 'etd-04142005-224115', 'etd-11142014-202218', 'etd-06292010-161138', 'etd-05142013-152834', 'etd-11032008-095555', 'etd-11092012-092032', 'etd-04102013-163651', 'etd-11122012-215715', 'etd-03282014-170940', 'etd-11062012-115022', 'etd-05262016-223435', 'etd-01192007-120328', 'etd-08282008-180237', 'etd-0619103-214616', 'etd-11082006-151706', 'etd-04132014-072128', 'etd-06242010-111630', 'etd-0416102-152847', 'etd-11042008-150403', 'etd-04082016-125740', 'etd-04182010-091634', 'etd-04102008-225554', 'etd-06122008-025626', 'etd-06232011-172622', 'etd-06042014-132917', 'etd-03312014-143918', 'etd-03192009-145515', 'etd-11082015-143513', 'etd-11122004-113256', 'etd-07052011-203013', 'etd-04082013-092930', 'etd-04042013-125626', 'etd-07012014-201902', 'etd-06172013-102515', 'etd-07082009-102124', 'etd-06042014-162538', 'etd-11112012-181035', 'etd-12192014-151437', 'etd-04272010-174411', 'etd-0730103-102957', 'etd-06222007-095426', 'etd-11162006-180203', 'etd-02192015-171339', 'etd-11022009-214820', 'etd-01212009-182854', 'etd-06282011-112408', 'etd-06192012-162823', 'etd-04132009-113413', 'etd-03302012-084257', 'etd-0710102-184016', 'etd-03182014-135909', 'etd-04142014-153222', 'etd-10302015-093842', 'etd-04092013-131159', 'etd-06222004-162727', 'etd-11132015-133833', 'etd-08162011-132924', 'etd-01152010-145510', 'etd-0606103-090949', 'etd-03312015-092659', 'etd-08272014-211241', 'etd-04262012-131446', 'etd-0120103-194126', 'etd-03282005-123322', 'etd-04102013-125810', 'etd-04202010-153537', 'etd-01122010-163908', 'etd-11152005-134445', 'etd-06052012-162857', 'etd-07122012-135928', 'etd-01142005-095834', 'etd-01222008-160333', 'etd-01242007-165928', 'etd-04202012-133757', 'etd-07102007-212209', 'etd-11142007-111455', 'etd-03172014-191033', 'etd-08072011-084238', 'etd-01192015-102433', 'etd-07132005-091607', 'etd-07072007-174342', 'etd-03302007-143201', 'etd-04132007-093422', 'etd-04042006-140255', 'etd-04122004-111801', 'etd-03182014-190138', 'etd-04222012-163941', 'etd-10282011-105021', 'etd-0412102-135248', 'etd-06162004-080204', 'etd-03252013-110727', 'etd-07072010-160348', 'etd-06302014-170310', 'etd-08272012-123825', 'etd-11152013-135204', 'etd-05262008-174029', 'etd-07062009-174630', 'etd-07092009-140417', 'etd-01162014-151454', 'etd-1111102-144455', 'etd-11122013-175429', 'etd-07122007-133132', 'etd-04012007-160408', 'etd-11182013-125244', 'etd-04212010-194837', 'etd-1110103-154159', 'etd-04152004-101850', 'etd-04032007-153033', 'etd-07112016-115002', 'etd-01192005-120441', 'etd-07022012-164017', 'etd-07092014-175611', 'etd-06212013-074018', 'etd-06042015-002704', 'etd-04082008-222926', 'etd-11152012-184558', 'etd-0515102-153007', 'etd-01232010-063910', 'etd-06142005-103358', 'etd-07082015-171255', 'etd-11092011-211336', 'etd-04052004-160403', 'etd-03302005-151536', 'etd-04062006-121321', 'etd-03122010-131541', 'etd-04042016-180549', 'etd-04072015-143832', 'etd-07052011-084113', 'etd-06262015-115616', 'etd-07082010-182920', 'etd-06062013-010835', 'etd-11072011-084037', 'etd-05202016-175422', 'etd-0409102-112238', 'etd-11162006-111635', 'etd-04072005-181452', 'etd-08292014-115237', 'etd-07122005-095626', 'etd-04282011-172052', 'etd-03242010-164958', 'etd-0804103-170835', 'etd-11142007-225619', 'etd-04292014-010649', 'etd-11142013-113029', 'etd-06222015-133755', 'etd-04132015-102412', 'etd-05152011-212114', 'etd-07032012-180127', 'etd-10082015-214831', 'etd-0417102-092007', 'etd-03082013-103044', 'etd-0902103-204309', 'etd-11172006-114309', 'etd-04092010-154422', 'etd-08302010-202253', 'etd-01212015-115545', 'etd-04152013-095825', 'etd-07012014-115252', 'etd-06042012-133038', 'etd-0405103-105145', 'etd-04112016-145117', 'etd-0215103-220227', 'etd-11012011-093748', 'etd-0412102-204633', 'etd-07092016-215445', 'etd-08192014-010829', 'etd-12122003-145432', 'etd-04072015-154406', 'etd-04192010-140026', 'etd-06302011-063215', 'etd-0709102-143522', 'etd-11062009-153202', 'etd-11142015-011810', 'etd-11192010-123241', 'etd-05152015-141837', 'etd-06182012-152357', 'etd-06292006-183004', 'etd-01272004-091930', 'etd-1113101-205213', 'etd-11182013-133340', 'etd-07112016-135402', 'etd-06282011-141122', 'etd-01102007-074140', 'etd-03262009-132122', 'etd-01092008-225444', 'etd-10272008-191756', 'etd-04012010-110403', 'etd-04072015-094935', 'etd-03292016-151830', 'etd-04092013-171749', 'etd-06242015-122231', 'etd-1106102-100713', 'etd-05242013-092017', 'etd-11052010-190841', 'etd-04162009-223853', 'etd-12152003-133403', 'etd-03262010-001926', 'etd-06262008-092657', 'etd-07062010-173559', 'etd-04142009-234949', 'etd-11052015-221358', 'etd-1009102-153228', 'etd-10252012-124844', 'etd-06052012-150153', 'etd-11102008-120516', 'etd-11052013-124819', 'etd-08262009-111906', 'etd-06052015-231947', 'etd-11152007-101053', 'etd-11162006-181325', 'etd-0711102-152933', 'etd-07072014-112722', 'etd-0407103-120733', 'etd-1110103-143019', 'etd-08122010-131153', 'etd-08222013-152010', 'etd-06072004-171156', 'etd-07052013-142005', 'etd-01182011-183838', 'etd-11112009-102005', 'etd-06292012-211739', 'etd-06202006-215324', 'etd-11182013-090349', 'etd-08212015-014524', 'etd-11052012-152622', 'etd-04032006-154756', 'etd-06252012-182352', 'etd-11162015-122937', 'etd-04012005-134822', 'etd-07022004-122652', 'etd-1106102-114007', 'etd-04132007-101339', 'etd-06302005-152718', 'etd-04112016-141324', 'etd-10242012-111022', 'etd-07102014-063109', 'etd-08042016-151320', 'etd-06222013-114147', 'etd-0919103-103154', 'etd-06162016-160705', 'etd-07082009-120148', 'etd-01222015-004745', 'etd-04252012-152015', 'etd-04022012-094627', 'etd-04132004-121421', 'etd-04122011-113559', 'etd-07082013-120440', 'etd-04132004-141920', 'etd-04142004-150517', 'etd-07102015-160750', 'etd-11102014-155304', 'etd-0802102-105201', 'etd-11042014-233743', 'etd-07062016-114144', 'etd-07132006-150446', 'etd-01062009-155450', 'etd-01222015-151256', 'etd-09172015-165318', 'etd-07032012-165733', 'etd-06272013-193138', 'etd-01042013-105013', 'etd-0418102-173116', 'etd-06272014-131806', 'etd-0123103-194128', 'etd-11082007-095808', 'etd-01212014-151823', 'etd-01202011-141648', 'etd-05152006-113602', 'etd-06052012-163734', 'etd-07102016-221149', 'etd-07032008-100238', 'etd-03022010-163627', 'etd-04052007-111635', 'etd-11062009-084659', 'etd-12092004-163021', 'etd-10262007-115116', 'etd-06062014-121941', 'etd-01252012-112627', 'etd-07082013-143714', 'etd-04132004-172352', 'etd-01282004-133655', 'etd-11062009-093816', 'etd-0407103-115838', 'etd-06232011-143040', 'etd-05142016-181632', 'etd-04032007-204806', 'etd-11052015-022023', 'etd-11182010-192232', 'etd-11112005-201715', 'etd-11082006-160629', 'etd-10252009-141849', 'etd-11092005-211258', 'etd-06232016-142614', 'etd-06172004-190624', 'etd-11032008-112312', 'etd-03252013-122934', 'etd-07022012-120906', 'etd-10302008-143944', 'etd-11152005-183139', 'etd-0730102-222405', 'etd-01262010-143236', 'etd-0417102-215906', 'etd-1115102-003214', 'etd-0401102-165849', 'etd-1029102-194713', 'etd-04072007-130455', 'etd-11082010-212600', 'etd-04052016-153947', 'etd-11082012-161610', 'etd-03062012-121049', 'etd-04052013-151705', 'etd-01242012-181059', 'etd-11172014-152711', 'etd-07062009-095040', 'etd-11162014-211729', 'etd-04062016-103735', 'etd-03302007-004530', 'etd-07082009-120329', 'etd-07122015-223237', 'etd-0709103-114011', 'etd-11102004-072203', 'etd-11062015-155755', 'etd-04062016-200439', 'etd-04082004-223228', 'etd-11152015-141853', 'etd-03022007-095025', 'etd-07082010-195417', 'etd-08242015-161647', 'etd-1114102-122738', 'etd-04142011-153625', 'etd-06102012-063907', 'etd-10122011-114146', 'etd-1111102-164704', 'etd-07062007-112158', 'etd-04132015-124829', 'etd-07072013-180657', 'etd-07072014-115319', 'etd-04162010-111826', 'etd-11122012-214403', 'etd-05312007-094533', 'etd-07082010-165000', 'etd-0328103-091826', 'etd-07072016-135632', 'etd-01182010-210857', 'etd-06012015-195140', 'etd-11182013-154811', 'etd-06032014-040245', 'etd-01262010-091740', 'etd-07082013-153208', 'etd-04062006-083846', 'etd-04072015-230133', 'etd-11162010-161733', 'etd-04022015-100145', 'etd-11142012-144543', 'etd-10022015-140942', 'etd-10222013-115111', 'etd-10242012-153450', 'etd-04082008-110410', 'etd-01132009-154823', 'etd-04142010-140421', 'etd-07072011-112114', 'etd-03162014-182010', 'etd-07142014-081438', 'etd-11152006-103338', 'etd-08212015-003958', 'etd-06102013-194938', 'etd-04022009-230601', 'etd-11062005-213043', 'etd-05252015-170813', 'etd-06082004-205150', 'etd-11122015-160845', 'etd-06032004-215359', 'etd-07012010-170704', 'etd-04102016-184850', 'etd-1106102-121027', 'etd-07132009-194835', 'etd-06252012-104955', 'etd-04132013-222105', 'etd-07272009-144116', 'etd-10212005-113231', 'etd-07082004-143318', 'etd-07062016-171359', 'etd-05152013-135256', 'etd-04292011-090820', 'etd-07072004-191224', 'etd-03312008-000156', 'etd-1023102-064719', 'etd-06272011-152252', 'etd-04052016-182023', 'etd-04052006-130421', 'etd-0409103-084327', 'etd-04202010-154715', 'etd-11122014-211555', 'etd-05052011-104952', 'etd-07062015-091745', 'etd-04012013-222737', 'etd-03262014-133039', 'etd-10222006-172424', 'etd-07012015-153644', 'etd-05132004-154411', 'etd-04082015-130239', 'etd-07122005-100946', 'etd-11102010-125026', 'etd-07132014-192309', 'etd-03162005-093310', 'etd-09192008-141454', 'etd-0708103-012004', 'etd-03232012-154906', 'etd-07082010-102921', 'etd-0417102-201343', 'etd-11092011-125736', 'etd-10272014-093343', 'etd-04052005-161052', 'etd-03152006-222905', 'etd-07082013-101700', 'etd-04082016-162117', 'etd-11102004-140641', 'etd-08142008-092027', 'etd-06192012-110252', 'etd-05132015-081026', 'etd-04092014-022214', 'etd-0418102-132925', 'etd-10272006-114027', 'etd-11132013-230025', 'etd-11132009-034120', 'etd-07072009-105706', 'etd-08302009-214009', 'etd-03172015-140354', 'etd-04042008-105327', 'etd-07082007-222724', 'etd-05202011-221600', 'etd-06042012-123258', 'etd-07032013-200644', 'etd-10262004-151149', 'etd-11052010-115744', 'etd-01202012-124540', 'etd-12192008-154304', 'etd-04092007-161907', 'etd-0402103-154425', 'etd-09012004-081308', 'etd-04122007-004138', 'etd-0522103-164451', 'etd-03072008-191609', 'etd-11112013-150154', 'etd-04242012-192522', 'etd-10232007-121548', 'etd-11122008-144747', 'etd-10282013-172517', 'etd-04072014-160705', 'etd-11112008-202234', 'etd-11152012-170221', 'etd-06062014-132202', 'etd-11042005-111923', 'etd-11082013-133239', 'etd-03012006-172433', 'etd-03202006-080052', 'etd-04192012-162611', 'etd-04122004-171939', 'etd-03272013-195756', 'etd-06302014-150004', 'etd-11042014-210128', 'etd-04072014-120512', 'etd-04262012-000830', 'etd-04062006-085455', 'etd-07062016-163453', 'etd-04072009-164241', 'etd-04152005-014414', 'etd-11142007-115425', 'etd-04072015-170409', 'etd-06282011-175735', 'etd-11182010-143923', 'etd-06032004-191432', 'etd-04192010-085808', 'etd-1113102-161933', 'etd-10152015-092252', 'etd-07082013-140407', 'etd-03012004-180616', 'etd-11122012-124413', 'etd-04082014-135837', 'etd-07052010-160819', 'etd-05192004-122138', 'etd-10192015-140617', 'etd-08242007-130451', 'etd-04122005-162104', 'etd-04152013-112915', 'etd-11112010-164646', 'etd-11182013-142211', 'etd-04122005-214053', 'etd-10272006-144739', 'etd-11142007-134235', 'etd-05312012-230503', 'etd-10142004-104955', 'etd-0801103-191645', 'etd-04072015-141212', 'etd-01212009-101352', 'etd-10292013-141110', 'etd-03022004-015240', 'etd-04012013-151324', 'etd-11072012-134904', 'etd-07032015-223738', 'etd-03312006-105616', 'etd-11112004-171734', 'etd-09052013-143838', 'etd-11102005-121031', 'etd-04042016-220803', 'etd-11152005-001938', 'etd-07102007-185034', 'etd-07072015-143355', 'etd-07022010-180232', 'etd-08112010-134702', 'etd-06292011-161018', 'etd-03232010-164719', 'etd-0925103-125746', 'etd-04022008-150837', 'etd-11022006-192135', 'etd-06042014-184625', 'etd-08272014-150534', 'etd-11112004-111407', 'etd-10302009-131200', 'etd-06142005-124552', 'etd-01232012-151056', 'etd-04192011-112901', 'etd-05292006-100715', 'etd-06272013-153152', 'etd-12132009-104629', 'etd-07082013-100829', 'etd-04092009-190232', 'etd-11102011-101118', 'etd-04262012-102541', 'etd-04252014-133514', 'etd-07042011-173856', 'etd-0324103-105157', 'etd-04032016-213616', 'etd-07092012-161720', 'etd-05282008-145627', 'etd-07252006-150056', 'etd-02132008-102450', 'etd-05202014-135802', 'etd-05292009-214931', 'etd-07132005-160351', 'etd-04112005-120514', 'etd-04202010-235906', 'etd-11142007-164339', 'etd-08102006-154037', 'etd-11082011-145602', 'etd-04052007-102159', 'etd-1205101-103814', 'etd-06082011-093407', 'etd-1112102-180551', 'etd-07042012-075734', 'etd-04222012-091217', 'etd-11152006-165854', 'etd-10192009-210102', 'etd-11042011-110933', 'etd-07102015-153350', 'etd-04212010-203536', 'etd-03042008-215227', 'etd-05212005-102450', 'etd-06302014-160933', 'etd-02082013-102605', 'etd-06292015-150459', 'etd-06092014-153349', 'etd-07112016-153153', 'etd-07062015-211850', 'etd-04072014-160559', 'etd-07132005-161622', 'etd-04052006-150746', 'etd-0606103-074108', 'etd-11132006-142302', 'etd-07132006-124901', 'etd-11072011-143438', 'etd-11122010-074336', 'etd-07112007-142442', 'etd-0606102-074339', 'etd-04042014-165531', 'etd-04262011-104341', 'etd-05242014-124524', 'etd-04052006-105847', 'etd-04152010-163044', 'etd-05282015-073316', 'etd-0828103-132935', 'etd-01282004-033349', 'etd-0419102-085015', 'etd-1104103-090121', 'etd-02112014-160207', 'etd-07082011-120153', 'etd-10302015-171353', 'etd-04062015-184812', 'etd-04112007-131133', 'etd-12192003-234328', 'etd-07122006-200229', 'etd-11142008-071704', 'etd-09042012-140648', 'etd-05282013-060302', 'etd-1111102-204823', 'etd-07102015-144520', 'etd-04152013-015414', 'etd-10272005-155140', 'etd-04152009-110153', 'etd-1106102-143219', 'etd-10292012-130643', 'etd-07082009-142239', 'etd-0416102-142714', 'etd-0415102-111440', 'etd-06232015-110945', 'etd-10152015-111933', 'etd-06102010-120904', 'etd-07072010-154205', 'etd-04152004-101232', 'etd-04102013-230253', 'etd-10192011-162907', 'etd-07102007-154149', 'etd-10272010-122323', 'etd-11142014-135913', 'etd-01082014-124806', 'etd-0404102-210501', 'etd-0411102-103825', 'etd-04042016-085638', 'etd-11152013-105517', 'etd-0708103-104637', 'etd-07022006-191331', 'etd-07152005-085916', 'etd-01242007-130409', 'etd-11102011-213825', 'etd-04172009-084505', 'etd-01212005-212501', 'etd-1219102-224535', 'etd-04132009-142305', 'etd-05192014-220307', 'etd-08242015-195042', 'etd-09302015-073047', 'etd-1114102-171613', 'etd-04042016-082257', 'etd-07062012-101234', 'etd-05232016-105200', 'etd-0411103-112520', 'etd-07022014-162438', 'etd-04092007-102150', 'etd-06042015-160031', 'etd-11072011-135711', 'etd-10302012-133755', 'etd-08192011-121356', 'etd-1112103-071121', 'etd-11132013-093245', 'etd-05202011-111815', 'etd-07102016-204601', 'etd-04192010-093617', 'etd-04212012-133431', 'etd-04132005-165719', 'etd-0701103-164728', 'etd-06152005-103317', 'etd-07052004-120622', 'etd-07062014-095946', 'etd-04222012-152528', 'etd-07022004-211551', 'etd-04112014-140144', 'etd-11162005-111135', 'etd-0305103-101059', 'etd-0320103-151347', 'etd-1106103-111337', 'etd-07112007-163816', 'etd-11102012-171915', 'etd-01242012-152651', 'etd-08182013-213856', 'etd-04042006-174834', 'etd-04132015-085920', 'etd-1112102-140432', 'etd-04082005-140536', 'etd-0828103-180739', 'etd-06272011-141751', 'etd-04162010-155954', 'etd-04032005-204912', 'etd-04032007-213244', 'etd-01222007-144953', 'etd-09242007-161309', 'etd-01052006-114234', 'etd-07022010-132624', 'etd-07072009-091017', 'etd-04102015-144049', 'etd-06242010-163803', 'etd-04072009-131912', 'etd-11152012-111639', 'etd-06062012-112314', 'etd-11102015-094342', 'etd-04022014-210318', 'etd-04252011-165305', 'etd-04102016-111636', 'etd-08132013-195641', 'etd-06282013-125347', 'etd-0415102-213609', 'etd-04132013-131628', 'etd-11142005-125954', 'etd-04032006-171933', 'etd-04082015-122726', 'etd-04102013-165948', 'etd-08212007-141121', 'etd-11172013-220126', 'etd-01192004-141706', 'etd-04092014-110652', 'etd-04102012-140116', 'etd-11182005-104726', 'etd-08182011-103436', 'etd-04162009-223459', 'etd-07082009-124916', 'etd-04262011-110012', 'etd-01192015-153811', 'etd-11112015-143157', 'etd-04062015-003846', 'etd-11072008-111456', 'etd-06082006-095451', 'etd-11112009-122814', 'etd-10082013-141119', 'etd-04092010-162834', 'etd-07062016-223833', 'etd-06102008-005833', 'etd-07102015-121146', 'etd-04012015-153442', 'etd-11142014-225047', 'etd-07042012-171543', 'etd-04092004-081738', 'etd-11022011-221513', 'etd-04172012-145151', 'etd-06182010-143907', 'etd-07132015-161603', 'etd-06072011-165413', 'etd-07142005-085806', 'etd-10312011-200227', 'etd-07072004-160643', 'etd-11082009-203548', 'etd-11112005-173308', 'etd-07082016-144631', 'etd-11102014-102344', 'etd-0327102-091522', 'etd-1114102-161158', 'etd-03122013-115539', 'etd-07032013-180842', 'etd-10212004-151726', 'etd-10252010-144927', 'etd-04252012-135009', 'etd-06112014-141645', 'etd-01212014-160117', 'etd-02232005-134727', 'etd-07142005-151356', 'etd-11112009-173430', 'etd-04092010-125258', 'etd-11162010-135231', 'etd-06292005-171700', 'etd-0702103-151627', 'etd-04072014-230907', 'etd-0416102-191921', 'etd-03312004-124858', 'etd-06112014-145332', 'etd-07052010-071455', 'etd-07102015-090513', 'etd-01262010-121648', 'etd-06272011-115911', 'etd-11162005-160340', 'etd-04182012-190202', 'etd-0402102-124842', 'etd-11182010-160247', 'etd-06092005-221512', 'etd-11152010-220038', 'etd-11102006-112931', 'etd-01192015-175348', 'etd-11142013-173050', 'etd-1111101-194018', 'etd-04142005-152845', 'etd-07042016-100450', 'etd-07082013-155401', 'etd-11072011-162326', 'etd-11142014-233558', 'etd-05292014-142239', 'etd-0111103-194328', 'etd-07102008-150456', 'etd-07082008-111403', 'etd-03182016-104532', 'etd-04102015-111028', 'etd-11102015-131948', 'etd-05262006-013154', 'etd-07112006-123524', 'etd-07022013-153313', 'etd-03302007-123745', 'etd-12082010-111132', 'etd-04032007-214950', 'etd-07112016-141521', 'etd-0321103-171804', 'etd-07062007-121327', 'etd-05302008-162322', 'etd-10262005-135712', 'etd-06282011-130632', 'etd-1112103-151158', 'etd-11142007-140547', 'etd-02112004-132337', 'etd-01272014-114926', 'etd-04152013-154242', 'etd-04092013-172440', 'etd-11072007-103105', 'etd-01222013-142342', 'etd-07122015-143704', 'etd-07082013-105051', 'etd-04042016-175322', 'etd-07082015-135920', 'etd-07052013-123849', 'etd-03312016-103717', 'etd-0528102-130208', 'etd-06122005-122824', 'etd-01092005-181433', 'etd-06122006-133233', 'etd-04042013-150012', 'etd-11122008-171510', 'etd-07122015-125729', 'etd-06272011-125731', 'etd-04052006-095507', 'etd-07072016-213723', 'etd-1204101-184456', 'etd-11182005-035932', 'etd-11112009-200229', 'etd-04042005-154619', 'etd-07312006-124855', 'etd-05292007-123917', 'etd-05162007-124454', 'etd-11172010-164456', 'etd-11052014-143018', 'etd-08262015-152739', 'etd-11142006-205417', 'etd-04152013-154902', 'etd-0708103-100542', 'etd-04102014-161432', 'etd-11052015-152122', 'etd-04162016-115324', 'etd-03282007-002447', 'etd-02112008-102607', 'etd-04102008-110648', 'etd-08272012-232909', 'etd-0407103-143704', 'etd-11192015-104941', 'etd-11092004-005103', 'etd-1111103-120146', 'etd-07072010-115415', 'etd-01152004-152014', 'etd-09052006-132716', 'etd-05292009-195605', 'etd-07132015-131705', 'etd-03312015-144718', 'etd-07072009-123333', 'etd-11152006-125034', 'etd-0414102-023909', 'etd-06152016-140854', 'etd-04072016-183346', 'etd-04142005-152625', 'etd-10132013-190025', 'etd-11052014-045926', 'etd-11152010-105813', 'etd-0410103-185433', 'etd-09042007-163344', 'etd-07022008-143903', 'etd-11172013-161129', 'etd-11122012-170655', 'etd-11092005-102051', 'etd-05192014-144506', 'etd-03262010-174331', 'etd-04132015-130352', 'etd-11042009-102505', 'etd-04242012-133904', 'etd-07102015-163850', 'etd-04132004-192931', 'etd-11102015-112732', 'etd-08122004-091254', 'etd-09172004-141159', 'etd-06052011-170855', 'etd-07082013-161635', 'etd-08272009-000441', 'etd-06292014-132224', 'etd-11142005-182327', 'etd-06062016-165730', 'etd-04122013-131617', 'etd-06062012-150754', 'etd-04122007-120024', 'etd-11102015-213419', 'etd-11092004-194516', 'etd-06052012-123115', 'etd-07052012-200216', 'etd-04022012-102047', 'etd-04202011-103203', 'etd-04062005-152112', 'etd-11092006-114843', 'etd-10132011-152121', 'etd-04132015-160154', 'etd-1111103-095030', 'etd-10242005-143348', 'etd-04052015-173634', 'etd-07102007-162132', 'etd-09052008-114605', 'etd-04102008-084519', 'etd-04022015-100028', 'etd-0408102-073903', 'etd-04022009-111826', 'etd-07082007-154325', 'etd-04082016-121720', 'etd-04022006-211850', 'etd-01152014-114456', 'etd-0326103-212409', 'etd-08152008-093238', 'etd-0929103-120652', 'etd-06292013-162140', 'etd-11092011-164128', 'etd-01262012-114620', 'etd-08272009-102321', 'etd-0418102-131049', 'etd-03202012-214441', 'etd-08272012-145008', 'etd-07072008-182001', 'etd-0618103-160439', 'etd-06192014-153259', 'etd-11142006-115235', 'etd-09222015-211325', 'etd-11172005-162210', 'etd-07082010-162705', 'etd-04042014-211939', 'etd-11082006-085807', 'etd-04252012-124710', 'etd-08102015-151333', 'etd-11102009-101918', 'etd-05142013-103916', 'etd-11092006-172750', 'etd-08312004-165648', 'etd-05262013-124909', 'etd-04012004-155002', 'etd-10272014-143801', 'etd-06142016-213427', 'etd-05202011-111953', 'etd-11122009-112612', 'etd-06122007-114023', 'etd-06302016-110350', 'etd-04082009-192205', 'etd-06262012-110651', 'etd-04132005-135311', 'etd-06132007-113852', 'etd-11102008-130405', 'etd-08192014-100028', 'etd-11132006-203732', 'etd-01202006-231121', 'etd-07012014-114811', 'etd-04282011-163544', 'etd-11182013-112412', 'etd-01062005-142505', 'etd-04012015-212059', 'etd-07082010-142254', 'etd-04082004-110028', 'etd-06272007-215655', 'etd-07072005-121013', 'etd-05262006-081829', 'etd-11252009-165756', 'etd-11052015-073757', 'etd-07082013-003804', 'etd-07142014-141643', 'etd-04132009-143801', 'etd-04032007-084834', 'etd-07202006-193727', 'etd-04152013-153227', 'etd-05202013-101814', 'etd-01282004-122859', 'etd-07122006-222945', 'etd-08302013-155812', 'etd-04112013-140531', 'etd-04262012-121452', 'etd-0124102-122713', 'etd-11092010-101700', 'etd-09082011-024316', 'etd-11132007-113712', 'etd-0124102-083259', 'etd-07132006-174133', 'etd-05132013-172359', 'etd-06062009-162953', 'etd-10052015-082040', 'etd-04162004-124831', 'etd-11102007-154707', 'etd-10252004-163127', 'etd-06092009-151048', 'etd-04262011-153646', 'etd-04132015-095811', 'etd-11112014-192530', 'etd-04192011-150751', 'etd-03182014-112746', 'etd-07022007-155743', 'etd-04152013-121245', 'etd-11162005-101240', 'etd-04142009-102827', 'etd-04132007-094737', 'etd-01252012-144700', 'etd-04102012-152001', 'etd-0404102-114211', 'etd-04142010-005228', 'etd-07012014-224110', 'etd-04052012-145650', 'etd-03302006-145845', 'etd-06262012-001352', 'etd-04212005-133917', 'etd-12232015-081725', 'etd-06062013-210833', 'etd-06302009-221431', 'etd-09012009-140708', 'etd-06072014-174625', 'etd-0820103-110029', 'etd-12012009-140956', 'etd-04132009-090803', 'etd-09052006-090942', 'etd-07012013-120130', 'etd-1107103-233833', 'etd-1112103-140654', 'etd-02152016-171906', 'etd-03312006-093527', 'etd-10232012-150847', 'etd-10312011-155643', 'etd-09042007-150437', 'etd-07082013-020113', 'etd-07062005-145608', 'etd-07072008-091901', 'etd-04222010-135546', 'etd-04112008-095727', 'etd-0610102-162914', 'etd-03172004-203127', 'etd-04272011-134600', 'etd-04092008-140338', 'etd-0709102-190444', 'etd-11162005-115821', 'etd-0918102-205920', 'etd-07102006-103349', 'etd-04262011-153424', 'etd-04212012-141327', 'etd-08212013-205604', 'etd-04092013-164331', 'etd-06212013-171235', 'etd-06112015-154139', 'etd-07062012-120939', 'etd-04052006-185750', 'etd-04142012-062820', 'etd-04102009-163053', 'etd-11182010-123053', 'etd-03262014-152030', 'etd-0710102-225727', 'etd-06052009-153559', 'etd-0615102-024628', 'etd-04072016-091044', 'etd-04152013-093456', 'etd-06272014-134332', 'etd-05242014-094934', 'etd-06192009-142044', 'etd-07022015-120634', 'etd-04062013-172204', 'etd-04272011-161136', 'etd-09042006-111005', 'etd-04162009-230234', 'etd-08082016-164729', 'etd-04182012-145423', 'etd-11172005-155650', 'etd-08302011-140650', 'etd-02032010-123436', 'etd-1113103-080105', 'etd-04052007-112941', 'etd-07112012-150956', 'etd-04062005-121156', 'etd-05222004-002142', 'etd-06092009-115723', 'etd-0707103-124904', 'etd-0415102-154118', 'etd-05242016-134040', 'etd-04122005-110820', 'etd-06072011-220250', 'etd-03152013-122943', 'etd-11102009-171301', 'etd-11162007-115010', 'etd-04112014-114109', 'etd-06242004-144906', 'etd-06262015-172631', 'etd-06072009-213520', 'etd-07062016-152603', 'etd-06062009-092103', 'etd-01252006-134813', 'etd-11202013-215353', 'etd-0410103-190112', 'etd-07102009-112220', 'etd-11122007-231956', 'etd-06032008-192314', 'etd-0118103-203326', 'etd-02172004-140925', 'etd-11152006-145927', 'etd-08292008-085519', 'etd-0130103-112353', 'etd-11072008-084923', 'etd-11182013-162101', 'etd-06032007-180334', 'etd-06192009-232613', 'etd-04262011-005356', 'etd-0701102-160130', 'etd-04112005-142641', 'etd-0411102-044045', 'etd-11142005-104912', 'etd-06122006-184339', 'etd-11152006-205531', 'etd-04142013-223923', 'etd-04082016-121543', 'etd-04142014-084235', 'etd-07082013-161828', 'etd-03262014-083938', 'etd-06072011-114020', 'etd-11102010-203038', 'etd-07132005-115140', 'etd-04012008-111116', 'etd-0515103-184631', 'etd-11052009-170151', 'etd-08252013-225404', 'etd-07062015-135757', 'etd-0124103-165322', 'etd-03132015-125330', 'etd-03072007-091607', 'etd-11042014-175430', 'etd-01052013-152253', 'etd-04112014-073250', 'etd-10282011-162414', 'etd-08272013-232853', 'etd-07112014-160007', 'etd-06152005-131704', 'etd-0828103-114026', 'etd-04192012-173641', 'etd-07072008-234509', 'etd-1028102-183920', 'etd-06112007-225430', 'etd-05262005-161704', 'etd-06112007-111434', 'etd-07082010-181752', 'etd-1107103-092611', 'etd-0124103-142051', 'etd-06102013-151903', 'etd-06112009-104535', 'etd-07122006-163348', 'etd-11182010-200202', 'etd-1110103-133154', 'etd-08022012-145258', 'etd-09302008-084220', 'etd-0806103-213613', 'etd-11082012-150920', 'etd-07032016-171836', 'etd-03262007-195825', 'etd-01092007-213954', 'etd-11052010-151739', 'etd-09252014-193135', 'etd-07082013-134001', 'etd-06232005-133134', 'etd-11022004-163743', 'etd-01142014-173213', 'etd-07102008-110841', 'etd-11072006-091228', 'etd-04152004-095658', 'etd-0409103-102328', 'etd-11112013-154655', 'etd-04052013-135903', 'etd-0710103-144206', 'etd-01152013-144231', 'etd-06252015-162416', 'etd-12222004-163817', 'etd-11032010-084217', 'etd-05122008-115728', 'etd-1112102-151040', 'etd-04192011-135539', 'etd-07092009-032536', 'etd-0324103-145617', 'etd-04152005-112438', 'etd-10142011-044352', 'etd-10282011-103822', 'etd-04092015-224028', 'etd-04112013-135003', 'etd-11092015-151412', 'etd-04192011-164628', 'etd-03242016-195015', 'etd-11082012-112643', 'etd-06272014-122519', 'etd-04122005-145005', 'etd-04072010-175854', 'etd-11122009-085042', 'etd-0616103-114424', 'etd-11142012-132112', 'etd-04252011-140332', 'etd-1113103-123910', 'etd-02242014-212940', 'etd-06252012-143806', 'etd-04112016-113325', 'etd-11132013-171155', 'etd-02222008-085519', 'etd-08142014-145549', 'etd-05212013-123831', 'etd-04242012-014100', 'etd-11072006-134214', 'etd-11072011-123501', 'etd-11142006-015043', 'etd-08142008-150737', 'etd-08142006-103536', 'etd-01162013-060632', 'etd-04072016-144840', 'etd-04112015-125740', 'etd-07132006-153626', 'etd-10212015-204941', 'etd-07132006-091708', 'etd-07032009-143701', 'etd-08102014-085202', 'etd-09042013-193017', 'etd-04202011-091322', 'etd-04012008-084345', 'etd-09102009-223610', 'etd-0129103-124729', 'etd-10222004-111952', 'etd-11112004-113835', 'etd-04252012-160606', 'etd-11122012-085034', 'etd-07012004-160623', 'etd-1218102-163218', 'etd-0702103-155417', 'etd-11052009-141101', 'etd-01062015-165417', 'etd-01232007-000319', 'etd-11032011-213129', 'etd-07102012-131039', 'etd-01082010-110943', 'etd-11122008-143700', 'etd-02152007-114707', 'etd-04062016-162646', 'etd-03122013-113127', 'etd-06092015-101851', 'etd-03202008-135616', 'etd-05192016-172949', 'etd-07112014-182603', 'etd-12192014-155945', 'etd-07072006-090411', 'etd-09032013-151841', 'etd-01242005-160814', 'etd-04112016-151259', 'etd-10092009-211126', 'etd-11102006-151738', 'etd-07102013-180431', 'etd-11172014-134851', 'etd-0731103-213545', 'etd-04012014-110556', 'etd-10262005-155109', 'etd-01062015-094959', 'etd-11062008-175603', 'etd-06222015-122919', 'etd-05172010-163122', 'etd-1113103-134239', 'etd-04122013-145920', 'etd-07072016-165158', 'etd-1114102-182657', 'etd-06132007-063452', 'etd-06022006-184225', 'etd-04282006-135424', 'etd-07012011-113432', 'etd-11112010-204900', 'etd-07102009-121806', 'etd-0418102-225829', 'etd-01152014-220519', 'etd-07122006-161342', 'etd-0801102-165734', 'etd-07052005-221018', 'etd-07082013-151028', 'etd-11102006-165330', 'etd-08122009-082312', 'etd-0127103-211254', 'etd-04132005-091358', 'etd-0710102-230709', 'etd-07062009-094329', 'etd-0707103-152855', 'etd-04082014-154630', 'etd-04092015-230646', 'etd-11102011-171915', 'etd-07022004-214740', 'etd-0328102-102651', 'etd-05062009-105351', 'etd-11122015-232818', 'etd-02212008-155401', 'etd-03252004-204704', 'etd-11122006-232902', 'etd-08252010-154811', 'etd-04032012-091137', 'etd-05262005-115411', 'etd-01062004-164031', 'etd-07202011-173312', 'etd-11152007-181520', 'etd-11112011-113402', 'etd-06232015-140222', 'etd-04142011-210902', 'etd-01162012-122647', 'etd-04042004-150921', 'etd-03082013-140923', 'etd-11182010-163349', 'etd-07122005-145903', 'etd-06032014-164025', 'etd-11042009-151333', 'etd-1119103-164312', 'etd-03022009-084850', 'etd-1011102-170733', 'etd-10262006-155514', 'etd-0411102-154714', 'etd-06042010-232315', 'etd-11172014-140601', 'etd-0401102-130106', 'etd-06232007-125012', 'etd-01172013-154859', 'etd-11162005-111803', 'etd-03182005-105307', 'etd-11082011-165455', 'etd-04102014-200010', 'etd-07102013-005839', 'etd-04122005-063949', 'etd-1112103-102606', 'etd-11162005-150755', 'etd-07082013-001815', 'etd-07112005-163713', 'etd-11132014-230809', 'etd-04062011-105550', 'etd-03242004-154627', 'etd-11042011-130137', 'etd-04202011-140606', 'etd-0708103-163628', 'etd-05162011-112744', 'etd-03312009-003522', 'etd-03312005-144110', 'etd-05282010-131630', 'etd-1028102-215542', 'etd-05312012-221734', 'etd-09102012-193745', 'etd-04092015-071914', 'etd-10292015-144756', 'etd-07122012-095333', 'etd-11142008-121452', 'etd-06232015-171518', 'etd-05202011-162625', 'etd-11162015-102349', 'etd-04062006-140823', 'etd-03032008-085730', 'etd-04102015-105324', 'etd-07082015-141705', 'etd-06092015-123604', 'etd-08222011-110825', 'etd-10142011-115001', 'etd-01042010-165217', 'etd-06222015-163920', 'etd-1116101-083403', 'etd-03312015-204624', 'etd-11022008-231524', 'etd-03282006-161207', 'etd-04092013-134044', 'etd-1107102-220029', 'etd-02012016-174202', 'etd-01052005-021803', 'etd-11182010-135451', 'etd-11132013-132906', 'etd-04122013-144144', 'etd-04102015-162654', 'etd-11162013-145137', 'etd-09252013-112041', 'etd-10152012-133904', 'etd-10312015-023151', 'etd-07122005-112749', 'etd-05162011-094742', 'etd-07012016-104111', 'etd-07062009-150829', 'etd-07072014-114838', 'etd-03082014-105351', 'etd-06232005-162008', 'etd-04282011-153507', 'etd-07102008-154544', 'etd-07052014-001609', 'etd-04182012-204238', 'etd-04132007-115442', 'etd-07022009-081214', 'etd-10272015-124138', 'etd-04012011-161536', 'etd-1106102-195257', 'etd-07092009-141954', 'etd-12012009-155424', 'etd-05272014-113602', 'etd-05282009-075025', 'etd-07082013-142038', 'etd-04082011-222242', 'etd-01222013-212301', 'etd-0122102-140841', 'etd-05092007-145302', 'etd-0224103-101444', 'etd-07112016-114436', 'etd-11072011-122718', 'etd-04202011-093522', 'etd-0406103-154007', 'etd-11102007-121443', 'etd-06232010-102200', 'etd-02042009-144038', 'etd-1029103-101200', 'etd-04022009-110021', 'etd-03052008-184657', 'etd-06272016-122920', 'etd-1104102-191220', 'etd-12262015-111914', 'etd-10292007-182007', 'etd-04102010-171133', 'etd-0415102-123400', 'etd-04092014-150801', 'etd-01102006-141140', 'etd-0610103-103014', 'etd-06272013-123831', 'etd-1113103-230518', 'etd-0111103-123017', 'etd-06242010-205201', 'etd-03312005-162802', 'etd-11162015-113909', 'etd-07012005-130406', 'etd-06282013-164734', 'etd-04262012-173005', 'etd-04072004-165133', 'etd-04072014-084244', 'etd-11132007-102323', 'etd-05252010-142640', 'etd-06062008-150443', 'etd-09032009-144125', 'etd-11142013-071009', 'etd-12102013-142516', 'etd-0610102-080619', 'etd-11132006-190148', 'etd-12052007-130029', 'etd-10132011-155152', 'etd-03272014-211112', 'etd-10312009-123459', 'etd-08172014-190037', 'etd-04112013-035412', 'etd-03192015-101823', 'etd-10022009-140919', 'etd-01272010-094207', 'etd-11112010-153958', 'etd-03142005-161647', 'etd-07102007-134324', 'etd-04192011-165042', 'etd-05192016-141010', 'etd-0722103-113753', 'etd-11032015-151713', 'etd-08162008-123732', 'etd-05312012-123251', 'etd-01102012-203246', 'etd-07112011-153829', 'etd-11182010-115637', 'etd-07082016-135400', 'etd-12022004-133039', 'etd-09142011-165624', 'etd-06062012-153941', 'etd-06162016-142835', 'etd-06032015-181706', 'etd-06142016-151819', 'etd-10262011-122108', 'etd-11092011-204628', 'etd-11162015-150200', 'etd-0128103-151631', 'etd-06302009-154228', 'etd-11162010-161252', 'etd-04062004-092416', 'etd-03082008-082125', 'etd-11152005-101830', 'etd-04152011-201728', 'etd-10222013-221810', 'etd-04102009-123722', 'etd-07112016-140347', 'etd-08302004-213631', 'etd-04152005-094214', 'etd-0315102-090113', 'etd-11012006-151515', 'etd-06272013-205033', 'etd-10312013-120652', 'etd-11102008-120141', 'etd-06032014-161305', 'etd-04102012-135844', 'etd-04092008-110148', 'etd-07012016-163931', 'etd-06062008-092115', 'etd-0403103-111134', 'etd-12222014-133735', 'etd-07082009-182740', 'etd-06112013-084932', 'etd-10062010-132750', 'etd-04122004-234159', 'etd-04042016-205851', 'etd-11042011-105812', 'etd-1027102-193854', 'etd-0415102-110441', 'etd-06082004-163256', 'etd-11042015-092016', 'etd-07102007-180009', 'etd-04012016-122832', 'etd-04072014-001557', 'etd-01212015-181639', 'etd-0220102-223805', 'etd-1114102-170228', 'etd-06032008-133935', 'etd-0418102-111522', 'etd-04132005-153531', 'etd-04152013-092715', 'etd-11152012-112107', 'etd-0127103-183211', 'etd-10292014-151704', 'etd-04102007-163136', 'etd-04172009-033329', 'etd-05232011-124955', 'etd-07112012-123730', 'etd-07122007-190055', 'etd-06262013-095701', 'etd-0401103-155847', 'etd-04052005-222449', 'etd-07122012-205907', 'etd-07092009-230134', 'etd-0129103-074005', 'etd-06232007-221319', 'etd-11112010-185919', 'etd-07132015-112412', 'etd-02132004-164149', 'etd-01052015-161402', 'etd-10282011-155405', 'etd-05162005-154916', 'etd-11112014-190646', 'etd-08032011-162858', 'etd-06172004-091346', 'etd-11142013-162423', 'etd-05162007-090802', 'etd-11172014-085718', 'etd-03272015-172840', 'etd-10262009-172626', 'etd-04072004-102821', 'etd-04012006-004043', 'etd-08172006-221816', 'etd-04122013-194031', 'etd-11062014-193448', 'etd-01262015-115758', 'etd-04262012-143227', 'etd-11082013-164218', 'etd-03312015-095948', 'etd-11052012-222144', 'etd-1114102-152724', 'etd-06182010-090736', 'etd-04262012-103019', 'etd-02082005-130556', 'etd-0417102-054620', 'etd-04102015-141826', 'etd-03242014-125642', 'etd-11112004-133415', 'etd-0702103-154150', 'etd-04092009-083639', 'etd-01052016-164703', 'etd-0629102-132738', 'etd-04032016-160153', 'etd-0403103-184303', 'etd-11072015-113759', 'etd-01232006-162915', 'etd-07022012-155752', 'etd-11042009-161313', 'etd-0222103-230811', 'etd-03182016-142522', 'etd-11072014-140029', 'etd-11022015-134700', 'etd-04042006-152518', 'etd-11042013-142929', 'etd-05192013-203652', 'etd-08062016-005626', 'etd-04162010-152205', 'etd-08302011-121158', 'etd-0325103-162304', 'etd-1112102-113818', 'etd-11092007-160127', 'etd-11162006-110802', 'etd-06122007-174354', 'etd-07112012-195256', 'etd-08222006-195437', 'etd-05242016-154303', 'etd-04102013-145402', 'etd-11102015-134219', 'etd-06182009-120305', 'etd-01102016-181109', 'etd-0407103-142724', 'etd-05272010-113932', 'etd-12122003-144340', 'etd-04252012-002535', 'etd-07122005-120721', 'etd-11122010-175148', 'etd-0330103-204041', 'etd-0417102-074443', 'etd-04022009-210023', 'etd-06272007-084024', 'etd-01222013-100539', 'etd-03112005-165301', 'etd-11092004-132734', 'etd-0703103-140535', 'etd-04142009-191748', 'etd-12312014-115952', 'etd-11122007-131810', 'etd-04152009-145757', 'etd-0607102-080200', 'etd-07072016-223744', 'etd-11032011-153514', 'etd-04202010-165308', 'etd-0903103-125920', 'etd-08142008-163923', 'etd-0131102-135337', 'etd-01072004-084210', 'etd-08262015-141117', 'etd-07082008-125904', 'etd-07022015-155819', 'etd-04122011-111053', 'etd-05122008-133612', 'etd-12162004-123704', 'etd-04042016-175544', 'etd-07062015-194707', 'etd-07012004-121750', 'etd-01202012-195627', 'etd-04242012-061201', 'etd-03272015-204552', 'etd-1113102-100011', 'etd-03182009-110204', 'etd-07102015-165738', 'etd-06122008-125449', 'etd-04112013-203731', 'etd-07132005-171713', 'etd-0128103-124914', 'etd-07062014-093350', 'etd-0711102-184930', 'etd-06072010-090000', 'etd-11142006-115517', 'etd-10052006-091215', 'etd-0713101-132954', 'etd-11182005-091336', 'etd-07112012-132740', 'etd-09092008-214045', 'etd-0605103-202818', 'etd-0130102-181818', 'etd-01282009-144632', 'etd-04042014-153331', 'etd-04212011-153427', 'etd-01202012-123829', 'etd-07112005-130522', 'etd-03142006-134928', 'etd-10292005-141121', 'etd-03292012-162739', 'etd-06022006-112548', 'etd-07082016-081805', 'etd-11012004-131832', 'etd-01152015-124206', 'etd-10312009-095845', 'etd-06232016-094037', 'etd-05122013-103734', 'etd-04152009-151049', 'etd-0707103-102616', 'etd-06282012-151528', 'etd-06182013-122929', 'etd-03302014-165823', 'etd-11112008-123118', 'etd-04032016-134916', 'etd-12212015-103617', 'etd-04112005-145019', 'etd-04032015-080642', 'etd-06152012-100912', 'etd-06082010-092441', 'etd-04192011-170837', 'etd-04112011-154845', 'etd-04042006-143225', 'etd-03022010-163447', 'etd-0709102-111850', 'etd-09162011-151909', 'etd-04012004-142458', 'etd-11012011-144816', 'etd-05112005-113453', 'etd-11152007-194712', 'etd-03262009-185106', 'etd-07082016-143208', 'etd-10302006-082326', 'etd-06092004-160703', 'etd-11072014-161553', 'etd-07102008-120225', 'etd-06102015-162057', 'etd-03312008-163317', 'etd-08212014-170759', 'etd-01212009-071834', 'etd-10012007-155906', 'etd-06072007-094204', 'etd-07142005-085556', 'etd-07022010-154501', 'etd-06302010-155903', 'etd-05162007-113114', 'etd-01242011-153809', 'etd-10262007-110333', 'etd-01272014-160928', 'etd-04242012-110026', 'etd-10312011-123708', 'etd-11132014-153659', 'etd-11072012-133030', 'etd-01162009-144927', 'etd-07062006-134722', 'etd-04042013-163026', 'etd-0711102-205436', 'etd-06122013-011932', 'etd-04102015-130447', 'etd-1114101-090416', 'etd-11072011-223443', 'etd-07092012-105415', 'etd-04112016-120738', 'etd-11052009-094915', 'etd-04202010-131409', 'etd-11052015-121656', 'etd-0725102-114518', 'etd-06242007-210547', 'etd-01302014-191109', 'etd-11062011-131536', 'etd-11022008-175503', 'etd-07082015-182940', 'etd-07082008-170558', 'etd-04102009-095328', 'etd-10202005-154721', 'etd-07042011-201553', 'etd-05132009-160203', 'etd-11152007-095442', 'etd-04082004-120041', 'etd-0820102-132849', 'etd-07052011-095522', 'etd-11182010-173826', 'etd-01052010-161005', 'etd-0831103-223008', 'etd-07022015-212919', 'etd-11162005-111427', 'etd-04132004-175832', 'etd-06092004-172114', 'etd-04142009-195924', 'etd-09272009-065154', 'etd-06262011-105200', 'etd-04082009-180114', 'etd-04102014-142821', 'etd-04022008-155152', 'etd-11172010-124431', 'etd-10182007-121416', 'etd-04242012-191702', 'etd-1218101-120133', 'etd-0404102-183420', 'etd-04042006-144949', 'etd-04152013-142658', 'etd-11182013-131717', 'etd-07112012-151711', 'etd-04082016-093328', 'etd-01272010-205801', 'etd-08162011-124732', 'etd-04042016-203043', 'etd-08282007-114820', 'etd-08082011-123216', 'etd-06112014-173952', 'etd-06242008-205101', 'etd-06272008-090151', 'etd-04272011-170624', 'etd-07042014-154201', 'etd-06082005-145209', 'etd-07062011-124124', 'etd-09162011-163314', 'etd-07112005-105630', 'etd-03272008-223743', 'etd-08092016-145719', 'etd-1112102-165610', 'etd-11062006-143723', 'etd-11162006-121033', 'etd-01262010-214805', 'etd-08222008-151955', 'etd-11012007-210852', 'etd-03172016-112700', 'etd-06182015-152532', 'etd-07102012-010652', 'etd-11062015-094958', 'etd-06222013-121741', 'etd-0710103-135135', 'etd-11102014-095548', 'etd-06162016-102134', 'etd-04082016-160346', 'etd-07022014-132348', 'etd-04112013-121316', 'etd-08112005-112930', 'etd-06252013-154139', 'etd-12162004-173707', 'etd-11012008-022726', 'etd-07082013-110639', 'etd-05172011-122931', 'etd-06112007-175920', 'etd-11182013-102645', 'etd-04152004-155709', 'etd-11142013-204105', 'etd-06102004-143321', 'etd-10192005-144729', 'etd-0702103-161717', 'etd-04132005-131235', 'etd-07282005-174600', 'etd-04152013-143203', 'etd-03042013-205101', 'etd-07102009-111915', 'etd-08222007-113440', 'etd-07012013-211426', 'etd-0702102-172401', 'etd-01222006-164241', 'etd-0807102-231708', 'etd-04082004-134440', 'etd-04072014-140233', 'etd-06222015-204406', 'etd-04172012-105149', 'etd-03172006-101515', 'etd-07062016-130759', 'etd-10252010-215700', 'etd-0130103-070613', 'etd-01242011-152221', 'etd-04072005-104024', 'etd-0408103-003549', 'etd-1105102-162215', 'etd-0416102-214517', 'etd-11122004-093614', 'etd-07132006-173748', 'etd-01162009-095730', 'etd-06012014-223310', 'etd-06032015-000351', 'etd-04192010-182506', 'etd-0611102-100354', 'etd-11142006-124649', 'etd-06082009-104331', 'etd-05162007-100031', 'etd-11082006-150553', 'etd-07022008-105747', 'etd-09272010-162056', 'etd-10302007-090619', 'etd-04092007-161111', 'etd-04072011-213237', 'etd-04112013-144823', 'etd-07052013-150623', 'etd-04232012-132803', 'etd-01092009-122836', 'etd-10112006-120724', 'etd-07022014-110151', 'etd-03162015-140321', 'etd-07032012-110851', 'etd-04082010-094612', 'etd-04142005-211331', 'etd-07112016-152754', 'etd-05272006-205658', 'etd-11052015-095817', 'etd-08292008-073317', 'etd-1029102-165835', 'etd-09092008-174302', 'etd-04072008-130926', 'etd-04102008-154852', 'etd-04112007-183710', 'etd-04162012-152246', 'etd-01092008-113450', 'etd-04122010-150911', 'etd-04042007-221415', 'etd-07042016-113601', 'etd-04102007-153445', 'etd-07062016-220249', 'etd-07062013-211518', 'etd-08102016-001026', 'etd-04022007-102803', 'etd-11132012-111019', 'etd-01242012-200750', 'etd-0409102-114305', 'etd-10302015-153637', 'etd-01192016-135349', 'etd-06072009-111849', 'etd-05172016-085953', 'etd-06212013-132011', 'etd-07232014-132831', 'etd-06092011-084046', 'etd-11142012-144831', 'etd-04252011-172401', 'etd-11132009-114826', 'etd-07102005-155325', 'etd-07012011-224930', 'etd-10192007-151040', 'etd-04102013-120946', 'etd-01292010-090247', 'etd-06012012-160049', 'etd-07092008-231611', 'etd-0708102-174635', 'etd-05262016-144035', 'etd-11082007-151528', 'etd-01242006-104756', 'etd-07092007-154138', 'etd-0621102-092939', 'etd-07052004-121943', 'etd-11072006-150416', 'etd-0403103-110129', 'etd-11022015-091914', 'etd-11062008-105226', 'etd-0903103-151457', 'etd-04202012-195901', 'etd-07042016-001719', 'etd-07112016-085209', 'etd-11132014-163221', 'etd-04142011-135915', 'etd-05312013-085359', 'etd-11032009-142431', 'etd-07082005-164140', 'etd-03012016-131829', 'etd-04122013-154240', 'etd-06232011-173240', 'etd-04092015-111316', 'etd-06022011-171502', 'etd-04192013-130821', 'etd-0718101-153927', 'etd-10122011-115219', 'etd-11172014-072934', 'etd-1105103-140728', 'etd-07132005-141738', 'etd-11152014-101222', 'etd-11082015-141227', 'etd-08292006-121153', 'etd-06302009-212437', 'etd-0710103-102938', 'etd-04082008-090416', 'etd-04032007-195119', 'etd-1112103-133929', 'etd-01252012-153306', 'etd-12162009-145521', 'etd-08272008-000329', 'etd-1115101-142722', 'etd-06212012-212936', 'etd-11042015-191214', 'etd-10282004-201637', 'etd-11162015-115106', 'etd-01242006-093828', 'etd-04042012-211349', 'etd-05272016-184155', 'etd-01282004-145119', 'etd-06252010-160336', 'etd-11102011-181211', 'etd-07072013-175702', 'etd-04072014-022058', 'etd-04212010-135523', 'etd-11072014-012029', 'etd-04072016-123302', 'etd-11102009-144247', 'etd-11112013-104001', 'etd-01202009-150026', 'etd-0602103-131314', 'etd-1113102-162549', 'etd-06172016-100051', 'etd-09252011-211450', 'etd-03152004-113133', 'etd-06262015-153612', 'etd-11052013-095415', 'etd-1113102-131008', 'etd-04012004-202528', 'etd-04112014-230830', 'etd-06242010-172924', 'etd-11132012-121722', 'etd-01202007-193731', 'etd-06232015-090030', 'etd-12082005-112815', 'etd-10132011-154436', 'etd-06122012-132459', 'etd-06092010-123919', 'etd-04102007-160616', 'etd-07092015-133223', 'etd-10302015-120951', 'etd-06282011-161247', 'etd-11152005-173234', 'etd-04092014-204235', 'etd-03092006-163704', 'etd-05282015-220211', 'etd-03212015-032141', 'etd-04232012-094645', 'etd-04142014-102134', 'etd-04112016-145800', 'etd-11112013-122916', 'etd-12142006-110928', 'etd-0813103-164024', 'etd-0619103-155815', 'etd-07102015-092030', 'etd-01122015-174341', 'etd-07032009-232128', 'etd-06092011-153355', 'etd-0117102-140940', 'etd-03312014-093050', 'etd-07012005-183857', 'etd-11052007-165817', 'etd-01162014-201105', 'etd-11092015-172715', 'etd-06022011-114724', 'etd-07062015-135614', 'etd-06222015-135811', 'etd-04082004-162323', 'etd-06102009-104808', 'etd-06132005-121856', 'etd-11122015-205316', 'etd-07052011-114408', 'etd-10282014-171514', 'etd-11032009-160356', 'etd-07032005-090208', 'etd-11012010-125011', 'etd-04102016-215548', 'etd-04262012-113406', 'etd-07082014-141723', 'etd-0410102-134451', 'etd-10212004-135921', 'etd-06092008-142228', 'etd-04182011-165249', 'etd-10302015-094318', 'etd-07052004-155122', 'etd-11152006-194212', 'etd-04242012-160655', 'etd-04242012-150651', 'etd-11132006-222452', 'etd-04072006-091838', 'etd-04152013-074355', 'etd-03292006-140347', 'etd-01112008-180517', 'etd-1108101-143050', 'etd-07072004-131623', 'etd-05192014-150835', 'etd-11152010-131223', 'etd-02212004-111053', 'etd-04212010-110316', 'etd-03222016-104748', 'etd-06282012-222835', 'etd-03162006-135622', 'etd-02182015-220355', 'etd-04082008-095304', 'etd-04092015-154852', 'etd-05302013-203418', 'etd-1112101-150039', 'etd-06082011-054344', 'etd-11152007-155610', 'etd-1112102-211702', 'etd-04092015-155243', 'etd-04152009-092337', 'etd-11072013-093630', 'etd-11172013-232152', 'etd-03232005-161807', 'etd-01192015-011054', 'etd-01162009-215521', 'etd-10142009-154753', 'etd-07142005-114344', 'etd-0417102-220244', 'etd-07072014-142859', 'etd-02262004-111054', 'etd-09152008-112337', 'etd-07142014-122759', 'etd-10312014-133428', 'etd-11032015-162218', 'etd-07112005-203920', 'etd-11172013-164518', 'etd-07072013-184633', 'etd-11142005-232641', 'etd-03272008-092832', 'etd-06032011-002259', 'etd-11142008-112549', 'etd-1110103-111641', 'etd-11092011-081802', 'etd-1104103-130651', 'etd-06052011-215717', 'etd-08242010-124753', 'etd-05232007-214956', 'etd-03242011-201607', 'etd-10192006-091537', 'etd-04212011-165830', 'etd-04032007-145610', 'etd-04112014-203515', 'etd-07052007-091741', 'etd-03052015-153324', 'etd-07072015-163646', 'etd-04142005-121818', 'etd-06032014-162815', 'etd-10312014-000003', 'etd-01232007-002401', 'etd-03212016-125717', 'etd-04212010-084044', 'etd-11142006-125308', 'etd-07112016-130446', 'etd-07062010-155814', 'etd-01172004-133705', 'etd-07132005-110918', 'etd-07082005-043614', 'etd-01092004-162154', 'etd-1018101-000938', 'etd-11052011-100030', 'etd-0415102-025627', 'etd-11122010-113434', 'etd-04302012-163058', 'etd-1109103-231721', 'etd-05192014-201105', 'etd-06122006-154814', 'etd-04132005-202304', 'etd-1111102-155240', 'etd-1115101-202112', 'etd-07122006-205835', 'etd-11142006-103340', 'etd-11152012-130619', 'etd-04022016-193042', 'etd-1029103-145352', 'etd-06092010-152627', 'etd-10282006-234836', 'etd-11052014-163845', 'etd-07062005-114730', 'etd-11072014-180014', 'etd-06192008-090345', 'etd-02112007-151028', 'etd-11132007-101101', 'etd-06052012-103811', 'etd-11102012-124519', 'etd-07102013-000251', 'etd-04132009-091738', 'etd-11122008-115728', 'etd-10192010-151247', 'etd-07092008-153016', 'etd-06242010-172854', 'etd-07072009-142414', 'etd-04042008-112837', 'etd-0302102-175245', 'etd-07052014-002034', 'etd-07142014-160251', 'etd-07052011-151716', 'etd-07122006-171959', 'etd-01082004-115607', 'etd-11112015-211505', 'etd-11152005-184410', 'etd-07112006-163449', 'etd-06092009-162436', 'etd-04032006-201317', 'etd-06102014-115515', 'etd-1128101-092913', 'etd-09062006-123248', 'etd-08082013-121135', 'etd-07132005-113114', 'etd-08202005-215619', 'etd-11112015-120704', 'etd-04052005-111446', 'etd-07102015-125551', 'etd-0807102-094558', 'etd-07092012-230732', 'etd-11052009-083906', 'etd-1008102-162013', 'etd-01262011-153300', 'etd-07072005-001118', 'etd-11102008-115238', 'etd-11052013-042107', 'etd-06112008-095515', 'etd-04042016-165633', 'etd-11102009-033317', 'etd-07062011-150935', 'etd-07042014-112538', 'etd-07122007-155612', 'etd-05312012-093118', 'etd-04192012-131025', 'etd-07142005-231924', 'etd-06282014-112233', 'etd-07022015-160056', 'etd-0711102-133722', 'etd-04082008-110148', 'etd-11122008-074526', 'etd-0611103-164757', 'etd-04152004-123641', 'etd-05182005-154832', 'etd-04142014-035621', 'etd-01232012-163905', 'etd-07052015-201659', 'etd-04272011-135511', 'etd-06222011-170536', 'etd-07072013-222509', 'etd-05192004-145930', 'etd-07292009-202203', 'etd-1031102-161824', 'etd-08012005-183105', 'etd-07022014-115847', 'etd-0702103-012837', 'etd-11162007-084227', 'etd-06062012-085909', 'etd-04092007-215754', 'etd-07232013-143545', 'etd-07022008-090602', 'etd-07112007-143905', 'etd-10282009-144508', 'etd-06032014-190910', 'etd-08202015-221535', 'etd-07032013-181824', 'etd-11042014-161235', 'etd-07122006-093251', 'etd-09022009-100408', 'etd-03082010-094758', 'etd-04102007-142100', 'etd-01252010-220828', 'etd-04042004-154851', 'etd-01212015-094504', 'etd-07052016-075046', 'etd-09082005-085238', 'etd-05172016-095925', 'etd-01172007-191718', 'etd-05252009-192847', 'etd-04162010-184914', 'etd-08152008-203703', 'etd-06132007-220514', 'etd-08272012-141933', 'etd-03072016-120111', 'etd-03282006-195009', 'etd-11102009-150925', 'etd-03312014-142636', 'etd-10272015-132916', 'etd-04262012-164053', 'etd-04292011-100054', 'etd-04242012-161226', 'etd-07052010-224427', 'etd-03252008-165251', 'etd-04032013-120222', 'etd-0815103-141412', 'etd-06022014-114518', 'etd-0706102-200545', 'etd-0528103-095224', 'etd-11062004-081538', 'etd-10292010-133201', 'etd-07082010-194030', 'etd-11102004-120902', 'etd-11152010-112728', 'etd-07052016-102537', 'etd-07032015-161725', 'etd-0710103-222546', 'etd-04252011-164525', 'etd-0409103-145414', 'etd-06302011-213826', 'etd-07132007-014052', 'etd-08042011-230059', 'etd-01222013-102917', 'etd-11072014-111925', 'etd-09222015-193221', 'etd-07022010-111320', 'etd-04072016-165145', 'etd-0904103-104529', 'etd-0903103-141852', 'etd-08042016-141900', 'etd-04112008-082433', 'etd-07062011-183918', 'etd-01272014-102705', 'etd-06022014-204241', 'etd-03312015-144425', 'etd-08242015-174230', 'etd-01102012-174719', 'etd-0702103-114837', 'etd-04132007-093916', 'etd-0903103-093639', 'etd-01252006-163656', 'etd-10212009-110239', 'etd-0418102-125804', 'etd-04172009-123540', 'etd-11042011-161820', 'etd-03082012-073140', 'etd-08182011-160213', 'etd-05272013-105338', 'etd-04042004-210830', 'etd-11162005-150630', 'etd-06022006-122847', 'etd-06202016-131938', 'etd-11112009-111519', 'etd-06052007-145138', 'etd-06222015-135942', 'etd-06282005-170859', 'etd-07062009-151205', 'etd-04192010-142435', 'etd-03232005-145224', 'etd-06032005-122947', 'etd-04192011-164932', 'etd-11082013-124052', 'etd-04152010-174523', 'etd-06072007-073810', 'etd-0710102-192931', 'etd-06022004-155712', 'etd-04212010-140623', 'etd-05282014-134705', 'etd-06052012-163626', 'etd-07022015-212930', 'etd-07072016-135255', 'etd-08072014-205316', 'etd-11102008-100331', 'etd-11182013-105931', 'etd-04122005-204341', 'etd-04102014-151839', 'etd-06132013-114654', 'etd-03272006-103957', 'etd-04012006-145623', 'etd-04082010-181645', 'etd-08232015-202905', 'etd-07082014-203635', 'etd-0522103-115259', 'etd-01022013-205953', 'etd-04132010-115519', 'etd-04242012-134855', 'etd-1114102-151515', 'etd-07022006-123237', 'etd-11162010-100024', 'etd-07102007-163043', 'etd-04082016-153408', 'etd-06042013-134734', 'etd-11012008-005747', 'etd-12102012-072732', 'etd-04092007-093933', 'etd-06182010-150903', 'etd-06012015-161114', 'etd-11142006-175046', 'etd-05202008-172327', 'etd-04272004-182920', 'etd-07052004-112200', 'etd-11112008-180201', 'etd-03252004-134502', 'etd-04142010-204514', 'etd-04242012-152215', 'etd-11112004-162220', 'etd-07092008-211200', 'etd-06022015-174920', 'etd-01202006-163647', 'etd-0609103-140047', 'etd-06012010-164011', 'etd-0711102-174326', 'etd-06032013-163249', 'etd-04162004-073552', 'etd-10082006-190542', 'etd-11152013-113101', 'etd-11112008-130652', 'etd-08122009-154515', 'etd-04272011-165903', 'etd-06192013-221502', 'etd-04072016-172419', 'etd-04122010-194308', 'etd-07112007-170251', 'etd-10262006-154509', 'etd-05162009-154138', 'etd-01272011-133441', 'etd-0414102-144220', 'etd-07052006-110113', 'etd-01202006-082858', 'etd-11132014-120939', 'etd-07132005-172811', 'etd-04062015-100151', 'etd-04212010-120501', 'etd-11022011-164027', 'etd-11062007-181936', 'etd-11042010-140435', 'etd-01262011-120523', 'etd-06292016-100109', 'etd-05192013-150547', 'etd-0524102-130801', 'etd-10222009-151211', 'etd-11162006-233014', 'etd-10302011-181759', 'etd-04142005-161744', 'etd-01032012-155853', 'etd-04132005-102650', 'etd-04232010-005628', 'etd-0711102-111720', 'etd-07302015-122241', 'etd-02082007-162541', 'etd-10272011-063500', 'etd-05292015-151433', 'etd-06202011-085953', 'etd-11122008-174848', 'etd-01102007-171455', 'etd-08202007-204147', 'etd-11252013-092816', 'etd-11162005-141003', 'etd-06082004-141808', 'etd-08272009-173846', 'etd-01202006-182745', 'etd-04102013-065554', 'etd-06072004-194052', 'etd-11072005-095808', 'etd-07062004-114807', 'etd-01192006-191441', 'etd-10252011-122025', 'etd-06102012-133327', 'etd-08302011-123352', 'etd-09042013-120004', 'etd-07112016-141349', 'etd-06132005-001841', 'etd-03302004-132906', 'etd-01202009-104542', 'etd-11192013-155138', 'etd-04042016-081410', 'etd-07092012-155952', 'etd-01212009-095627', 'etd-04152005-111817', 'etd-05302014-155220', 'etd-07042014-141943', 'etd-06082011-145228', 'etd-03132015-132345', 'etd-05272011-065546', 'etd-01222004-110049', 'etd-07032013-142703', 'etd-06202012-112510', 'etd-06262007-200349', 'etd-07082015-224106', 'etd-10172015-095434', 'etd-04072016-131354', 'etd-05172016-144822', 'etd-04012015-134023', 'etd-11112008-203316', 'etd-11092010-223757', 'etd-11092012-145532', 'etd-08142009-141932', 'etd-01092015-211259', 'etd-02192015-200616', 'etd-1023102-173923', 'etd-04152010-164531', 'etd-07082009-141346', 'etd-01252006-135443', 'etd-03142005-155707', 'etd-11052014-192543', 'etd-1031103-154611', 'etd-01182008-152053', 'etd-04152013-122830', 'etd-02292016-112634', 'etd-04112016-150034', 'etd-03312011-113625', 'etd-04262011-133651', 'etd-07092009-143433', 'etd-06052014-124029', 'etd-03102006-234119', 'etd-07122015-201805', 'etd-10182013-111759', 'etd-10022010-115320', 'etd-03312009-143854', 'etd-06132013-143618', 'etd-04052013-121916', 'etd-07112014-212342', 'etd-08302011-110023', 'etd-09182008-131849', 'etd-07112016-060702', 'etd-11122004-054504', 'etd-04052006-143046', 'etd-06302016-134009', 'etd-11092009-121503', 'etd-07032012-120846', 'etd-04082013-134816', 'etd-03232010-132628', 'etd-07022012-221936', 'etd-04282011-010230', 'etd-03162016-205307', 'etd-04112007-113326', 'etd-06212006-151734', 'etd-0612103-213510', 'etd-01272004-121446', 'etd-04082009-143703', 'etd-03182009-161153', 'etd-06132013-160635', 'etd-04092009-003951', 'etd-09182010-161714', 'etd-10222014-120255', 'etd-11052013-134945', 'etd-11132012-152821', 'etd-07102007-114010', 'etd-07022010-172549', 'etd-05122011-222119', 'etd-07042015-191603', 'etd-01272004-085638', 'etd-07082014-173616', 'etd-04202010-122318', 'etd-11172014-155433', 'etd-04042006-154742', 'etd-06022015-223507', 'etd-04202005-161203', 'etd-0123103-183035', 'etd-10202009-111745', 'etd-04042015-140752', 'etd-10302009-233227', 'etd-11012004-135458', 'etd-0129103-042127', 'etd-12142010-113353', 'etd-11062007-210550', 'etd-07062016-105859', 'etd-06222007-194016', 'etd-01192007-091902', 'etd-04072009-135536', 'etd-04182011-161619', 'etd-0326103-111214', 'etd-04102008-161802', 'etd-04212010-172429', 'etd-04102014-140003', 'etd-11092006-201044', 'etd-0711103-104039', 'etd-04212011-163312', 'etd-11132005-195819', 'etd-04082010-190439', 'etd-06032009-172258', 'etd-11142008-114837', 'etd-01182006-203351', 'etd-07082009-144920', 'etd-11142012-215948', 'etd-02112014-150650', 'etd-11152007-182141', 'etd-12062012-162954', 'etd-06132006-161041', 'etd-11092004-135151', 'etd-07092007-100545', 'etd-07042012-093921', 'etd-08172012-091735', 'etd-03222006-164406', 'etd-06262015-151156', 'etd-04102014-232102', 'etd-07032007-091344', 'etd-11152013-182854', 'etd-04012015-200844', 'etd-10252012-161342', 'etd-04152013-124900', 'etd-0701103-171552', 'etd-06242013-140724', 'etd-06102008-122209', 'etd-06262007-183823', 'etd-06072013-163304', 'etd-0409103-184148', 'etd-0416102-005632', 'etd-11122004-092630', 'etd-1106103-172944', 'etd-07012009-193032', 'etd-04212010-130438', 'etd-10252006-144137', 'etd-06022005-225849', 'etd-0529103-123537', 'etd-01262015-003841', 'etd-11132008-173703', 'etd-04042005-092947', 'etd-07112005-201540', 'etd-06062014-145406', 'etd-06262012-224105', 'etd-0324102-225153', 'etd-04142014-142446', 'etd-06252004-141320', 'etd-1107103-100118', 'etd-07112008-114743', 'etd-06092004-150512', 'etd-02012013-004101', 'etd-04112005-150114', 'etd-06162005-155027', 'etd-04152013-144121', 'etd-01262005-072743', 'etd-0902103-202649', 'etd-06072013-161222', 'etd-10122015-124021', 'etd-1021103-111509', 'etd-04182012-091506', 'etd-08172004-174948', 'etd-0704102-112124', 'etd-04072015-123410', 'etd-07092014-113110', 'etd-04052015-120510', 'etd-11032015-150204', 'etd-08182015-155225', 'etd-04022009-185330', 'etd-07122015-135658', 'etd-11082011-160845', 'etd-06292016-091134', 'etd-02012016-171039', 'etd-07112014-083403', 'etd-09302011-133240', 'etd-04092004-115604', 'etd-05262014-170723', 'etd-04112011-173742', 'etd-04112013-125749', 'etd-07082009-210213', 'etd-04202012-073419', 'etd-07132006-023633', 'etd-04032007-173430', 'etd-01142013-181330', 'etd-06262007-190651', 'etd-04092009-165026', 'etd-11142006-132217', 'etd-04102016-231502', 'etd-07122006-141758', 'etd-04152010-230541', 'etd-03112011-110131', 'etd-0408103-120732', 'etd-04202011-092524', 'etd-0402103-165113', 'etd-07082013-105553', 'etd-04152013-162204', 'etd-04032007-163739', 'etd-11162007-084516', 'etd-03302006-202033', 'etd-06272008-112653', 'etd-11122004-113829', 'etd-07102007-081138', 'etd-07072013-111640', 'etd-07092010-102821', 'etd-06082011-090019', 'etd-07022015-181918', 'etd-03152006-101730', 'etd-11112004-174155', 'etd-1112101-150225', 'etd-03072008-123039', 'etd-12202008-115800', 'etd-04112013-150610', 'etd-04292011-030741', 'etd-07022008-050958', 'etd-07042012-223853', 'etd-10232013-142737', 'etd-09292004-174714', 'etd-11172014-151348', 'etd-11162004-084402', 'etd-06122006-100903', 'etd-07032009-131245', 'etd-07122006-235055', 'etd-11112011-085618', 'etd-07082016-173732', 'etd-08312005-163644', 'etd-0415102-230223', 'etd-04212011-160643', 'etd-06252013-173807', 'etd-04112005-201353', 'etd-0624103-122017', 'etd-04152010-184626', 'etd-0221103-161521', 'etd-11102011-105354', 'etd-11172014-122205', 'etd-07102009-105825', 'etd-11052012-104924', 'etd-0708102-155615', 'etd-11162006-154047', 'etd-11192015-140114', 'etd-01192016-134111', 'etd-06142016-133552', 'etd-04092009-111227', 'etd-11132007-163128', 'etd-06272011-200559', 'etd-11142013-175910', 'etd-0822103-011713', 'etd-0414103-172013', 'etd-04142010-134016', 'etd-07102008-142709', 'etd-06032014-152951', 'etd-01162014-093302', 'etd-0822103-173901', 'etd-10082012-141849', 'etd-04122016-073623', 'etd-11122010-155346', 'etd-08112009-145503', 'etd-07112011-161646', 'etd-04152004-153146', 'etd-04092007-181146', 'etd-04252012-225800', 'etd-05042008-181650', 'etd-06262015-154503', 'etd-01232012-115608', 'etd-07012011-123447', 'etd-06232016-083944', 'etd-06092008-223231', 'etd-04102015-165715', 'etd-10062006-233328', 'etd-04222010-125635', 'etd-05272009-222440', 'etd-04162010-101313', 'etd-08192013-205130', 'etd-11142014-171500', 'etd-06192015-111023', 'etd-04032007-171734', 'etd-01082007-093643', 'etd-0127103-115005', 'etd-11042011-140512', 'etd-11122010-160001', 'etd-07042011-232407', 'etd-0409103-223522', 'etd-05112015-140927', 'etd-07052011-175139', 'etd-0410103-161749', 'etd-04162010-001906', 'etd-07062016-113754', 'etd-11142005-143000', 'etd-07032012-062139', 'etd-10242011-135130', 'etd-11132006-104057', 'etd-05302008-163309', 'etd-04092015-082757', 'etd-01212014-155100', 'etd-06292014-114911', 'etd-06272008-064129', 'etd-11132008-162153', 'etd-03102014-110622', 'etd-11052005-195648', 'etd-06232010-112745', 'etd-01192010-104211', 'etd-03312006-160644', 'etd-11152010-145059', 'etd-10222014-154103', 'etd-07082009-133446', 'etd-06052007-123527', 'etd-11092012-170636', 'etd-06302009-171138', 'etd-04172009-095944', 'etd-01212010-151353', 'etd-11082005-140856', 'etd-04022014-193809', 'etd-07112014-143520', 'etd-07112012-204148', 'etd-04062009-003007', 'etd-09102008-161838', 'etd-03172010-212949', 'etd-06262012-091949', 'etd-06302014-190504', 'etd-11152006-094043', 'etd-06112014-154108', 'etd-04022015-094325', 'etd-06292016-100558', 'etd-05172009-164115', 'etd-05272012-174438', 'etd-09222004-101511', 'etd-01122015-170634', 'etd-04022014-113455', 'etd-1018102-094954', 'etd-07012014-080052', 'etd-04132009-162140', 'etd-10062015-114001', 'etd-01262012-001013', 'etd-07102009-114004', 'etd-01122006-120717', 'etd-03272013-153428', 'etd-11122007-192344', 'etd-06292010-170315', 'etd-07052012-140651', 'etd-11152006-152550', 'etd-11182010-123842', 'etd-07082013-161802', 'etd-04152011-005659', 'etd-07012013-193735', 'etd-07012015-133608', 'etd-03282010-124349', 'etd-07082013-101522', 'etd-04072015-172511', 'etd-03062009-093856', 'etd-04082009-170649', 'etd-08252011-163658', 'etd-11142014-160036', 'etd-03312006-234733', 'etd-1023102-140854', 'etd-07112012-110049', 'etd-04072005-082039', 'etd-06152011-110611', 'etd-02282015-131531', 'etd-12112003-143624', 'etd-07062016-170458', 'etd-11142006-163133', 'etd-11132008-144238', 'etd-03052007-103140', 'etd-06272011-121728', 'etd-04132015-111803', 'etd-06022011-110116', 'etd-03082016-111401', 'etd-07132012-105303', 'etd-0123102-140805', 'etd-11022011-135030', 'etd-04082008-092615', 'etd-04112005-220637', 'etd-07142005-180715', 'etd-11082013-170304', 'etd-03092015-132959', 'etd-10102012-182014', 'etd-11082011-115541', 'etd-1113102-164314', 'etd-03022006-111555', 'etd-10292010-135500', 'etd-07072008-154718', 'etd-04132007-093749', 'etd-04052015-180218', 'etd-01192007-151756', 'etd-06232012-104447', 'etd-04062015-100633', 'etd-05312013-084246', 'etd-11022011-091541', 'etd-09242009-081657', 'etd-06142005-172246', 'etd-11162005-221527', 'etd-06082011-141500', 'etd-08172010-154004', 'etd-04012015-164717', 'etd-11012006-162335', 'etd-06282015-211437', 'etd-11042014-144333', 'etd-12092003-121423', 'etd-04112016-135419', 'etd-06222007-083116', 'etd-11162012-103236', 'etd-07092014-080700', 'etd-04092013-103115', 'etd-07062007-110742', 'etd-04102013-025926', 'etd-06022016-150032', 'etd-0130103-102538', 'etd-10172013-122815', 'etd-01192015-055258', 'etd-11182010-180558', 'etd-08272004-080427', 'etd-07122006-135145', 'etd-06182013-205633', 'etd-03112015-111425', 'etd-04012005-150732', 'etd-11102012-144639', 'etd-0708102-141353', 'etd-02262016-201516', 'etd-04052012-125933', 'etd-11122010-153530', 'etd-11072014-130925', 'etd-03192010-102911', 'etd-07122012-185135', 'etd-1111102-110715', 'etd-03252010-094110', 'etd-05182004-144851', 'etd-07092014-142804', 'etd-07092008-073929', 'etd-09242011-151628', 'etd-06142016-121104', 'etd-11102004-101210', 'etd-03152011-150434', 'etd-0610103-121948', 'etd-07202011-163444', 'etd-11122015-163609', 'etd-0523102-211853', 'etd-06302016-154343', 'etd-07132006-055232', 'etd-04202010-231414', 'etd-0128103-212843', 'etd-07092014-152032', 'etd-04052006-105548', 'etd-06042015-221103', 'etd-03312010-083400', 'etd-11092011-110630', 'etd-07062015-212900', 'etd-0228103-142646', 'etd-11182013-120418', 'etd-05212015-210859', 'etd-04112016-122514', 'etd-10302006-120056', 'etd-11152012-040803', 'etd-0517102-111730', 'etd-07022009-000702', 'etd-06252013-143105', 'etd-04172010-175506', 'etd-04112016-154606', 'etd-04032006-093417', 'etd-11032004-070937', 'etd-10172013-230022', 'etd-10302013-142308', 'etd-0611103-153359', 'etd-11182013-150645', 'etd-07032008-112156', 'etd-03272008-152354', 'etd-0115103-183313', 'etd-04082015-125133', 'etd-08132013-190600', 'etd-07072008-105049', 'etd-04232010-092757', 'etd-06262014-234749', 'etd-07072006-223611', 'etd-07072010-210948', 'etd-03222013-143623', 'etd-06202011-094735', 'etd-1112103-221719', 'etd-06292012-093114', 'etd-05252005-133708', 'etd-04042016-010804', 'etd-03272008-094704', 'etd-01222008-100811', 'etd-04242011-190837', 'etd-06182010-104212', 'etd-04022013-174313', 'etd-0125102-111357', 'etd-07082013-113917', 'etd-11142007-183147', 'etd-0709103-163907', 'etd-08242011-210636', 'etd-11162015-140124', 'etd-06092014-143548', 'etd-01132004-114719', 'etd-07122015-170830', 'etd-11132007-235550', 'etd-04112016-150136', 'etd-07102007-194806', 'etd-1113102-210422', 'etd-11072014-161403', 'etd-04092008-102559', 'etd-07082011-074944', 'etd-11122007-132025', 'etd-03052008-075120', 'etd-11152005-190700', 'etd-01152016-104502', 'etd-07012009-065845', 'etd-10052014-073714', 'etd-05232013-110656', 'etd-05092006-145816', 'etd-03252016-202739', 'etd-10312012-113031', 'etd-04252012-172650', 'etd-03302016-075902', 'etd-06112005-183452', 'etd-12152010-142448', 'etd-0613102-124530', 'etd-12202010-185148', 'etd-06222006-090232', 'etd-06092009-110016', 'etd-10222012-094356', 'etd-04082010-105507', 'etd-1105102-112947', 'etd-0605103-093150', 'etd-04022007-085414', 'etd-10302004-154714', 'etd-11152006-135338', 'etd-05312006-181403', 'etd-06232011-111335', 'etd-0129103-160430', 'etd-07082009-154838', 'etd-11072007-162300', 'etd-04082013-145400', 'etd-02232011-110231', 'etd-11142007-122230', 'etd-07102014-090253', 'etd-05062005-163801', 'etd-03312004-154458', 'etd-0401103-145828', 'etd-06102008-101538', 'etd-10152012-212259', 'etd-07072009-151947', 'etd-04082015-213720', 'etd-01182012-173733', 'etd-04062006-152406', 'etd-03132013-131735', 'etd-04072010-012548', 'etd-04122013-161615', 'etd-10242004-185412', 'etd-03272007-172212', 'etd-04102014-101643', 'etd-11182013-153127', 'etd-07172009-113240', 'etd-07122006-185335', 'etd-01252012-101630', 'etd-09122010-101812', 'etd-04022007-161003', 'etd-01282010-093746', 'etd-06272016-084628', 'etd-06222006-123926', 'etd-07102008-142045', 'etd-11032015-124246', 'etd-03072012-101759', 'etd-0128103-083757', 'etd-10312014-135615', 'etd-11182013-144258', 'etd-03312006-175531', 'etd-08192008-104037', 'etd-11092009-135336', 'etd-04242012-215407', 'etd-06122006-135610', 'etd-11162015-102110', 'etd-04152005-100148', 'etd-04112007-175255', 'etd-04142004-083041', 'etd-07142006-111807', 'etd-1017102-103638', 'etd-04062016-081038', 'etd-11022009-153636', 'etd-04112010-174954', 'etd-01252005-152153', 'etd-03302006-210746', 'etd-11102006-195326', 'etd-04112010-162304', 'etd-04132014-153657', 'etd-03292005-135153', 'etd-07062014-123636', 'etd-11032010-174223', 'etd-08262004-112107', 'etd-03142014-095638', 'etd-0911101-173205', 'etd-04042011-181700', 'etd-06082007-114534', 'etd-11112013-073701', 'etd-11102006-083842', 'etd-0416102-172102', 'etd-06112014-140552', 'etd-03232004-234942', 'etd-04052015-094252', 'etd-04072008-200806', 'etd-11072013-150706', 'etd-01242005-183547', 'etd-07102006-145935', 'etd-07112008-115020', 'etd-04112016-142824', 'etd-11122010-182021', 'etd-07132006-150159', 'etd-04152010-114242', 'etd-0323103-094340', 'etd-05252012-132121', 'etd-1104102-104609', 'etd-06172004-154143', 'etd-04092009-223000', 'etd-04112005-105420', 'etd-04032013-175107', 'etd-04262012-182344', 'etd-06062016-112254', 'etd-06032011-172223', 'etd-07012013-173512', 'etd-08292014-122540', 'etd-07052012-115942', 'etd-0708102-140449', 'etd-11132007-103408', 'etd-04142009-223603', 'etd-0914102-150023', 'etd-11072015-133045', 'etd-04082011-132628', 'etd-10212010-150253', 'etd-05252006-164039', 'etd-11022004-113244', 'etd-11182010-150952', 'etd-0130103-053846', 'etd-05312012-215918', 'etd-06282007-200557', 'etd-11062009-115056', 'etd-06102014-151557', 'etd-1107102-200952', 'etd-06302009-161650', 'etd-03272008-115742', 'etd-04142011-094633', 'etd-07052011-160554', 'etd-07122007-092242', 'etd-04142005-172650', 'etd-11082004-113624', 'etd-03212007-114019', 'etd-11172005-113959', 'etd-06232006-095333', 'etd-04012015-135304', 'etd-1030103-161928', 'etd-07102014-120433', 'etd-11112006-104805', 'etd-0413102-182717', 'etd-11132013-135946', 'etd-04082015-162835', 'etd-07032007-113109', 'etd-0410103-215703', 'etd-12212015-122506', 'etd-0920102-143700', 'etd-04162010-182758', 'etd-04102007-200530', 'etd-04152004-123145', 'etd-11142005-124444', 'etd-04042008-145222', 'etd-10272010-151848', 'etd-07052011-144312', 'etd-03262006-110723', 'etd-12162005-090458', 'etd-04162009-205403', 'etd-06182010-093527', 'etd-07072013-161459', 'etd-07022009-023950', 'etd-06152005-122402', 'etd-01262015-160231', 'etd-10042013-162109', 'etd-04132004-191929', 'etd-06152011-203149', 'etd-0129102-172747', 'etd-03312006-164516', 'etd-01192016-132552', 'etd-04082005-095548', 'etd-04132014-163114', 'etd-06272011-205203', 'etd-0708103-131111', 'etd-07012009-175052', 'etd-10252015-165339', 'etd-04132005-192550', 'etd-04012016-120158', 'etd-0611102-122027', 'etd-08162013-214748', 'etd-04032013-141712', 'etd-07132015-101053', 'etd-04012008-223515', 'etd-04262012-170945', 'etd-07102015-134336', 'etd-11152012-094000', 'etd-11122008-180920', 'etd-11052004-083332', 'etd-04112016-083034', 'etd-10152007-103900', 'etd-07082010-183757', 'etd-07272012-154005', 'etd-01222014-142700', 'etd-11052004-002931', 'etd-04062015-152523', 'etd-07142014-090422', 'etd-04142016-153947', 'etd-07062016-153242', 'etd-11102006-161848', 'etd-11162010-155432', 'etd-06082011-160513', 'etd-04272011-214101', 'etd-03282011-163903', 'etd-08252004-155118', 'etd-01092016-212103', 'etd-10272008-132524', 'etd-11132014-083100', 'etd-03212008-114613', 'etd-05312013-134128', 'etd-0120102-223858', 'etd-0708103-073122', 'etd-07052011-094524', 'etd-04192010-144738', 'etd-06162016-172212', 'etd-06172016-184834', 'etd-04272011-180006', 'etd-06082004-172113', 'etd-07032012-221937', 'etd-11082007-204227', 'etd-0515103-151156', 'etd-09242012-121250', 'etd-1110103-090904', 'etd-01172005-201729', 'etd-04292013-150843', 'etd-11042010-185701', 'etd-04052016-173428', 'etd-04152009-093730', 'etd-02212008-165645', 'etd-04112011-121215', 'etd-04122014-224820', 'etd-01272011-134931', 'etd-04132015-114500', 'etd-07052011-113126', 'etd-03222013-161556', 'etd-04032014-120538', 'etd-11162015-153607', 'etd-07072010-162419', 'etd-07132015-100708', 'etd-06262008-135641', 'etd-04162010-221600', 'etd-07082010-164601', 'etd-07132007-091706', 'etd-04182011-150236', 'etd-07022013-083948', 'etd-11162012-110609', 'etd-04162009-154845', 'etd-08172010-120522', 'etd-11152010-220929', 'etd-04102007-113758', 'etd-12222015-104840', 'etd-0124102-102623', 'etd-10282004-114654', 'etd-10292011-122541', 'etd-10192011-185619', 'etd-04052010-231449', 'etd-04142014-094749', 'etd-07032012-182337', 'etd-04012008-161207', 'etd-01062016-145336', 'etd-03302006-122704', 'etd-06162005-082538', 'etd-08172015-135151', 'etd-0613102-141016', 'etd-0612103-092453', 'etd-04132013-022208', 'etd-04152013-152630', 'etd-04272011-144941', 'etd-0603102-142641', 'etd-0708102-152502', 'etd-03042005-094443', 'etd-06232009-122944', 'etd-05192011-113732', 'etd-04022008-111035', 'etd-11072006-153104', 'etd-04272011-164727', 'etd-06132016-123923', 'etd-10242005-122322', 'etd-01202012-083443', 'etd-08212015-140535', 'etd-11022011-160301', 'etd-05262011-140000', 'etd-04082014-202541', 'etd-08252012-152026', 'etd-0702103-100105', 'etd-11112011-090013', 'etd-07122012-153347', 'etd-04122007-183135', 'etd-03022004-093241', 'etd-05202004-160106', 'etd-04242012-101116', 'etd-1111103-131337', 'etd-03302006-155731', 'etd-07072016-142902', 'etd-01232007-095304', 'etd-07112008-100442', 'etd-01132005-144724', 'etd-04132010-110934', 'etd-04192012-170451', 'etd-11062008-130552', 'etd-01162007-180732', 'etd-04262012-150423', 'etd-01202005-163419', 'etd-05252012-135210', 'etd-0729103-130157', 'etd-10252006-134555', 'etd-0114103-115012', 'etd-1108102-093615', 'etd-11042011-122517', 'etd-04092013-093002', 'etd-08192015-104156', 'etd-0128102-144113', 'etd-03212006-164209', 'etd-06062013-225727', 'etd-04112016-144544', 'etd-12212004-191037', 'etd-06292011-153733', 'etd-04052013-164820', 'etd-10242014-084819', 'etd-04132015-125534', 'etd-11112009-210447', 'etd-0828103-105656', 'etd-08252015-181738', 'etd-06122014-154723', 'etd-06152011-094407', 'etd-10152011-114327', 'etd-07302013-133503', 'etd-04152009-120528', 'etd-04112011-102646', 'etd-01242005-174454', 'etd-08182004-161435', 'etd-04092016-225712', 'etd-0607102-185008', 'etd-0405103-153006', 'etd-0123102-151708', 'etd-08252010-143743', 'etd-0612102-164611', ])

In [ ]:
def turn_csvs_to_dict(filepaths):
    csv_dict = dict()
    for filepath in filepaths:
        with open(filepath, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=';', quotechar='"', escapechar='\\')
            for num, row in enumerate(spamreader):
                if num == 0:
                    headers = [*row]
                    CSVSheet = namedtuple('CSVSheet', headers)
                    continue
                item = CSVSheet(*row)
                csv_dict[item.urn] = item
    return csv_dict

In [ ]:
types_items_dict = dict()

for file_ending in ('advisors', 'desc', 'filepath', 'keyword'):
    filepaths = list()
    for folder in os.listdir('Sept2017/databasetables/'):
        filepath = os.path.join('Sept2017', 'databasetables', folder, '{}.csv'.format(file_ending))
        filepaths.append(filepath)
    types_items_dict[file_ending] = turn_csvs_to_dict(filepaths)

In [ ]:
sept_urns = set()

for file, csv_dict in types_items_dict.items():
    for urn in csv_dict:
        sept_urns.add(urn)

In [ ]:
len(sept_urns)

In [ ]:
types_items_dict.keys()

In [ ]:
keyword_filepaths = ['Sept2017/databasetables/{}/keyword.csv'.format(i) for i in ('avail', 'submitted', 'withheld')]
advisor_filepaths = ['Sept2017/databasetables/{}/advisors.csv'.format(i) for i in ('avail', 'submitted', 'withheld')]
filenames_filepaths = ['Sept2017/databasetables/{}/filepath.csv'.format(i) for i in ('avail', 'submitted', 'withheld')]
main_filepaths = ['Sept2017/databasetables/{}/desc.csv'.format(i) for i in ('avail', 'submitted', 'withheld')]

In [ ]:
urns_to_do = sept_urns - set(uploaded_urns)
main_sheet = parse_main_sheet(main_filepaths)
catalog_sheet = parse_catalog_sheet()
filenames_sheet = parse_filenames_sheet(filenames_filepaths)
keywords_sheet = parse_keywords_sheet(keyword_filepaths)
advisors_sheet = parse_advisors_sheet(advisor_filepaths)

In [ ]:
build_csv(urns_to_do, main_sheet, catalog_sheet, filenames_sheet, keywords_sheet, advisors_sheet)